In [ ]:
pip install ray[rllib] --upgrade

In [ ]:
import torch

print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())
print(torch.version.cuda)


In [ ]:
# This code prints out the path of the Python executable
import sys
print(sys.executable)

# It might be useful to know which Python version you're using as well
print(sys.version)


In [ ]:
%conda env list


In [ ]:
pip install py_dss_interface

In [ ]:
pip install matplotlib

In [ ]:
pip install stable_baselines3

In [ ]:
pip install gymnasium

In [ ]:
env = MultiAgentEnvCompatibility(original_env)

In [2]:


#Testing with partial observability of voltage violatiosn for each agent

import gymnasium as gym
from gymnasium import spaces
from gymnasium.spaces import Tuple, Dict

from ray.rllib.env.multi_agent_env import MultiAgentEnv
import numpy as np
from ray.rllib.algorithms.sac import SAC
from ray.rllib.algorithms.ppo import PPO
from ray.tune.registry import register_env
from ray import tune
import numpy as np

from py_dss_interface import DSSDLL
#import stable_baselines3
#from stable_baselines3 import SAC
import matplotlib.pyplot as plt
import csv
import pandas as pd
#from stable_baselines3 import A2C, DQN, PPO, TD3, SAC
import torch
import warnings
warnings.filterwarnings('ignore')
import logging

logging.basicConfig(level=logging.ERROR)





device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import time

start_time = time.time()



# Initialize OpenDSS
dss = DSSDLL(r"C:\Program Files\OpenDSS")
dss_file = r"D:\Alaa_Selim\123Bus\IEEE123Master.dss"
dss.text(f"compile [{dss_file}]")

# defining lengths for each segment
PV_KVAR_ACTION_LEN = 30
PV_KW_ACTION_LEN = 30
BESS_KW_ACTION_LEN = 30
TRANSFORMER_TAPS_ACTION_LEN = 1
CAPACITOR_ACTION_LEN = 4

# defining values for each segment
PV_KVAR_ACTION_LOW = -80
PV_KVAR_ACTION_HIGH = 80

PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 100

BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 100

TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1

CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1


class PowerSystemEnv(MultiAgentEnv):
    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file):
        super(PowerSystemEnv, self).__init__()

        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        self.ranked_buses  = [
                                '1', '7', '8', '13', '21', '23', '29', '250', '35', '40', '42', '55',
                                '56', '65', '76', '78', '66', '79', '81', '83', '91', '95', '100',
                                '197', '300', '110', '135', '160', '152', '610'
                            ]
        self.capacitor_names = ["C83", "C88a", "C90b", "C92c"]
        self.KWrated=100
        self.previous_reward = 0.0
        self.alpha = 0.1  #
         # Apply actions to PV systems and batteries
        for i in range(30):
            bus = self.ranked_buses[i]
            self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=100 Pmpp=80")
            self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=100 kVAR=0")
        
        with open(irradiance_csv_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            self.irradiance_profile = [float(row[0]) for row in reader]
    
        with open(load_profile_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader, None)  # Skip the header
            self.load_profile = [float(row[0]) for row in reader]
            
            
        self.action_space = spaces.Box(
            low=np.array(
                [PV_KVAR_ACTION_LOW]*PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_LOW]*PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_LOW]*BESS_KW_ACTION_LEN 
            ),
            high=np.array(
                [PV_KVAR_ACTION_HIGH]*PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_HIGH]*PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_HIGH]*BESS_KW_ACTION_LEN 
               
            ),
            dtype=np.float32
        )
            

     
        # Assuming observation space is the voltage at each bus
        self.observation_space = spaces.Box(low=0, high=2, shape=(278,), dtype=np.float32)  # Modified shape
            
         
        # Update the action and observation spaces for each agent
        self.action_space_dict = {
            f"agent_{i}": spaces.Box(
                low=np.concatenate(([PV_KW_ACTION_LOW] * 6, [PV_KVAR_ACTION_LOW] * 6, [BESS_KW_ACTION_LOW] * 6)),
                high=np.concatenate(([PV_KW_ACTION_HIGH] * 6, [PV_KVAR_ACTION_HIGH] * 6, [BESS_KW_ACTION_HIGH] * 6)),
                shape=(18,), dtype=np.float32) for i in range(5)
        }



        self.observation_space_dict = {}

        for i in range(4):  # For the first four agents
            self.observation_space_dict[f'agent_{i}'] = spaces.Box(low=0, high=2, shape=(50,), dtype=np.float32)

        # For the last agent
        self.observation_space_dict['agent_4'] = spaces.Box(low=0, high=2, shape=(78,), dtype=np.float32)

        
        self.current_step = 0

        
         # Initialize control step counter
        self.control_steps = 0

        # Maximum control steps allowed in one episode
        self.max_control_steps =int(2)  # for example
        
        



          # Define the agent IDs
        self._agent_ids = [f'agent_{i}' for i in range(5)]


        
   

        
    def _take_action(self, action_dict):
    # Define total actions per agent
        TOTAL_ACTIONS_PER_AGENT = 6 + 6 + 6

    # Iterate through the action_dict
        for agent_id, action in action_dict.items():
            agent_idx = int(agent_id.split('_')[1])

        # Determine action segments for this agent
            pv_kw_actions = action[:6]
            pv_kvar_actions = action[6:12]
            bess_kw_actions = action[12:18]

        # Index offset based on agent_idx for PV and battery control
            pv_idx_offset = agent_idx * 6
            battery_idx_offset = agent_idx * 6

        # Actions for PVs
            for idx, (kw_action_value, kvar_action_value) in enumerate(zip(pv_kw_actions, pv_kvar_actions)):
            # handle kW actions
                irradiance = self.irradiance_profile[(self.current_step) % 8760]
                scaled_pv_kw = kw_action_value * irradiance
                if scaled_pv_kw > irradiance * self.KWrated:  # clip to max
                    scaled_pv_kw = irradiance * self.KWrated

            # handle kVAR actions
                pv_kvar = kvar_action_value
                S_max = 100  # Maximum apparent power (example value)
                q_max1 = np.sqrt(S_max**2 - np.power(scaled_pv_kw, 2))
                pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)
                self.controller.text(f"edit PVSystem.PV{idx + pv_idx_offset + 1} phases=3 kV=4.16 Pmpp={scaled_pv_kw} kVAR={pv_kvar}")

            for idx, action_value in enumerate(bess_kw_actions):
                self.controller.text(f"edit Storage.Battery{idx + battery_idx_offset + 1} phases=3 kV=4.16 kW={action_value} kVAR=0")

        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")




    def step(self, action_dict):
    # Execute the action
        self._take_action(action_dict)

    # Calculate the rewards
        losses = sum(self.controller.circuit_losses())
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        voltage_violations = sum(1 for v in all_bus_voltages if v <= 0.95 or v >= 1.05)




        
        # Calculate the sum of squared voltage deviations from 1. This penalizes larger deviations more heavily.
        voltage_deviations = sum((v - 1)**2 for v in all_bus_voltages)



        # Define penalty weights for different components
        w_deviation = 1.0
        w_violation = 5.0
        w_loss = 0.001

        # Combine losses, violations, and deviations in the reward
        #reward = - w_deviation * voltage_deviations - w_violation * voltage_violations
        reward= - voltage_violations 
        
        #-  self.control_steps

        


    # Gather observations for all agents
        observations = {agent_id: self.get_observation(agent_id) for agent_id in self._agent_ids}

    # Set the reward for all agents
        rewards = {agent_id: reward for agent_id in self._agent_ids}
        print(rewards)
        print(reward)

    # Set termination status for all agents
        terminations = {agent_id: (self.control_steps >= self.max_control_steps) or (voltage_violations == 0) for agent_id in self._agent_ids}
        terminations['__all__'] = any(terminations.values())

    # Assuming no truncation; modify as needed
        truncateds = {agent_id: False for agent_id in self._agent_ids}
        truncateds['__all__'] = any(truncateds.values())

    # Gather additional information if required
        infos = {agent_id: {} for agent_id in self._agent_ids}

    # After taking action, increment control steps
        self.control_steps += 1

        return observations, rewards, terminations, truncateds, infos


   

    def reset(self, *, seed=None, options=None):
    # If a seed is provided, set the random seed for numpy
        if seed is None:
            seed = 1

    # Now set the random seed for numpy
        np.random.seed(seed)

    # You can use options to customize the reset function if needed
    # For now, we'll just print the options
        if options is not None:
            print(f"Reset options: {options}")
        # Reset power system to initial state
        self.controller.text(f"compile [{dss_file}]")
        
        #self.current_step = int(np.clip(np.random.normal(loc=0, scale=8760), 0, 8760))
        self.current_step = np.random.randint(0, 8761)
       

 # assuming the profile has 8760 hours  #use gaussian distribution
        self.control_steps = 0


     
        
        
         # Load names
        load_names = [
            "S1a", "S2b", "S4c", "S5c", "S6c", "S7a", "S9a", "S10a", "S11a", "S12b",
            "S16c", "S17c", "S19a", "S20a", "S22b", "S24c", "S28a", "S29a", "S30c", "S31c",
            "S32c", "S33a", "S34c", "S35a", "S37a", "S38b", "S39b", "S41c", "S42a", "S43b",
            "S45a", "S46a", "S47", "S48", "S49a", "S49b", "S49c", "S50c", "S51a", "S52a",
            "S53a", "S55a", "S56b", "S58b", "S59b", "S60a", "S62c", "S63a", "S64b", "S65a",
            "S65b", "S65c", "S66c", "S68a", "S69a", "S70a", "S71a", "S73c", "S74c", "S75c",
            "S76a", "S76b", "S76c", "S77b", "S79a", "S80b", "S82a", "S83c", "S84c", "S85c",
            "S86b"
            ]
  
            



        # Read CSV file into list
        #load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).tolist()
        # Convert the first column to a list
        load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).iloc[:, 0].tolist()


        # Create a dictionary with load names as keys and load powers as values
        load_dict = dict(zip(load_names, load_powers))

# Generate load scales using a Gaussian distribution
        np.random.seed(seed)  # Set the seed again before generating random numbers
        load_scales = np.random.normal(loc=self.load_profile[self.current_step % 8760], scale=0.5, size=len(load_names))

        for load_name, load_scale in zip(load_names, load_scales):
    # Get the load power corresponding to load_name from the dictionary
            load_power = load_dict[load_name]
    
    # Multiply load power by load_scale
            result = load_power * load_scale
    
    # Use the result in your controller
            self.controller.text(f"edit Load.{load_name} kW={result}")

            
            
        # Load the irradiance for the current hour
        
        
        irradiance = self.irradiance_profile[self.current_step % 8760]
        

    
        

        # Initialize random states for controlled devices
        for i, bus in enumerate(self.ranked_buses):
            # Randomly initialize PV systems and batteries
            #kvar should be set to very small
            fixed_power_factor = 0.9  # Set a fixed power factor value (e.g., 0.9)
            pv_kw = 100 * self.irradiance_profile[self.current_step % 10]  # Scale PV kW by irradiance
            power_factor_angle = np.arccos(fixed_power_factor)  # Calculate the angle corresponding to the power factor
            pv_kvar = pv_kw * np.tan(power_factor_angle)  # Calculate reactive power (kVAR) based on kW and power factor
            battery_kw = self.load_profile[self.current_step % 10]*100  # Scale Battery kW by load
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0") #select charging or discharging
            #make a comparison study beween different modes
            #investigate batteries charging/dicharging pattern




        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        
        # Get the observations for all agents
        observations = {agent_id: self.get_observation(agent_id) for agent_id in self._agent_ids}

    # You can include any additional information here. If there's nothing, just return an empty dictionary.
        infos = {}

        return observations, infos


    def render(self, mode='human'):
        pass

    def close(self):
        pass

    def get_observation(self, agent_id):
    # Get the voltage at all buses
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
    # Flatten the list of bus voltages
        all_bus_voltages = np.array(all_bus_voltages).flatten()

    # Extract agent index from the agent_id
        agent_idx = int(agent_id.split('_')[1])

    # Determine the start and end index for the observation chunk for this agent
        if agent_idx < 4:
            start_idx = agent_idx * 50
            end_idx = start_idx + 50
        else:  # This is agent_4
            start_idx = 200
            end_idx = 278  # or you could use end_idx = len(all_bus_voltages) if the size is not guaranteed to be 278

    # Extract the observation chunk for this agent
        agent_observation = all_bus_voltages[start_idx:end_idx]

        return agent_observation


    def get_agent_observation(self):
        agent_obs = {}
        for agent_id in self.agents:
            agent_obs[agent_id] = self.get_observation()
        return agent_obs

    def _end_of_episode(self):
        return self.current_step >= len(self.irradiance_profile)
    

irradiance_csv_file = r"D:\Alaa_Selim\Irradiance_Profile_Santa_Clara.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"

    
    
original_env = PowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)


from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility

env = MultiAgentEnvCompatibility(original_env)

def env_creator(env_config):
    return PowerSystemEnv(
                            dss_path=r"C:\Program Files\OpenDSS",
                            dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
                            irradiance_csv_file=irradiance_csv_file,
                            load_profile_file=load_profile_file
                            )

register_env("PowerSystemEnv", env_creator)



# Suppose you have N agents
N = 5  # adjust this as per your actual number of agents

policies = {
    f"policy_{i}": (None, 
                    original_env.observation_space_dict[f"agent_{i}"], 
                    original_env.action_space_dict[f"agent_{i}"], 
                    {"agent_id": i})  # Set 'agent_id' key here
    for i in range(N)
}


def policy_mapping_fn(agent_id, episode, **kwargs):
    if "agent_" in agent_id:
        # Extract agent number and return the corresponding policy
        agent_num = int(agent_id.split("_")[1])
        return f"policy_{agent_num}"
    else:
        raise ValueError(f"Unknown agent_id: {agent_id}")






2023-11-12 16:27:04,806	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-11-12 16:27:06,896	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-11-12 16:27:06,904	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!


OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 


OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 




In [5]:
from ray.rllib.algorithms.impala import ImpalaConfig
from ray import air
from ray import tune
config = ImpalaConfig()
# Print out some default values.
print(config.vtrace)  
# Update the config object.
config = config.training(   
    lr=tune.grid_search([0.00001])
)
# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=6) 
config = config.environment(env="PowerSystemEnv")  
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "IMPALA",
    run_config=air.RunConfig(stop={"episode_reward_mean": 0}),
    param_space=config.to_dict(),
).fit()

(pid=20108) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(Impala pid=20108) 2023-11-03 16:45:08,659	WARNING algorithm_config.py:656 -- Cannot create ImpalaConfig from given `config_dict`! Property __stdout_file__ not supported.
(pid=22968) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future! [repeated 3x across cluster]
(RolloutWorker pid=20024) 2023-11-03 16:45:13,918	WARNING multi_agent_env.py:169 -- observation_space_contains() of <PowerSystemEnv instance> has not been implemented. You can either implement it yourself or bring the observation space into the preferred format of a mapping from agent ids to their individual observation spaces. 
(RolloutWorker pid=20024) 2023-11-03 16:45:13,918	WARNING multi_agent_env.py:169 -- observation_space_contains() of <PowerSystemEnv instance> has not been implemented. You can either implement it yourself or bring the observation space int

(RolloutWorker pid=20024) OpenDSS Started successfully! 
(RolloutWorker pid=20024) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 
(RolloutWorker pid=20024) 
(RolloutWorker pid=20024) 
(RolloutWorker pid=20024) Reset options: {}
(RolloutWorker pid=20024) {'agent_2': -5, 'agent_0': -5, 'agent_4': -5, 'agent_1': -5, 'agent_3': -5}
(RolloutWorker pid=20024) -5


2023-11-03 16:45:14,169	ERROR tune_controller.py:911 -- Trial task failed for trial IMPALA_PowerSystemEnv_ddac9_00000
Traceback (most recent call last):
  File "C:\Users\Alaa\.conda\envs\py310\lib\site-packages\ray\air\execution\_internal\event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "C:\Users\Alaa\.conda\envs\py310\lib\site-packages\ray\_private\auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "C:\Users\Alaa\.conda\envs\py310\lib\site-packages\ray\_private\client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\Alaa\.conda\envs\py310\lib\site-packages\ray\_private\worker.py", line 2524, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(ValueError): ray::Impala.train() (pid=20108, ip=127.0.0.1, actor_id=015c953a3d4d9c16c7eb358801000000, repr=Impala)
  File "python\ray\_raylet.pyx", line 1424, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx",

(Impala pid=20108) Caught sync error: Sync process failed: GetFileInfo() yielded path 'C:/Users/Alaa/ray_results/IMPALA/IMPALA_PowerSystemEnv_ddac9_00000_0_lr=0.0000_2023-11-03_16-45-04', which is outside base dir 'C:\Users\Alaa\ray_results\IMPALA\IMPALA_PowerSystemEnv_ddac9_00000_0_lr=0.0000_2023-11-03_16-45-04\'. Retrying after sleeping for 1.0 seconds...
(Impala pid=20108) Caught sync error: Sync process failed: GetFileInfo() yielded path 'C:/Users/Alaa/ray_results/IMPALA/IMPALA_PowerSystemEnv_ddac9_00000_0_lr=0.0000_2023-11-03_16-45-04', which is outside base dir 'C:\Users\Alaa\ray_results\IMPALA\IMPALA_PowerSystemEnv_ddac9_00000_0_lr=0.0000_2023-11-03_16-45-04\'. Retrying after sleeping for 1.0 seconds...
(Impala pid=20108) Could not upload checkpoint to c://\Users\Alaa\ray_results\IMPALA\IMPALA_PowerSystemEnv_ddac9_00000_0_lr=0.0000_2023-11-03_16-45-04 even after 3 retries.Please check if the credentials expired and that the remote filesystem is supported. For large checkpoints o

ValueError: `C:\Users\Alaa\ray_results\IMPALA\experiment_state-2023-11-03_16-45-04.json` must either be a path to an experiment checkpoint file, or a directory containing an experiment checkpoint file.

In [ ]:
tune.run(
    "SAC",
    stop={"episodes_total": 300000},
    config={
        "env": "PowerSystemEnv",
        "environment": {
            "disable_env_checking": True
        },
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn
        },
        "num_gpus": 0,
        "num_workers": 19,
        "lr": 3e-5,
    },
    local_dir="SAC-Test",
    # callbacks=[eval_callback]
)


In [ ]:
from ray import tune

class EntropyScheduleCallback(tune.Trainable):
    def setup(self, config):
        # This is called at the beginning of training.
        self.config = config

    def step(self):
        # This is called at each training step.
        iteration = self._iteration
        start = 0.01
        end = 0.001
        decay_steps = 10000
        decayed_value = (start - end) * ((1 - iteration / decay_steps) ** 2) + end
        self.config["entropy_coeff"] = max(decayed_value, end)
        # ... rest of your training logic ...
        return {"mean_reward": ...}  # return your training result here

# Define the training configuration
training_config = {
    "env": "PowerSystemEnv",
    "num_workers": 19,
    "multiagent": {
        "policies": policies,
        "policy_mapping_fn": policy_mapping_fn,
    },
    # SAC-specific hyperparameters
    "timesteps_per_iteration": 1000,
    "target_entropy": "auto",
    "buffer_size": 100000,
    "learning_starts": 1000,
    "tau": 0.01,
    "gamma": 0.99,
    "train_batch_size": 512,
    "lr": 1e-4,
    "optimization": {
        "actor_learning_rate": 1e-4,
        "critic_learning_rate": 1e-4,
        "entropy_learning_rate": 1e-4,
    },
    "entropy_coeff": 0.01,  # initial value
    "clip_actions": False,
    "normalize_actions": True,
}

# Execute the training run
tune.run(
    EntropyScheduleCallback,
    config=training_config,
    stop={"training_iteration": 20000},  # Adjust as needed
)


In [ ]:
import ray
from ray import tune
from ray.rllib.algorithms.sac.sac import SAC
from ray.tune import Callback

class EntropyScheduleCallback(Callback):
    def on_train_result(self, trainer, result):
        iteration = result["training_iteration"]
        start = 0.01
        end = 0.001
        decay_steps = 10000
        decayed_value = (start - end) * ((1 - iteration / decay_steps) ** 2) + end
        trainer.config["entropy_coeff"] = max(decayed_value, end)

# Define the training configuration
training_config = {
    "env": "PowerSystemEnv",
    "num_workers": 19,
    "multiagent": {
        "policies": policies,
        "policy_mapping_fn": policy_mapping_fn,
    },
    # SAC-specific hyperparameters
    "timesteps_per_iteration": 1000,
    "target_entropy": "auto",
    "buffer_size": 500000,
    "learning_starts": 1000,
    "tau": 0.02,
    "gamma": 0.95,
    "train_batch_size": 1024,
    "lr": 1e-4,
    "optimization": {
        "actor_learning_rate": 5e-4,
        "critic_learning_rate": 5e-4,
        "entropy_learning_rate": 5e-4,
    },
    "entropy_coeff": 0.01,  # initial value
    "clip_actions": False,
    "normalize_actions": True,
}

# Execute the training run
tune.run(
    SAC,
    config=training_config,
    stop={"training_iteration": 20000},  # Adjust as needed
    callbacks=[EntropyScheduleCallback()]
)


In [ ]:
import ray
print(ray.__version__)


In [ ]:
def polynomial_decay_schedule(iteration, start=0.01, end=0.001, decay_steps=10000):
    decayed_value = (start - end) * ((1 - iteration / decay_steps) ** 2) + end
    return max(decayed_value, end)

# Define the training configuration
training_config = {
    "env": "PowerSystemEnv",
    "num_workers": 19,
    "multiagent": {
        "policies": policies,
        "policy_mapping_fn": policy_mapping_fn,
    },
    # SAC-specific hyperparameters
    "timesteps_per_iteration": 1000,
    "target_entropy": "auto",
    "buffer_size": 100000,
    "learning_starts": 1000,
    "tau": 0.01,
    "gamma": 0.99,
    "train_batch_size": 512,
    "lr": 1e-4,
    "optimization": {
        "actor_learning_rate": 1e-4,
        "critic_learning_rate": 1e-4,
        "entropy_learning_rate": 1e-4,
    },
    "clip_actions": False,
    "normalize_actions": True
    #"entropy_coeff_schedule": polynomial_decay_schedule,
}

# Execute the training run
tune.run(
    "SAC",
    config=training_config,
    stop={"training_iteration": 20000},  # Adjust as needed
)


In [3]:
print(f'dss.circuit_all_node_names(): {dss.circuit_all_node_names()}')


dss.circuit_all_node_names(): ['150.1', '150.2', '150.3', '150r.1', '150r.2', '150r.3', '149.1', '149.2', '149.3', '1.1', '1.2', '1.3', '2.2', '3.3', '7.1', '7.2', '7.3', '4.3', '5.3', '6.3', '8.1', '8.2', '8.3', '12.2', '9.1', '13.1', '13.2', '13.3', '9r.1', '14.1', '34.3', '18.1', '18.2', '18.3', '11.1', '10.1', '15.3', '16.3', '17.3', '19.1', '21.1', '21.2', '21.3', '20.1', '22.2', '23.1', '23.2', '23.3', '24.3', '25.1', '25.2', '25.3', '25r.1', '25r.3', '26.1', '26.3', '28.1', '28.2', '28.3', '27.1', '27.3', '31.3', '33.1', '29.1', '29.2', '29.3', '30.1', '30.2', '30.3', '250.1', '250.2', '250.3', '32.3', '35.1', '35.2', '35.3', '36.1', '36.2', '40.1', '40.2', '40.3', '37.1', '38.2', '39.2', '41.3', '42.1', '42.2', '42.3', '43.2', '44.1', '44.2', '44.3', '45.1', '47.1', '47.2', '47.3', '46.1', '48.1', '48.2', '48.3', '49.1', '49.2', '49.3', '50.1', '50.2', '50.3', '51.1', '51.2', '51.3', '151.1', '151.2', '151.3', '52.1', '52.2', '52.3', '53.1', '53.2', '53.3', '54.1', '54.2', '54.

In [4]:
buses = ['150.1', '150.2', '150.3', '150r.1', '150r.2', '150r.3', '149.1', '149.2', '149.3', '1.1', '1.2', '1.3', '2.2', '3.3', '7.1', '7.2', '7.3', '4.3', '5.3', '6.3', '8.1', '8.2', '8.3', '12.2', '9.1', '13.1', '13.2', '13.3', '9r.1', '14.1', '34.3', '18.1', '18.2', '18.3', '11.1', '10.1', '15.3', '16.3', '17.3', '19.1', '21.1', '21.2', '21.3', '20.1', '22.2', '23.1', '23.2', '23.3', '24.3', '25.1', '25.2', '25.3', '25r.1', '25r.3', '26.1', '26.3', '28.1', '28.2', '28.3', '27.1', '27.3', '31.3', '33.1', '29.1', '29.2', '29.3', '30.1', '30.2', '30.3', '250.1', '250.2', '250.3', '32.3', '35.1', '35.2', '35.3', '36.1', '36.2', '40.1', '40.2', '40.3', '37.1', '38.2', '39.2', '41.3', '42.1', '42.2', '42.3', '43.2', '44.1', '44.2', '44.3', '45.1', '47.1', '47.2', '47.3', '46.1', '48.1', '48.2', '48.3', '49.1', '49.2', '49.3', '50.1', '50.2', '50.3', '51.1', '51.2', '51.3', '151.1', '151.2', '151.3', '52.1', '52.2', '52.3', '53.1', '53.2', '53.3', '54.1', '54.2', '54.3', '55.1', '55.2', '55.3', '57.1', '57.2', '57.3', '56.1', '56.2', '56.3', '58.2', '60.1', '60.2', '60.3', '59.2', '61.1', '61.2', '61.3', '62.1', '62.2', '62.3', '63.1', '63.2', '63.3', '64.1', '64.2', '64.3', '65.1', '65.2', '65.3', '66.1', '66.2', '66.3', '67.1', '67.2', '67.3', '68.1', '72.1', '72.2', '72.3', '97.1', '97.2', '97.3', '69.1', '70.1', '71.1', '73.3', '76.1', '76.2', '76.3', '74.3', '75.3', '77.1', '77.2', '77.3', '86.1', '86.2', '86.3', '78.1', '78.2', '78.3', '79.1', '79.2', '79.3', '80.1', '80.2', '80.3', '81.1', '81.2', '81.3', '82.1', '82.2', '82.3', '84.3', '83.1', '83.2', '83.3', '85.3', '87.1', '87.2', '87.3', '88.1', '89.1', '89.2', '89.3', '90.2', '91.1', '91.2', '91.3', '92.3', '93.1', '93.2', '93.3', '94.1', '95.1', '95.2', '95.3', '96.2', '98.1', '98.2', '98.3', '99.1', '99.2', '99.3', '100.1', '100.2', '100.3', '450.1', '450.2', '450.3', '197.1', '197.2', '197.3', '101.1', '101.2', '101.3', '102.3', '105.1', '105.2', '105.3', '103.3', '104.3', '106.2', '108.1', '108.2', '108.3', '107.2', '109.1', '300.1', '300.2', '300.3', '110.1', '111.1', '112.1', '113.1', '114.1', '135.1', '135.2', '135.3', '152.1', '152.2', '152.3', '160r.1', '160r.2', '160r.3', '160.1', '160.2', '160.3', '61s.1', '61s.2', '61s.3', '300_open.1', '300_open.2', '300_open.3', '94_open.1', '610.1', '610.2', '610.3']
print(len(buses))

278


In [ ]:


#Testing with partial observability of voltage violatiosn for each agent

import gymnasium as gym
from gymnasium import spaces
from gymnasium.spaces import Tuple, Dict

from ray.rllib.env.multi_agent_env import MultiAgentEnv
import numpy as np
from ray.rllib.algorithms.sac import SAC
from ray.rllib.algorithms.ppo import PPO
from ray.tune.registry import register_env
from ray import tune
import numpy as np

from py_dss_interface import DSSDLL
#import stable_baselines3
#from stable_baselines3 import SAC
import matplotlib.pyplot as plt
import csv
import pandas as pd
#from stable_baselines3 import A2C, DQN, PPO, TD3, SAC
import torch
import warnings
warnings.filterwarnings('ignore')
import logging

logging.basicConfig(level=logging.ERROR)





device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import time

start_time = time.time()



# Initialize OpenDSS
dss = DSSDLL(r"C:\Program Files\OpenDSS")
dss_file = r"D:\Alaa_Selim\123Bus\IEEE123Master.dss"
dss.text(f"compile [{dss_file}]")

# defining lengths for each segment
PV_KVAR_ACTION_LEN = 30
PV_KW_ACTION_LEN = 30
BESS_KW_ACTION_LEN = 30
TRANSFORMER_TAPS_ACTION_LEN = 1
CAPACITOR_ACTION_LEN = 4

# defining values for each segment
PV_KVAR_ACTION_LOW = -800
PV_KVAR_ACTION_HIGH = 800

PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 1000

BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 1000

TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1

CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1


class PowerSystemEnv(MultiAgentEnv):
    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file):
        super(PowerSystemEnv, self).__init__()

        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        self.ranked_buses  = [
                                '1', '7', '8', '13', '21', '23', '29', '250', '35', '40', '42', '55',
                                '56', '65', '76', '78', '66', '79', '81', '83', '91', '95', '100',
                                '197', '300', '110', '135', '160', '152', '610'
                            ]
        self.capacitor_names = ["C83", "C88a", "C90b", "C92c"]
        self.KWrated=1000
        self.previous_reward = 0.0
        self.alpha = 0.1  #
        #self.last_actions = {agent_id: np.zeros(original_action_size) for agent_id in self._agent_ids}

         # Apply actions to PV systems and batteries
        for i in range(30):
            bus = self.ranked_buses[i]
            self.controller.text(f"new Generator.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=1000 kW=800")
            self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=1000 kVAR=0")
        
        with open(irradiance_csv_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            self.irradiance_profile = [float(row[0]) for row in reader]
    
        with open(load_profile_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader, None)  # Skip the header
            self.load_profile = [float(row[0]) for row in reader]
            
            
        self.action_space = spaces.Box(
            low=np.array(
                [PV_KVAR_ACTION_LOW]*PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_LOW]*PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_LOW]*BESS_KW_ACTION_LEN 
            ),
            high=np.array(
                [PV_KVAR_ACTION_HIGH]*PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_HIGH]*PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_HIGH]*BESS_KW_ACTION_LEN 
               
            ),
            dtype=np.float32
        )
            

     
        # Assuming observation space is the voltage at each bus
        self.observation_space = spaces.Box(low=0, high=2, shape=(278,), dtype=np.float32)  # Modified shape
            
         
        # Update the action and observation spaces for each agent
        self.action_space_dict = {
            f"agent_{i}": spaces.Box(
                low=np.concatenate(([PV_KW_ACTION_LOW] * 6, [PV_KVAR_ACTION_LOW] * 6, [BESS_KW_ACTION_LOW] * 6)),
                high=np.concatenate(([PV_KW_ACTION_HIGH] * 6, [PV_KVAR_ACTION_HIGH] * 6, [BESS_KW_ACTION_HIGH] * 6)),
                shape=(18,), dtype=np.float32) for i in range(5)
        }



        self.observation_space_dict = {}
        
        #self.observation_space_dict = {
              # agent_id: spaces.Box(low=0, high=2, shape=(278,), dtype=np.float32) for agent_id in self._agent_ids
              #  }

        
        

        # Initializing the observation_space_dict
        #self.observation_space_dict = {}

        # Define the sizes for each agent's observations
        agent_sizes = [62, 62, 62, 60, 32]

        for i in range(5):
            agent_obs_size = agent_sizes[i]
            self.observation_space_dict[f'agent_{i}'] = spaces.Box(low=0, high=2, shape=(agent_obs_size,), dtype=np.float32)


        # For the last agent
        #self.observation_space_dict['agent_4'] = spaces.Box(low=0, high=2, shape=(78,), dtype=np.float32)

        
        self.current_step = 0

        
         # Initialize control step counter
        self.control_steps = 0

        # Maximum control steps allowed in one episode
        self.max_control_steps =int(5)  # for example
        
        



          # Define the agent IDs
        self._agent_ids = [f'agent_{i}' for i in range(5)]


        
   

        
    def _take_action(self, action_dict):
    # Define total actions per agent
        TOTAL_ACTIONS_PER_AGENT = 6 + 6 + 6

    # Iterate through the action_dict
        for agent_id, action in action_dict.items():
            agent_idx = int(agent_id.split('_')[1])

        # Determine action segments for this agent
            pv_kw_actions = action[:6]
            pv_kvar_actions = action[6:12]
            bess_kw_actions = action[12:18]

        # Index offset based on agent_idx for PV and battery control
            pv_idx_offset = agent_idx * 6
            battery_idx_offset = agent_idx * 6

        # Actions for PVs
            for idx, (kw_action_value, kvar_action_value) in enumerate(zip(pv_kw_actions, pv_kvar_actions)):
            # handle kW actions
                irradiance = self.irradiance_profile[(self.current_step) % 24]
                scaled_pv_kw = kw_action_value * irradiance
                if scaled_pv_kw > irradiance * self.KWrated:  # clip to max
                    scaled_pv_kw = irradiance * self.KWrated

            # handle kVAR actions
                pv_kvar = kvar_action_value
                S_max = 1000  # Maximum apparent power (example value)
                q_max1 = np.sqrt(S_max**2 - np.power(scaled_pv_kw, 2))
                pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)
                self.controller.text(f"edit Generator.PV{idx + pv_idx_offset + 1} phases=3 kV=4.16 kW={scaled_pv_kw} kVAR={pv_kvar}")

            for idx, action_value in enumerate(bess_kw_actions):
                self.controller.text(f"edit Storage.Battery{idx + battery_idx_offset + 1} phases=3 kV=4.16 kW={action_value} kVAR=0")

        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")




    def step(self, action_dict):
    # Execute the action
        self._take_action(action_dict)

    # Calculate the rewards
        losses = sum(self.controller.circuit_losses())
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        voltage_violations = sum(1 for v in all_bus_voltages if v <= 0.95 or v >= 1.05)
        




        
        # Calculate the sum of squared voltage deviations from 1. This penalizes larger deviations more heavily.
        voltage_deviations = sum((v - 1)**2 for v in all_bus_voltages)



        # Define penalty weights for different components
        w_deviation = 1.0
        w_violation = 5.0
        w_loss = 0.001

        # Combine losses, violations, and deviations in the reward
        #reward = - w_deviation * voltage_deviations - w_violation * voltage_violations
        #reward = - (2 ** voltage_violations) - 1
        #reward = -  voltage_deviations
        reward = - voltage_violations
        

        
        #-  self.control_steps

        


    # Gather observations for all agents
        observations = {agent_id: self.get_observation(agent_id) for agent_id in self._agent_ids}

    # Set the reward for all agents
        rewards = {agent_id: reward for agent_id in self._agent_ids}

    # Set termination status for all agents
        terminations = {agent_id: (self.control_steps >= self.max_control_steps) or (voltage_violations == 0) for agent_id in self._agent_ids}
        terminations['__all__'] = any(terminations.values())

    # Assuming no truncation; modify as needed
        truncateds = {agent_id: False for agent_id in self._agent_ids}
        truncateds['__all__'] = any(truncateds.values())

    # Gather additional information if required
        infos = {agent_id: {} for agent_id in self._agent_ids}

    # After taking action, increment control steps
        self.control_steps += 1

        return observations, rewards, terminations, truncateds, infos


   

    def reset(self, *, seed=None, options=None):
    # If a seed is provided, set the random seed for numpy
        if seed is not None:
            np.random.seed(seed)

    # You can use options to customize the reset function if needed
    # For now, we'll just print the options
        if options is not None:
            print(f"Reset options: {options}")
        # Reset power system to initial state
        self.controller.text(f"compile [{dss_file}]")
        
        #self.current_step = int(np.clip(np.random.normal(loc=0, scale=8760), 0, 8760))
        self.current_step = np.random.randint(0, 25)
       

 # assuming the profile has 8760 hours  #use gaussian distribution
        self.control_steps = 0


     
        
        
         # Load names
        load_names = [
            "S1a", "S2b", "S4c", "S5c", "S6c", "S7a", "S9a", "S10a", "S11a", "S12b",
            "S16c", "S17c", "S19a", "S20a", "S22b", "S24c", "S28a", "S29a", "S30c", "S31c",
            "S32c", "S33a", "S34c", "S35a", "S37a", "S38b", "S39b", "S41c", "S42a", "S43b",
            "S45a", "S46a", "S47", "S48", "S49a", "S49b", "S49c", "S50c", "S51a", "S52a",
            "S53a", "S55a", "S56b", "S58b", "S59b", "S60a", "S62c", "S63a", "S64b", "S65a",
            "S65b", "S65c", "S66c", "S68a", "S69a", "S70a", "S71a", "S73c", "S74c", "S75c",
            "S76a", "S76b", "S76c", "S77b", "S79a", "S80b", "S82a", "S83c", "S84c", "S85c",
            "S86b"
            ]
  
            



        # Read CSV file into list
        #load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).tolist()
        # Convert the first column to a list
        load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).iloc[:, 0].tolist()


        # Create a dictionary with load names as keys and load powers as values
        load_dict = dict(zip(load_names, load_powers))

# Generate load scales using a Gaussian distribution
        load_scales = np.random.normal(loc=self.load_profile[self.current_step % 24], scale=0.5, size=len(load_names))

        for load_name, load_scale in zip(load_names, load_scales):
    # Get the load power corresponding to load_name from the dictionary
            load_power = load_dict[load_name]
    
    # Multiply load power by load_scale
            result = load_power * load_scale
    
    # Use the result in your controller
            self.controller.text(f"edit Load.{load_name} kW={result}")

            
            
        # Load the irradiance for the current hour
        
        
        irradiance = self.irradiance_profile[self.current_step % 24]
        

    
        

        # Initialize random states for controlled devices
        for i, bus in enumerate(self.ranked_buses):
            # Randomly initialize PV systems and batteries
            #kvar should be set to very small
            fixed_power_factor = 0.9  # Set a fixed power factor value (e.g., 0.9)
            pv_kw = 10000 * self.irradiance_profile[self.current_step % 24]  # Scale PV kW by irradiance
            power_factor_angle = np.arccos(fixed_power_factor)  # Calculate the angle corresponding to the power factor
            pv_kvar = pv_kw * np.tan(power_factor_angle)  # Calculate reactive power (kVAR) based on kW and power factor
            battery_kw = self.load_profile[self.current_step % 24]*1000  # Scale Battery kW by load
            self.controller.text(f"edit Generator.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} kW={pv_kw}")
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0") #select charging or discharging
            #make a comparison study beween different modes
            #investigate batteries charging/dicharging pattern




        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        
        # Get the observations for all agents
        observations = {agent_id: self.get_observation(agent_id) for agent_id in self._agent_ids}

    # You can include any additional information here. If there's nothing, just return an empty dictionary.
        infos = {}

        return observations, infos


    def render(self, mode='human'):
        pass

    def close(self):
        pass

    def get_observation(self, agent_id):
        # Get the voltage at all buses
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()

        agent_idx = int(agent_id.split('_')[1])

        # Determine the start and end indices for slicing the all_bus_voltages
        if agent_idx < 3:  # First three agents
            start_idx = agent_idx * 62
            end_idx = start_idx + 62
        elif agent_idx == 3:  # fourth agent
            start_idx = agent_idx * 62
            end_idx = start_idx + 60
        else:  # Fifth and last agent
            start_idx =3 * 62 + 60  # Cover all the previous indices
            end_idx = start_idx + 32

        agent_observations = all_bus_voltages[start_idx:end_idx]

        return agent_observations


    def get_agent_observation(self):
        agent_obs = {}
        for agent_id in self.agents:
            agent_obs[agent_id] = self.get_observation(agent_id)  # Note: We're still passing agent_id even if it's unused. This keeps the interface consistent.
        return agent_obs

    def _end_of_episode(self):
        return self.current_step >= len(self.irradiance_profile)

    

irradiance_csv_file = r"D:\Alaa_Selim\Irradiance_Profile_Santa_Clara.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"

    
    
original_env = PowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)


from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility

env = MultiAgentEnvCompatibility(original_env)

def env_creator(env_config):
    return PowerSystemEnv(
                            dss_path=r"C:\Program Files\OpenDSS",
                            dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
                            irradiance_csv_file=irradiance_csv_file,
                            load_profile_file=load_profile_file
                            )

register_env("PowerSystemEnv", env_creator)



# Suppose you have N agents
N = 5  # adjust this as per your actual number of agents

policies = {
    f"policy_{i}": (None, 
                    original_env.observation_space_dict[f"agent_{i}"], 
                    original_env.action_space_dict[f"agent_{i}"], 
                    {"agent_id": i})  # Set 'agent_id' key here
    for i in range(N)
}


def policy_mapping_fn(agent_id, episode, **kwargs):
    if "agent_" in agent_id:
        # Extract agent number and return the corresponding policy
        agent_num = int(agent_id.split("_")[1])
        return f"policy_{agent_num}"
    else:
        raise ValueError(f"Unknown agent_id: {agent_id}")






In [ ]:
import numpy as np
import gymnasium as gym
from stable_baselines3 import SAC, TD3
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.results_plotter import load_results, ts2xy
import matplotlib.pyplot as plt
from stable_baselines3.common.monitor import Monitor
import torch
import random
import time

SEED = 0
np.random.seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)

start_time = time.time()

# First, wrap the custom environment with Monitor
monitored_env = Monitor(PowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
), './models/')

# Then, wrap the monitored environment with DummyVecEnv
env = DummyVecEnv([lambda: monitored_env])

# Save a checkpoint every 1000 steps
checkpoint_callback = CheckpointCallback(save_freq=1000, save_path='./models/')

# Training configurations
num_timesteps = 100000
agents = {
    "SAC": SAC("MlpPolicy", env, verbose=1),
}

# Train each agent and store results
results = {}
for name, agent in agents.items():
    print(f"\nTraining {name}...\n")
    agent.learn(total_timesteps=num_timesteps, callback=checkpoint_callback)
    results[name] = load_results('./models/')

end_time = time.time()
print(f"Total optimization time: {end_time - start_time} seconds")

# Plot learning curves
def moving_average(values, window):
    """
    Smooth values by doing a moving average
    """
    weights = np.repeat(1.0, window) / window
    return np.convolve(values, weights, 'valid')

def plot_results():
    fig, ax = plt.subplots(figsize=(9, 4))
    
    for name, result in results.items():
        x, y = ts2xy(result, 'timesteps')
        y = moving_average(y, window=50)  # Window of 50 for smoothing
        ax.plot(x[:-49], y, label=name)
    
    ax.set_xlabel("Timesteps")
    ax.set_ylabel("Rewards")
    plt.tight_layout()
    # Save the figure
    plt.savefig('SAC_Learning.pdf', format='pdf')
    plt.show()

plot_results()


In [ ]:
tune.run(
    "SAC",
    stop={"episodes_total": 1000},
    config={
        "env": "PowerSystemEnv",
        "environment": {
            "disable_env_checking": True
        },
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn
        },
        "num_gpus": 0,
        "num_workers": 19,
        "lr": 3e-5,
        "tau": 5e-3,
        "prioritized_replay": True,
        "actor_learning_rate": 3e-5,
        "critic_learning_rate": 3e-5,
        "entropy_learning_rate": 3e-5,
        
        
        
    },
    local_dir="SAC-Test",
    # callbacks=[eval_callback]
)


In [ ]:
from ray.rllib.algorithms.sac.sac import SACConfig


config = SACConfig().training(gamma=0.9, lr=0.01)  
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=4)  
print(config.to_dict())  
# Build a Algorithm object from the config and run 1 training iteration.
algo = config.build(env="PowerSystemEnv")  
algo.train()  

In [ ]:
from ray.rllib.algorithms.maddpg.maddpg import MADDPGConfig
config = MADDPGConfig()
print(config.replay_buffer_config)  
replay_config = config.replay_buffer_config.update(  
    {
        "capacity": 100000,
        "prioritized_replay_alpha": 0.8,
        "prioritized_replay_beta": 0.45,
        "prioritized_replay_eps": 2e-6,
    }
)
config.training(replay_buffer_config=replay_config)   
config = config.resources(num_gpus=0)   
config = config.rollouts(num_rollout_workers=4)   
config = config.environment("PowerSystemEnv")   
algo = config.build()  
algo.train()  

In [ ]:
from ray.rllib.algorithms.maddpg.maddpg import MADDPGConfig
from ray import air
from ray import tune
config = MADDPGConfig()
config.training(n_step=tune.grid_search([3, 5]))  
config.environment(env="PowerSystemEnv")  
tune.Tuner(  
    "MADDPG",
    run_config=air.RunConfig(stop={"episode_reward_mean":200}),
    param_space=config.to_dict()
).fit()

In [ ]:
tune.run(
    "SAC",
    stop={"episodes_total": 30000},
    config={
        "env": "PowerSystemEnv",
         "evaluation_interval": 2,
        "evaluation_num_episodes": 10,
        "environment": {
            "disable_env_checking": True
        },
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn
        },
        "num_gpus": 0,
        "num_workers": 17,
        "lr": 3e-4,
        "grad_clip": 40.0,  # Clip gradients with L2 norm exceeding this value
    },
    local_dir="SAC-Test",
    # callbacks=[eval_callback]
)


In [ ]:
import numpy as np

class TestController:
    def __init__(self):
        self.observation_space = spaces.Box(low=0, high=2, shape=(278,), dtype=np.float32)
        self.irradiance_profile = np.ones(8760) * 0.5  # For simplicity, set irradiance to 0.5 for all hours
        self.KWrated = 100
        self.current_step = 0
        self.action_space_dict = {
            f"agent_{i}": spaces.Box(
                low=np.concatenate(([PV_KW_ACTION_LOW] * 6, [PV_KVAR_ACTION_LOW] * 6, [BESS_KW_ACTION_LOW] * 6)),
                high=np.concatenate(([PV_KW_ACTION_HIGH] * 6, [PV_KVAR_ACTION_HIGH] * 6, [BESS_KW_ACTION_HIGH] * 6)),
                shape=(18,), dtype=np.float32) for i in range(5)
        }

        self.observation_space_dict = {}
        for i in range(4):  # For the first four agents
            self.observation_space_dict[f'agent_{i}'] = spaces.Box(low=0, high=2, shape=(50,), dtype=np.float32)
        # For the last agent
        self.observation_space_dict['agent_4'] = spaces.Box(low=0, high=2, shape=(78,), dtype=np.float32)
        
        self._agent_ids = [f'agent_{i}' for i in range(5)]
    
    def _take_action(self, action_dict):
    # Define total actions per agent
        TOTAL_ACTIONS_PER_AGENT = 6 + 6 + 6

    # Iterate through the action_dict
        for agent_id, action in action_dict.items():
            agent_idx = int(agent_id.split('_')[1])

        # Determine action segments for this agent
            pv_kw_actions = action[:6]
            pv_kvar_actions = action[6:12]
            bess_kw_actions = action[12:18]

        # Index offset based on agent_idx for PV and battery control
            pv_idx_offset = agent_idx * 6
            battery_idx_offset = agent_idx * 6

        # Actions for PVs
            for idx, (kw_action_value, kvar_action_value) in enumerate(zip(pv_kw_actions, pv_kvar_actions)):
            # handle kW actions
                irradiance = self.irradiance_profile[(self.current_step) % 8760]
                scaled_pv_kw = kw_action_value * irradiance
                if scaled_pv_kw > irradiance * self.KWrated:  # clip to max
                    scaled_pv_kw = irradiance * self.KWrated

            # handle kVAR actions
                pv_kvar = kvar_action_value
                S_max = 100  # Maximum apparent power (example value)
                q_max1 = np.sqrt(S_max**2 - np.power(scaled_pv_kw, 2))
                pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)
                print(f"edit PVSystem.PV{idx + pv_idx_offset + 1} phases=3 kV=4.16 kW={scaled_pv_kw} kVAR={pv_kvar}")

            for idx, action_value in enumerate(bess_kw_actions):
                print(f"edit Storage.Battery{idx + battery_idx_offset + 1} phases=3 kV=4.16 kW={action_value} kVAR=0")

        # Solve the power flow
        print("set controlmode=off")
        print("solve")


    

    
    def text(self, cmd):
        print(cmd)  # Mock function to print the commands

# Assuming you've already defined or imported spaces.Box from somewhere (like OpenAI Gym)
class spaces:
    class Box:
        def __init__(self, low, high, shape, dtype):
            self.low = low
            self.high = high
            self.shape = shape
            self.dtype = dtype

# defining lengths for each segment
PV_KVAR_ACTION_LEN = 30
PV_KW_ACTION_LEN = 30
BESS_KW_ACTION_LEN = 30
TRANSFORMER_TAPS_ACTION_LEN = 1
CAPACITOR_ACTION_LEN = 4

# defining values for each segment
PV_KVAR_ACTION_LOW = -80
PV_KVAR_ACTION_HIGH = 80

PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 100

BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 100

# Instantiate the test controller
controller = TestController()

# Create a mock action_dict
action_dict = {
    f"agent_{i}": np.random.rand(18)*100 for i in range(5)
}

# Call the function
controller._take_action(action_dict)


In [2]:


#Testing with partial observability of voltage violatiosn for each agent

import gymnasium as gym
from gymnasium import spaces
from gymnasium.spaces import Tuple, Dict

from ray.rllib.env.multi_agent_env import MultiAgentEnv
import numpy as np
from ray.rllib.algorithms.sac import SAC
from ray.rllib.algorithms.ppo import PPO
from ray.tune.registry import register_env
from ray import tune
import numpy as np

from py_dss_interface import DSSDLL
#import stable_baselines3
#from stable_baselines3 import SAC
import matplotlib.pyplot as plt
import csv
import pandas as pd
#from stable_baselines3 import A2C, DQN, PPO, TD3, SAC
import torch
import warnings
warnings.filterwarnings('ignore')
import logging
import os
import csv


logging.basicConfig(level=logging.ERROR)





device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import time

start_time = time.time()



# Initialize OpenDSS
dss = DSSDLL(r"C:\Program Files\OpenDSS")
dss_file = r"D:\Alaa_Selim\123Bus\IEEE123Master.dss"
dss.text(f"compile [{dss_file}]")

# defining lengths for each segment
PV_KVAR_ACTION_LEN = 30
PV_KW_ACTION_LEN = 30
BESS_KW_ACTION_LEN = 30
TRANSFORMER_TAPS_ACTION_LEN = 1
CAPACITOR_ACTION_LEN = 4

# defining values for each segment
PV_KVAR_ACTION_LOW = -80
PV_KVAR_ACTION_HIGH = 80

PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 100

BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 100

TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1

CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1


class PowerSystemEnv(gym.Env):
    def __init__(self, dss_path, dss_file, irradiance_csv_file,load_profile_file):
        super(PowerSystemEnv, self).__init__()
        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        self.ranked_buses = [
                                '1', '7', '8', '13', '21', '23', '29', '250', '35', '40', '42', '55',
                                '56', '65', '76', '78', '66', '79', '81', '83', '91', '95', '100',
                                '197', '300', '110', '135', '160', '152', '610'
                            ]
        self.capacitor_names = ["C83", "C88a", "C90b", "C92c"]
        self.KWrated=100
        
        # Apply actions to PV systems and batteries
        for i in range(30):
            bus = self.ranked_buses[i]
            self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=100 Pmpp=80")
            self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=100 kVAR=0")
        
        with open(irradiance_csv_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            self.irradiance_profile = [float(row[0]) for row in reader]
    
        with open(load_profile_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader, None)  # Skip the header
            self.load_profile = [float(row[0]) for row in reader]
            
        

        # Define action and observation space
        # Actions are continuous values for pv_kvar, pv_kW, battery_kw, transformer_tap and capacitor_states
        
        self.action_space = spaces.Box(
            low=np.array(
                [PV_KVAR_ACTION_LOW]*PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_LOW]*PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_LOW]*BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_LOW]*TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_LOW]*CAPACITOR_ACTION_LEN
            ),
            high=np.array(
                [PV_KVAR_ACTION_HIGH]*PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_HIGH]*PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_HIGH]*BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_HIGH]*TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_HIGH]*CAPACITOR_ACTION_LEN
            ),
            dtype=np.float32
        )
        # Assuming observation space is the voltage at each bus
        self.observation_space = spaces.Box(low=0, high=100000, shape=(278,), dtype=np.float32)  # Modified shape
        
        self.current_step = 0

        
         # Initialize control step counter
        self.control_steps = 0

        # Maximum control steps allowed in one episode
        self.max_control_steps =0  # for example
        self.hourly_violations_count = []
        
        
    def update_csv(self, current_step, voltage_violations, pv_kvar, pv_kw, battery_kw, transformer_tap, capacitor_states):
        csv_file = 'training_data_all.csv'
        file_exists = os.path.isfile(csv_file)

        with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)

            # If the file doesn't exist, write the header
            if not file_exists:
                header = ['current_step', 'voltage_violations']
                header.extend([f'pv_kvar_{i}' for i in range(30)])
                header.extend([f'pv_kw_{i}' for i in range(30)])
                header.extend([f'battery_kw_{i}' for i in range(30)])
                header.append('transformer_tap')
                header.extend([f'capacitor_states_{i}' for i in range(len(capacitor_states))])  # Adjust the range if necessary

                writer.writerow(header)

            # Write the data row
            row = [current_step, voltage_violations]
            row.extend(pv_kvar)
            row.extend(pv_kw)
            row.extend(battery_kw)
            row.append(transformer_tap)
            row.extend(capacitor_states)

            writer.writerow(row)




    def _take_action(self, action):
        
        pv_kvar = action[:30].copy()
        pv_kw = action[30:60].copy()
        battery_kw = action[60:90].copy()
        transformer_tap = action[90].copy()
        capacitor_states = action[91:].copy()
        capacitor_states = np.round(capacitor_states)

        
        
        pv_kw_limits = [0, 100]  # Limits for PV kW (example values)
        q_max1_limits = [-80, 80]  # Limits for maximum reactive power (example values)
    
        irradiance = self.irradiance_profile[self.current_step % 8760]
        for z in range (0,30):
            if  pv_kw[z] <= irradiance* self.KWrated:
            # no clipping
                pv_kw[z] = pv_kw[z]
                #pv_kvar[z]  = np.sqrt(100**2 - np.power(pv_kw[z], 2))
            else:
                pv_kw[z] = irradiance* self.KWrated
     
        #pv_kw = np.clip(action[30:60], pv_kw_limits[0], pv_kw_limits[1])
    
        S_max = 100  # Maximum apparent power (example value)

        q_max1 = np.sqrt(S_max**2 - np.power(pv_kw, 2))
        
        #pv_kvar= 0.8 ** pv_kw
        #add checking condition on action space, or you can use PU. system or use clip action (make a constrined actio space)
           
    # Make pv_kvar controlled based on q_max1
        pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)
        
         
        
       
        # Apply actions to PV systems and batteries
        for i in range(30):
            bus = self.ranked_buses[i]
            scaled_pv_kw = pv_kw[i] 
            #* irradiance
          
            
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar[i]} Pmpp={scaled_pv_kw}")
            # need to add KVA in OpenDSS model
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw[i]} kVAR=0")

        # Apply transformer control
        self.controller.text(f"edit Transformer.reg1a taps={transformer_tap}")

        # Apply capacitor control
        for i, cap_name in enumerate(self.capacitor_names):
            self.controller.text(f"edit Capacitor.{cap_name} states={int(capacitor_states[i])}")
        #add control mode = off

        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve") 
        #print(f'dss.circuit_losses(): {dss.circuit_losses()}')


   


    def step(self, action):
    # Execute the action
        self._take_action(action)

    # Calculate the rewards
        losses = sum(self.controller.circuit_losses())
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        voltage_violations = sum(1 for v in all_bus_voltages if v <= 0.95 or v >= 1.05 and v != 0)

    # Calculate the sum of squared voltage deviations from 1. This penalizes larger deviations more heavily.
        voltage_deviations = sum((v - 1)**2 for v in all_bus_voltages)

    # Define penalty weights for different components
        w_deviation = 1.0
        w_violation = 5.0
        w_loss = 0.001

    # Combine losses, violations, and deviations in the reward
        reward = -voltage_violations 
        self.hourly_violations_count.append(voltage_violations)
       # print(f"violations count: {voltage_violations}")
        # Extract the action parameters
        pv_kvar = action[:30].copy()
        pv_kw = action[30:60].copy()
        battery_kw = action[60:90].copy()
        transformer_tap = action[90].copy()
        capacitor_states = action[91:].copy()
        capacitor_states = np.round(capacitor_states)

        # Call update_csv with all the necessary data
        self.update_csv(self.current_step, voltage_violations, pv_kvar, pv_kw, battery_kw, transformer_tap, capacitor_states.tolist())


    # Gather observations
        observation = self.get_observation()

        termination = (self.control_steps >= self.max_control_steps) or (voltage_violations == 0)

    # After taking action, increment control steps
        self.control_steps += 1

        return observation, reward, termination, False, {}


   

    def reset(self, *, seed=None, options=None):
    # If a seed is provided, set the random seed for numpy
        if seed is not None:
            seed = 1 
            np.random.seed(seed)

    # You can use options to customize the reset function if needed
    # For now, we'll just print the options
        if options is not None:
            print(f"Reset options: {options}")
        # Reset power system to initial state
        self.controller.text(f"compile [{dss_file}]")
        
        #self.current_step = int(np.clip(np.random.normal(loc=0, scale=8760), 0, 8760))
        #self.current_step = np.random.randint(0, 8761)
        self.current_step = (self.current_step + 1) % 8760
        #print(f"current_step : {self.current_step}")


       

 # assuming the profile has 8760 hours  #use gaussian distribution
        self.control_steps = 0


     
        
        
         # Load names
        load_names = [
            "S1a", "S2b", "S4c", "S5c", "S6c", "S7a", "S9a", "S10a", "S11a", "S12b",
            "S16c", "S17c", "S19a", "S20a", "S22b", "S24c", "S28a", "S29a", "S30c", "S31c",
            "S32c", "S33a", "S34c", "S35a", "S37a", "S38b", "S39b", "S41c", "S42a", "S43b",
            "S45a", "S46a", "S47", "S48", "S49a", "S49b", "S49c", "S50c", "S51a", "S52a",
            "S53a", "S55a", "S56b", "S58b", "S59b", "S60a", "S62c", "S63a", "S64b", "S65a",
            "S65b", "S65c", "S66c", "S68a", "S69a", "S70a", "S71a", "S73c", "S74c", "S75c",
            "S76a", "S76b", "S76c", "S77b", "S79a", "S80b", "S82a", "S83c", "S84c", "S85c",
            "S86b"
            ]
  
            



        # Read CSV file into list
        #load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).tolist()
        # Convert the first column to a list
        load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).iloc[:, 0].tolist()


        # Create a dictionary with load names as keys and load powers as values
        load_dict = dict(zip(load_names, load_powers))

# Generate load scales using a Gaussian distribution
        load_scales = np.random.normal(loc=self.load_profile[self.current_step % 8760], scale=0.001, size=len(load_names))

        for load_name, load_scale in zip(load_names, load_scales):
    # Get the load power corresponding to load_name from the dictionary
            load_power = load_dict[load_name]
    
    # Multiply load power by load_scale
            result = load_power * load_scale
    
    # Use the result in your controller
            self.controller.text(f"edit Load.{load_name} kW={result}")

            
            
        # Load the irradiance for the current hour
        
        
        irradiance = self.irradiance_profile[self.current_step % 8760]
        

    
        

        # Initialize random states for controlled devices
        for i, bus in enumerate(self.ranked_buses):
            # Randomly initialize PV systems and batteries
            #kvar should be set to very small
            fixed_power_factor = 0.9  # Set a fixed power factor value (e.g., 0.9)
            pv_kw = 100 * self.irradiance_profile[self.current_step % 8760]  # Scale PV kW by irradiance
            power_factor_angle = np.arccos(fixed_power_factor)  # Calculate the angle corresponding to the power factor
            pv_kvar = pv_kw * np.tan(power_factor_angle)  # Calculate reactive power (kVAR) based on kW and power factor
            battery_kw = self.load_profile[self.current_step % 8760]*100  # Scale Battery kW by load
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0") #select charging or discharging
            #make a comparison study beween different modes
            #investigate batteries charging/dicharging pattern




        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        
        # Get the observations for all agents
        observations = self.get_observation()

    # You can include any additional information here. If there's nothing, just return an empty dictionary.
        infos = {}

        return observations, infos


    def render(self, mode='human'):
        pass

    def close(self):
        pass

    def get_observation(self):
    # Get the voltage at all buses
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
    
    # Flatten the list of bus voltages
        all_bus_voltages = np.array(all_bus_voltages).flatten()
    
    # As there's only one agent, you should decide the observation length. 
    # If you'd like to keep the previous structure, the agent might observe the first 50, or all of them.
    # Here, I'm assuming the agent observes all of them:
        agent_observation = all_bus_voltages

        return agent_observation

    def _end_of_episode(self):
        return self.current_step >= len(self.irradiance_profile)

    

irradiance_csv_file = r"D:\Alaa_Selim\Irradiance_Profile_Santa_Clara.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"

    
    
original_env = PowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)


#from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility

#env = MultiAgentEnvCompatibility(original_env)

def env_creator(env_config):
    return PowerSystemEnv(
                            dss_path=r"C:\Program Files\OpenDSS",
                            dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
                            irradiance_csv_file=irradiance_csv_file,
                            load_profile_file=load_profile_file
                            )

register_env("PowerSystemEnv", env_creator)







2024-01-29 17:42:52,138	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2024-01-29 17:42:54,253	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2024-01-29 17:42:54,261	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!


OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 


OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 




In [6]:
import numpy as np
import matplotlib.pyplot as plt
from qpso import QDPSO
start_time = time.time()
# Create an instance of the environment
env = PowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)


# Constants
T = 48  # Time horizon

# Lists to capture actions and states for every time step
all_actions = []
all_states = []

# Modify the objective function
def f(x):
    cumulative_reward = 0
    env.reset()
    
    # Reshape and clip the action sequence according to the environment
    actions = x.reshape((T, env.action_space.shape[0]))
    actions = np.clip(actions, env.action_space.low, env.action_space.high)
    
    for t in range(T):
        obs, reward, done, truncated, info = env.step(actions[t])
       # reward = np.clip(reward, -1e, 1e1)
        cumulative_reward += reward
        if done:
            break
    
    # This is to gather actions for the last iteration
    if len(all_actions) < T:
        all_actions.extend(actions)
    
    return -cumulative_reward

# Define the number of particles, iterations, and dimensions
NParticle = 10
MaxIters = 100
NDim = T * env.action_space.shape[0]

# Define the bounds for the optimization
low_bound = np.tile(env.action_space.low, T)
high_bound = np.tile(env.action_space.high, T)
bounds = list(zip(low_bound, high_bound))


# Quantum behavior parameter
g = 0.99

# Initialize the optimizer
optimizer = QDPSO(f, NParticle, NDim, bounds, MaxIters, g)

# Define a callback function for logging
rewards = []

def log(s):
    rewards.append(-s.gbest_value)
    if s.iters % 10 == 0:
        print("{0: >5}  {1: >9}  {2: >9}".format("Iters.", "Best Reward", "Best(Mean Reward)"))
        best_mean = np.mean([p.best_value for p in optimizer.particles()])
        print("{0: >5}  {1: >9.3E}  {2: >9.3E}".format(s.iters, -s.gbest_value, -best_mean))

# Perform optimization
optimizer.update(callback=log, interval=1)

# Plotting the convergence of rewards
#plt.figure(figsize=(10,5))
#plt.plot(rewards)
#plt.xlabel("Iteration")
#plt.ylabel("Best Reward Value")
#plt.title("Convergence of QDPSO's Best Reward")
#plt.grid(True)

#plt.savefig('ConvergenceofQDPSO_1.pdf', bbox_inches='tight', dpi=1200)
#plt.show()

# Get the actions of the last iteration
best_actions_last_iter = all_actions[-T:]

# Record states and actions of the best solution from the last iteration
env.reset()
all_states_last_iter = []

#for action in best_actions_last_iter:
 #   if env.current_step >= T:  # Ensure we don't go beyond the time horizon
  #      break
   # obs, _, _, _ = env.step(action)
   # all_states_last_iter.append(obs)
    
end_time = time.time()
print(f"Total optimization time: {end_time - start_time} seconds")




OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 


Iters.  Best Reward  Best(Mean Reward)
    0  0.000E+00  -5.060E+01
Iters.  Best Reward  Best(Mean Reward)
   10  0.000E+00  -3.000E-01
Iters.  Best Reward  Best(Mean Reward)
   20  0.000E+00  -0.000E+00
Iters.  Best Reward  Best(Mean Reward)
   30  0.000E+00  -0.000E+00
Iters.  Best Reward  Best(Mean Reward)
   40  0.000E+00  -0.000E+00
Iters.  Best Reward  Best(Mean Reward)
   50  0.000E+00  -0.000E+00
Iters.  Best Reward  Best(Mean Reward)
   60  0.000E+00  -0.000E+00
Iters.  Best Reward  Best(Mean Reward)
   70  0.000E+00  -0.000E+00
Iters.  Best Reward  Best(Mean Reward)
   80  0.000E+00  -0.000E+00
Iters.  Best Reward  Best(Mean Reward)
   90  0.000E+00  -0.000E+00
Iters.  Best Reward  Best(Mean Reward)
  100  0.000E+00  -0.000E+00
Total optimization time: 36.88770341873169 seconds


In [9]:
pip install deap


  Obtaining dependency information for deap from https://files.pythonhosted.org/packages/38/6c/e240099dc857d38ec9e0eca7c020c3e18274a873e0f2ac3004e3b8783156/deap-1.4.1-cp310-cp310-win_amd64.whl.metadata
   ---------------------------------------- 0.0/109.3 kB ? eta -:--:--
   ----------------------------- ---------- 81.9/109.3 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 109.3/109.3 kB 3.1 MB/s eta 0:00:00


In [10]:
import numpy as np
import random
import time
from deap import base, creator, tools, algorithms
#from your_environment import PowerSystemEnv  # Make sure to import your environment correctly

# Initialize the environment
env = PowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)

# Constants
T = 48  # Time horizon
NDim = T * env.action_space.shape[0]
NParticle = 50  # Population size
NGEN = 200  # Number of generations

# Define bounds
low_bound = np.tile(env.action_space.low, T)
high_bound = np.tile(env.action_space.high, T)

# Fitness function
def f(individual):
    cumulative_reward = 0
    env.reset()
    actions = np.clip(np.array(individual).reshape(T, env.action_space.shape[0]), env.action_space.low, env.action_space.high)
    for t in range(T):
        obs, reward, done, truncated, info = env.step(actions[t])
        cumulative_reward += reward
        if done:
            break
    return -cumulative_reward,

# Set up GA
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, low_bound[0], high_bound[0])
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, NDim)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", f)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# Optimization
start_time = time.time()
population = toolbox.population(n=NParticle)
final_population = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=NGEN, verbose=False)[0]
best_ind = tools.selBest(final_population, 1)[0]
end_time = time.time()

print(f"Best individual: {best_ind}")
print(f"Best fitness: {best_ind.fitness.values[0]}")
print(f"Total optimization time: {end_time - start_time} seconds")


OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 


Best individual: [38.394895794168946, 25.57995736819936, -74.88319126747206, -22.59022727033196, -4.624519985558719, -79.56960601181135, -24.34057846498171, -13.659610511253167, 31.195564303490663, -55.145808528696186, 19.77736276202407, 69.4117372419064, -48.74849317632637, -54.72706969458532, 76.09303044641803, -6.637888282064422, -9.744515512657516, 24.80738926688001, -68.7320038270917, -28.97387118850586, 77.88091276383962, -75.34732613121325, -40.12608616682208, -28.15666833219092, 10.79773938180807, -37.146177171992456, 54.37118142253803, -79.92809718979365, 38.16528978326691, 60.91738133433588, -36.54765914119715, 27.37398391327251, -35.072022972517885, 48.68182975462453, 59.10034605730946, -69.08259807413485, 25.878029012269938, -3.0012928877758185, -59.02618529541037, 26.547450311238673, -46.296943963744575, 62.67831035393303, 33.63354001033733, -34.16413147308402, -6.480313371915565

In [11]:
pip install pyswarms

  Using cached pyswarms-1.3.0-py2.py3-none-any.whl (104 kB)
Note: you may need to restart the kernel to use updated packages.


In [16]:
import numpy as np
import pyswarms as ps
import time

# Initialize the environment
env = PowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)

# Constants
T = 48  # Time horizon
NDim = T * env.action_space.shape[0]
NParticle = 10 # Number of particles
MaxIters = 2  # Number of iterations

# Define bounds
bounds = (np.tile(env.action_space.low, T), np.tile(env.action_space.high, T))

# Fitness function
def f(x):
    n_particles = x.shape[0]
    j = [0] * n_particles
    for i in range(n_particles):
        cumulative_reward = 0
        env.reset()
        actions = np.clip(x[i].reshape(T, env.action_space.shape[0]), env.action_space.low, env.action_space.high)
        for t in range(T):
            obs, reward, done, truncated, info = env.step(actions[t])
            cumulative_reward += reward
            if done:
                break
        j[i] = -cumulative_reward
    return np.array(j)

# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
optimizer = ps.single.GlobalBestPSO(n_particles=NParticle, dimensions=NDim, options=options, bounds=bounds)

# Perform optimization
start_time = time.time()
cost, pos = optimizer.optimize(f, iters=MaxIters)
end_time = time.time()

print(f"Best position: {pos}")
print(f"Best cost: {cost}")
print(f"Total optimization time: {end_time - start_time} seconds")


2024-01-29 20:50:02,388 - pyswarms.single.global_best - INFO - Optimize for 2 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 




pyswarms.single.global_best: 100%|████████████████████████████████████████████████████████████████████|2/2, best_cost=0
2024-01-29 20:50:02,624 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0, best pos: [-29.84088979  -5.66003959 -59.33461352 ...   0.19653358   0.24299219
   0.54092495]


Best position: [-29.84088979  -5.66003959 -59.33461352 ...   0.19653358   0.24299219
   0.54092495]
Best cost: 0.0
Total optimization time: 0.23698687553405762 seconds


In [23]:
import numpy as np
import pyswarms as ps
import time

# Initialize the environment
env = PowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)

# Constants
T = 48  # Time horizon
NDim = T * env.action_space.shape[0]
NParticle = 1000  # Number of particles
MaxIters = 10  # Number of iterations

# Define bounds
bounds = (np.tile(env.action_space.low, T), np.tile(env.action_space.high, T))

# Fitness function
def f(x):
    n_particles = x.shape[0]
    j = [0] * n_particles
    for i in range(n_particles):
        cumulative_reward = 0
        env.reset()
        actions = np.clip(x[i].reshape(T, env.action_space.shape[0]), env.action_space.low, env.action_space.high)
        for t in range(T):
            obs, reward, done, truncated, info = env.step(actions[t])
            cumulative_reward += reward
            if done:
                break
        j[i] = -cumulative_reward
    return np.array(j)

# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
optimizer = ps.single.GlobalBestPSO(n_particles=NParticle, dimensions=NDim, options=options, bounds=bounds)

# Perform optimization
start_time = time.time()
cost, pos = optimizer.optimize(f, iters=MaxIters, verbose=True)
end_time = time.time()

# Compute average cost
average_cost = np.mean(optimizer.cost_history)

print(f"Best position: {pos}")
print(f"Size of best position vector: {pos.shape}")
print(f"Best cost: {cost}")
print(f"Average cost: {average_cost}")
print(f"Total optimization time: {end_time - start_time} seconds")


OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 




2024-01-29 20:58:47,527 - pyswarms.single.global_best - INFO - Optimize for 10 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████████████|10/10, best_cost=0
2024-01-29 21:01:41,632 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0, best pos: [ 53.72559027  49.81846992 -13.19211692 ...   0.41939655   0.73399681
   0.77626116]


Best position: [ 53.72559027  49.81846992 -13.19211692 ...   0.41939655   0.73399681
   0.77626116]
Size of best position vector: (4560,)
Best cost: 0.0
Average cost: 0.0
Total optimization time: 174.1072781085968 seconds


In [29]:
import numpy as np
import pyswarms as ps
import time

# Assuming PowerSystemEnv and other necessary imports are correctly set up
# from your_environment import PowerSystemEnv

# Initialize the environment
env = PowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)

# Constants
T = 48  # Time horizon
NDim = T * env.action_space.shape[0]
NParticle = 100  # Number of particles
MaxIters = 1000  # Number of iterations

# Define bounds
bounds = (np.tile(env.action_space.low, T), np.tile(env.action_space.high, T))

# Fitness function
def f(x):
    n_particles = x.shape[0]
    j = np.zeros(n_particles)
    for i in range(n_particles):
        cumulative_reward = 0
        env.reset()
        actions = np.clip(x[i].reshape(T, env.action_space.shape[0]), env.action_space.low, env.action_space.high)
        for t in range(T):
            obs, reward, done, truncated, info = env.step(actions[t])
            cumulative_reward += reward
            if done:
                break
        j[i] = cumulative_reward
    return j

# Initialize swarm with adjusted hyperparameters for more exploration
options = {'c1': 1.0, 'c2': 0.5, 'w': 0.9}
optimizer = ps.single.GlobalBestPSO(n_particles=NParticle, dimensions=NDim, options=options, bounds=bounds)

start_time = time.time()

# Manually iterate through the optimization process
for i in range(MaxIters):
    optimizer.optimize(f, iters=1, verbose=False)  # Run one iteration at a time

    # Calculate the average reward for this iteration
    current_rewards = f(optimizer.swarm.position)
    average_reward = np.mean(current_rewards)
    
    print(f"Iteration {i+1}, Average Reward: {average_reward}")

end_time = time.time()

# Final results
best_cost = optimizer.get_cost_history()[-1]
best_position = optimizer.get_pos_history()[-1]

print(f"Final Best Position: {best_position}")
print(f"Final Best Cost: {best_cost}")
print(f"Total optimization time: {end_time - start_time} seconds")


OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 


Iteration 1, Average Reward: -144.864
Iteration 2, Average Reward: -144.167
Iteration 3, Average Reward: -155.552
Iteration 4, Average Reward: -151.402
Iteration 5, Average Reward: -149.658
Iteration 6, Average Reward: -144.266
Iteration 7, Average Reward: -142.753
Iteration 8, Average Reward: -147.425
Iteration 9, Average Reward: -146.578
Iteration 10, Average Reward: -146.834


AttributeError: 'GlobalBestPSO' object has no attribute 'get_cost_history'

In [33]:
import numpy as np
import pyswarms as ps
import time
# from your_environment import PowerSystemEnv  # Replace with your actual environment import

# Initialize the environment
env = PowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)

# Constants
T = 48  # Time horizon
NDim = T * env.action_space.shape[0]
NParticle = 100  # Number of particles
MaxIters = 10000  # Number of iterations

# Define bounds
bounds = (np.tile(env.action_space.low, T), np.tile(env.action_space.high, T))

# Fitness function
def f(x):
    n_particles = x.shape[0]
    j = np.zeros(n_particles)
    for i in range(n_particles):
        cumulative_reward = 0
        env.reset()
        actions = np.clip(x[i].reshape(T, env.action_space.shape[0]), env.action_space.low, env.action_space.high)
        for t in range(T):
            obs, reward, done, truncated, info = env.step(actions[t])
            cumulative_reward += reward
            if done:
                break
        j[i] = cumulative_reward
    return j

# Initialize swarm with adjusted hyperparameters for more exploration
options = {'c1': 1.0, 'c2': 0.5, 'w': 0.9}
optimizer = ps.single.GlobalBestPSO(n_particles=NParticle, dimensions=NDim, options=options, bounds=bounds)

start_time = time.time()

# Manually iterate through the optimization process
for i in range(MaxIters):
    optimizer.optimize(f, iters=1, verbose=False)  # Run one iteration at a time

    # Calculate the average reward for this iteration
    current_rewards = f(optimizer.swarm.position)
    average_reward = np.mean(current_rewards)
    
    print(f"Iteration {i+1}, Average Reward: {average_reward}")

end_time = time.time()

# Final results
best_cost = optimizer.cost_history[-1]  # Corrected attribute
best_position = optimizer.pos_history[-1]  # Corrected attribute

print(f"Final Best Position: {best_position}")
print(f"Final Best Cost: {best_cost}")
print(f"Total optimization time: {end_time - start_time} seconds")


OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 


Iteration 1, Average Reward: -169.27
Iteration 2, Average Reward: -131.73
Iteration 3, Average Reward: -149.74
Iteration 4, Average Reward: -143.47
Iteration 5, Average Reward: -150.48
Iteration 6, Average Reward: -148.01
Iteration 7, Average Reward: -130.86
Iteration 8, Average Reward: -125.29
Iteration 9, Average Reward: -144.15
Iteration 10, Average Reward: -136.0
Iteration 11, Average Reward: -137.75
Iteration 12, Average Reward: -142.56
Iteration 13, Average Reward: -146.86
Iteration 14, Average Reward: -150.69
Iteration 15, Average Reward: -146.98
Iteration 16, Average Reward: -158.22
Iteration 17, Average Reward: -139.64
Iteration 18, Average Reward: -152.73
Iteration 19, Average Reward: -137.17
Iteration 20, Average Reward: -149.03
Iteration 21, Average Reward: -136.13
Iteration 22, Average Reward: -144.28
Iteration 23, Average Reward: -146.88
Iteration 24, Average Reward: -135.25
Ite

Iteration 213, Average Reward: -132.37
Iteration 214, Average Reward: -129.2
Iteration 215, Average Reward: -129.07
Iteration 216, Average Reward: -123.35
Iteration 217, Average Reward: -122.4
Iteration 218, Average Reward: -138.61
Iteration 219, Average Reward: -131.0
Iteration 220, Average Reward: -114.39
Iteration 221, Average Reward: -139.63
Iteration 222, Average Reward: -149.89
Iteration 223, Average Reward: -142.58
Iteration 224, Average Reward: -135.78
Iteration 225, Average Reward: -119.53
Iteration 226, Average Reward: -124.95
Iteration 227, Average Reward: -125.94
Iteration 228, Average Reward: -119.54
Iteration 229, Average Reward: -115.85
Iteration 230, Average Reward: -126.96
Iteration 231, Average Reward: -132.44
Iteration 232, Average Reward: -125.68
Iteration 233, Average Reward: -122.17
Iteration 234, Average Reward: -130.71
Iteration 235, Average Reward: -120.3
Iteration 236, Average Reward: -125.53
Iteration 237, Average Reward: -131.76
Iteration 238, Average Reward

Iteration 424, Average Reward: -136.81
Iteration 425, Average Reward: -129.54
Iteration 426, Average Reward: -99.37
Iteration 427, Average Reward: -119.93
Iteration 428, Average Reward: -120.89
Iteration 429, Average Reward: -124.65
Iteration 430, Average Reward: -139.43
Iteration 431, Average Reward: -137.23
Iteration 432, Average Reward: -145.49
Iteration 433, Average Reward: -144.44
Iteration 434, Average Reward: -121.1
Iteration 435, Average Reward: -131.03
Iteration 436, Average Reward: -131.17
Iteration 437, Average Reward: -126.83
Iteration 438, Average Reward: -124.9
Iteration 439, Average Reward: -121.66
Iteration 440, Average Reward: -112.21
Iteration 441, Average Reward: -113.58
Iteration 442, Average Reward: -125.52
Iteration 443, Average Reward: -130.09
Iteration 444, Average Reward: -139.74
Iteration 445, Average Reward: -132.1
Iteration 446, Average Reward: -128.92
Iteration 447, Average Reward: -140.99
Iteration 448, Average Reward: -125.52
Iteration 449, Average Reward

Iteration 635, Average Reward: -117.43
Iteration 636, Average Reward: -126.56
Iteration 637, Average Reward: -120.25
Iteration 638, Average Reward: -103.58
Iteration 639, Average Reward: -113.25
Iteration 640, Average Reward: -125.98
Iteration 641, Average Reward: -128.42
Iteration 642, Average Reward: -129.7
Iteration 643, Average Reward: -144.54
Iteration 644, Average Reward: -145.03
Iteration 645, Average Reward: -114.76
Iteration 646, Average Reward: -111.81
Iteration 647, Average Reward: -127.49
Iteration 648, Average Reward: -103.45
Iteration 649, Average Reward: -104.47
Iteration 650, Average Reward: -125.25
Iteration 651, Average Reward: -127.43
Iteration 652, Average Reward: -129.09
Iteration 653, Average Reward: -135.11
Iteration 654, Average Reward: -139.52
Iteration 655, Average Reward: -125.65
Iteration 656, Average Reward: -120.93
Iteration 657, Average Reward: -114.14
Iteration 658, Average Reward: -119.67
Iteration 659, Average Reward: -115.54
Iteration 660, Average Rew

Iteration 846, Average Reward: -138.66
Iteration 847, Average Reward: -124.77
Iteration 848, Average Reward: -140.29
Iteration 849, Average Reward: -128.95
Iteration 850, Average Reward: -126.11
Iteration 851, Average Reward: -116.17
Iteration 852, Average Reward: -120.75
Iteration 853, Average Reward: -118.0
Iteration 854, Average Reward: -125.35
Iteration 855, Average Reward: -140.06
Iteration 856, Average Reward: -139.68
Iteration 857, Average Reward: -118.28
Iteration 858, Average Reward: -126.13
Iteration 859, Average Reward: -130.97
Iteration 860, Average Reward: -123.78
Iteration 861, Average Reward: -117.55
Iteration 862, Average Reward: -139.1
Iteration 863, Average Reward: -129.04
Iteration 864, Average Reward: -137.72
Iteration 865, Average Reward: -130.9
Iteration 866, Average Reward: -136.62
Iteration 867, Average Reward: -115.42
Iteration 868, Average Reward: -104.46
Iteration 869, Average Reward: -119.81
Iteration 870, Average Reward: -133.81
Iteration 871, Average Rewar

...

Iteration 9090, Average Reward: -120.85
Iteration 9091, Average Reward: -115.37
Iteration 9092, Average Reward: -116.12
Iteration 9093, Average Reward: -118.9
Iteration 9094, Average Reward: -110.89
Iteration 9095, Average Reward: -124.65
Iteration 9096, Average Reward: -121.34
Iteration 9097, Average Reward: -127.16
Iteration 9098, Average Reward: -131.75
Iteration 9099, Average Reward: -118.43
Iteration 9100, Average Reward: -94.49
Iteration 9101, Average Reward: -92.9
Iteration 9102, Average Reward: -102.38
Iteration 9103, Average Reward: -100.82
Iteration 9104, Average Reward: -134.03
Iteration 9105, Average Reward: -127.59
Iteration 9106, Average Reward: -126.74
Iteration 9107, Average Reward: -133.7
Iteration 9108, Average Reward: -142.68
Iteration 9109, Average Reward: -117.3
Iteration 9110, Average Reward: -119.95
Iteration 9111, Average Reward: -111.26
Iteration 9112, Average Reward: -115.17
Iteration 9113, Average Reward: -127.18
Iteration 9114, Average Reward: -125.27
Iterat

Iteration 9296, Average Reward: -117.9
Iteration 9297, Average Reward: -97.3
Iteration 9298, Average Reward: -101.79
Iteration 9299, Average Reward: -139.07
Iteration 9300, Average Reward: -133.33
Iteration 9301, Average Reward: -129.38
Iteration 9302, Average Reward: -135.92
Iteration 9303, Average Reward: -135.75
Iteration 9304, Average Reward: -105.25
Iteration 9305, Average Reward: -111.12
Iteration 9306, Average Reward: -127.32
Iteration 9307, Average Reward: -123.98
Iteration 9308, Average Reward: -125.8
Iteration 9309, Average Reward: -126.87
Iteration 9310, Average Reward: -117.36
Iteration 9311, Average Reward: -138.44
Iteration 9312, Average Reward: -140.78
Iteration 9313, Average Reward: -141.26
Iteration 9314, Average Reward: -121.8
Iteration 9315, Average Reward: -107.54
Iteration 9316, Average Reward: -113.49
Iteration 9317, Average Reward: -113.55
Iteration 9318, Average Reward: -107.95
Iteration 9319, Average Reward: -100.03
Iteration 9320, Average Reward: -125.1
Iterat

Iteration 9502, Average Reward: -126.05
Iteration 9503, Average Reward: -134.42
Iteration 9504, Average Reward: -134.22
Iteration 9505, Average Reward: -117.19
Iteration 9506, Average Reward: -107.57
Iteration 9507, Average Reward: -105.82
Iteration 9508, Average Reward: -111.08
Iteration 9509, Average Reward: -117.75
Iteration 9510, Average Reward: -122.28
Iteration 9511, Average Reward: -128.18
Iteration 9512, Average Reward: -131.04
Iteration 9513, Average Reward: -131.79
Iteration 9514, Average Reward: -117.99
Iteration 9515, Average Reward: -125.71
Iteration 9516, Average Reward: -124.32
Iteration 9517, Average Reward: -117.08
Iteration 9518, Average Reward: -118.29
Iteration 9519, Average Reward: -127.71
Iteration 9520, Average Reward: -123.15
Iteration 9521, Average Reward: -120.53
Iteration 9522, Average Reward: -120.8
Iteration 9523, Average Reward: -118.22
Iteration 9524, Average Reward: -112.77
Iteration 9525, Average Reward: -99.66
Iteration 9526, Average Reward: -108.2
Ite

Iteration 9708, Average Reward: -111.29
Iteration 9709, Average Reward: -115.6
Iteration 9710, Average Reward: -118.74
Iteration 9711, Average Reward: -127.05
Iteration 9712, Average Reward: -119.62
Iteration 9713, Average Reward: -120.09
Iteration 9714, Average Reward: -116.09
Iteration 9715, Average Reward: -112.6
Iteration 9716, Average Reward: -123.53
Iteration 9717, Average Reward: -129.19
Iteration 9718, Average Reward: -125.92
Iteration 9719, Average Reward: -129.3
Iteration 9720, Average Reward: -131.28
Iteration 9721, Average Reward: -135.81
Iteration 9722, Average Reward: -128.86
Iteration 9723, Average Reward: -127.73
Iteration 9724, Average Reward: -119.17
Iteration 9725, Average Reward: -115.43
Iteration 9726, Average Reward: -121.04
Iteration 9727, Average Reward: -111.58
Iteration 9728, Average Reward: -122.49
Iteration 9729, Average Reward: -129.41
Iteration 9730, Average Reward: -128.85
Iteration 9731, Average Reward: -117.81
Iteration 9732, Average Reward: -132.6
Iter

Iteration 9914, Average Reward: -114.44
Iteration 9915, Average Reward: -114.64
Iteration 9916, Average Reward: -128.12
Iteration 9917, Average Reward: -140.04
Iteration 9918, Average Reward: -137.19
Iteration 9919, Average Reward: -130.81
Iteration 9920, Average Reward: -141.26
Iteration 9921, Average Reward: -130.29
Iteration 9922, Average Reward: -113.57
Iteration 9923, Average Reward: -118.18
Iteration 9924, Average Reward: -120.25
Iteration 9925, Average Reward: -140.2
Iteration 9926, Average Reward: -126.4
Iteration 9927, Average Reward: -114.37
Iteration 9928, Average Reward: -114.94
Iteration 9929, Average Reward: -126.66
Iteration 9930, Average Reward: -131.11
Iteration 9931, Average Reward: -125.05
Iteration 9932, Average Reward: -111.22
Iteration 9933, Average Reward: -121.93
Iteration 9934, Average Reward: -120.35
Iteration 9935, Average Reward: -116.14
Iteration 9936, Average Reward: -129.45
Iteration 9937, Average Reward: -132.24
Iteration 9938, Average Reward: -124.65
It

In [ ]:
from ray.rllib.algorithms.impala import ImpalaConfig
from ray import air
from ray import tune
config = ImpalaConfig()
# Print out some default values.
print(config.vtrace)  
# Update the config object.
config.training( 
    lr=tune.grid_search([ 0.0005]), train_batch_size=tune.grid_search([ 2500]), minibatch_buffer_size = tune.grid_search([128]),
        num_sgd_iter = tune.grid_search([ 1]))
# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=10) 
config = config.environment(env="PowerSystemEnv")  
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "IMPALA",
    run_config=air.RunConfig(stop={"episodes_total": 100000}),
    param_space=config.to_dict(),
).fit()
# Build a Algorithm object from the config and run 1 training iteration.
algo = config.build(env="PowerSystemEnv")  


(pid=58652) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(Impala pid=58652) 2023-11-03 17:48:26,427	WARNING algorithm_config.py:656 -- Cannot create ImpalaConfig from given `config_dict`! Property __stdout_file__ not supported.
(pid=58160) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(pid=32988) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


(RolloutWorker pid=58160) OpenDSS Started successfully! 
(RolloutWorker pid=58160) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 
(RolloutWorker pid=58160) 
(RolloutWorker pid=58160) 
(RolloutWorker pid=58160) Reset options: {}


(RolloutWorker pid=19320) 2023-11-03 17:48:32,341	WARNING env.py:162 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=19320) 2023-11-03 17:48:32,412	WARNING deprecation.py:50 -- DeprecationWarning: `LearningRateSchedule` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=19320) 2023-11-03 17:48:32,415	WARNING deprecation.py:50 -- DeprecationWarning: `EntropyCoeffSchedule` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=19320) 2023-11-03 17:48:32,421	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=19320) 2023-11-03 17:48:32,421	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_modelv2.TorchModelV2` has been deprecated. Use `ray.rllib.core.rl_module.rl_module

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray import air
from ray import tune
config = PPOConfig()
# Print out some default values.
print(config.kl_coeff)   
# Update the config object.
config.training( 
    lr=tune.grid_search([0.001, 0.0001, 0.00001, 0.000001]))
# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=23) 
config = config.environment(env="PowerSystemEnv")
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner( 
    "PPO",
    run_config=air.RunConfig(stop={"episodes_total": 100000}),
    param_space=config.to_dict(),
).fit()

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray import air
from ray import tune
config = PPOConfig()
# Print out some default values.
print(config.kl_coeff)   
# Update the config object.
config.training( 
    lr=tune.grid_search([ 0.00001]))
# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=5) 
config = config.environment(env="PowerSystemEnv")
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner( 
    "PPO",
    run_config=air.RunConfig(stop={"episodes_total": 100000}),
    param_space=config.to_dict(),
).fit()

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray import air
from ray import tune
config = PPOConfig()
# Print out some default values.
print(config.kl_coeff)   
# Update the config object.
config.training( 
    lr=tune.grid_search([ 0.00001]))
# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=11) 
config = config.environment(env="PowerSystemEnv")
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner( 
    "PPO",
    run_config=air.RunConfig(stop={"episodes_total": 100000}),
    param_space=config.to_dict(),
).fit()

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray import air
from ray import tune
config = PPOConfig()
# Print out some default values.


print(config.kl_coeff)   
# Update the config object.
config.training( 
    lr=tune.grid_search([ 0.00001]), train_batch_size=tune.grid_search([2500]), num_sgd_iter = tune.grid_search([1]))

# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=7) 

config = config.environment(env="PowerSystemEnv")
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner( 
    "PPO",
    run_config=air.RunConfig(stop={"episodes_total": 100000}),
    param_space=config.to_dict(),
).fit()

In [6]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray import air
from ray import tune
config = PPOConfig()
# Print out some default values.
print(config.kl_coeff)   
# Update the config object.
config.training( 
    lr=tune.grid_search([ 0.0005]))
# Set the config object's env.
config = config.resources(num_gpus=1)  
config = config.rollouts(num_rollout_workers=23) 
config = config.environment(env="PowerSystemEnv")
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner( 
    "PPO",
    run_config=air.RunConfig(stop={"episodes_total": 200000}),
    param_space=config.to_dict(),
).fit()

2023-11-04 13:29:41,565	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=61752) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=61752) 2023-11-04 13:29:45,331	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config=

(RolloutWorker pid=55976) OpenDSS Started successfully! 
(RolloutWorker pid=55976) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 
(RolloutWorker pid=55976) 
(RolloutWorker pid=55976) 
(RolloutWorker pid=52832) Reset options: {}


(RolloutWorker pid=55976) 2023-11-04 13:29:52,218	WARNING deprecation.py:50 -- DeprecationWarning: `ValueNetworkMixin` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=55976) 2023-11-04 13:29:52,218	WARNING deprecation.py:50 -- DeprecationWarning: `LearningRateSchedule` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=55976) 2023-11-04 13:29:52,218	WARNING deprecation.py:50 -- DeprecationWarning: `EntropyCoeffSchedule` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=55976) 2023-11-04 13:29:52,218	WARNING deprecation.py:50 -- DeprecationWarning: `KLCoeffMixin` has been deprecated. This will raise an error in the future!
(PPO pid=61752) Install gputil for GPU system monitoring.
2023-11-04 13:34:44,013	WARNING syncer.py:586 -- Last sync command failed: Sync process failed: GetFileInfo() yielded path 'C:/Users/Alaa/ray_results/PPO', which is outside base dir 'C:\Users\Alaa\ray_results\PPO'
2023-11

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'__all__': {'num_agent_steps_trained': 128.0, 'num_env_steps_trained': 4000.0, 'total_loss': 0.4479674385237033}, 'default_policy': {'total_loss': 0.4479674385237033, 'policy_loss': -0.014584969107145027, 'vf_loss': 0.4084113155489664, 'vf_loss_unclipped': 5.908612712010392, 'vf_explained_var': 3.1882765959066623e-06, 'entropy': 134.4855212636594, 'mean_kl_loss': 0.007041669684725487, 'curr_lr': 0.0005, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 7.688672065734863}}, 'num_env_steps_sampled': 200000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 200000, 'num_agent_steps_trained': 0}, 'sampler_results': {'episode_reward_max': 0.0, 'episode_reward_min': -48.0, 'episode_reward_mean': -0.36125, 'episode_len_mean': 1.0, 'episode_media': {}, 'episodes_this_iter': 4000, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'epis

In [ ]:
import ray
import ray.rllib.agents.ppo as ppo
from ray import serve

def train_ppo_model():
    trainer = ppo.PPOTrainer(
        config={"framework": "torch", "num_workers": 0},
        env="CartPole-v0",
    )
    # Train for one iteration
    trainer.train()
    trainer.save("/tmp/rllib_checkpoint")
    return "/tmp/rllib_checkpoint/checkpoint_000001/checkpoint-1"


checkpoint_path = train_ppo_model()

In [ ]:
from ray.rllib.algorithms.impala import ImpalaConfig
config = ImpalaConfig()
config = config.training(lr=0.0003, train_batch_size=512)  
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=20)  
print(config.to_dict())  
# Build a Algorithm object from the config and run 1 training iteration.
algo = config.build(env="PowerSystemEnv")  
algo.train()  

In [ ]:
from ray.rllib.algorithms.mbmpo import MBMPOConfig
config = MBMPOConfig()
config = config.training(lr=0.0003, train_batch_size=512)  
config = config.resources(num_gpus=0) 
config = config.rollouts(num_rollout_workers=23)  
print(config.to_dict())  
# Build a Algorithm object from the config and run 1 training iteration.
algo = config.build(env="PowerSystemEnv")  
algo.train()  

In [ ]:
from ray.rllib.algorithms.mbmpo import MBMPOConfig
from ray import air
from ray import tune
config = MBMPOConfig()
# Print out some default values.
#print(config.vtrace)  
# Update the config object.
config = config.training(   
    lr=tune.grid_search([0.00001])
)
# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=23) 
config = config.environment(env="PowerSystemEnv") 
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "AlphaStar",
    run_config=air.RunConfig(stop={"episode_reward_mean":0}),
    param_space=config.to_dict(),
).fit()

In [ ]:
from ray.rllib.algorithms.ars import ARSConfig
from ray import air
from ray import tune
config = ARSConfig()
# Print out some default values.
print(config.action_noise_std)  
# Update the config object.
config = config.training(  
    rollouts_used=tune.grid_search([32, 64]), eval_prob=0.5)
# Set the config object's env.
config = config.environment(env="PowerSystemEnv")  
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "ARS",
    run_config=air.RunConfig(stop={"sampler_results/episode_reward_mean": 0}),
    param_space=config.to_dict(),
).fit()

In [ ]:
from ray.rllib.algorithms.es import ESConfig
from ray import tune
config = ESConfig()
# Print out some default values.
print(config.action_noise_std)  
# Update the config object.
config = config.training(eval_prob=0.5)
# Set the config object's env.
config = config.environment(env="PowerSystemEnv")  
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "ES",
    run_config=air.RunConfig(stop={"sampler_results/episode_reward_mean": 0}),
    param_space=config.to_dict(),
).fit()

In [ ]:
pip install higher

In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available! PyTorch is using GPU:", torch.cuda.get_device_name(0))

    # Get the current cuda device
    current_device = torch.cuda.current_device()
    print("Current device:", current_device)

    # Get the number of GPU devices available
    num_devices = torch.cuda.device_count()
    print("Number of devices available:", num_devices)

    # Get the name of the current device
    device_name = torch.cuda.get_device_name(current_device)
    print("Device name:", device_name)

    # Additional information, like memory allocated, can be retrieved like this
    memory_allocated = torch.cuda.memory_allocated(current_device)
    memory_cached = torch.cuda.memory_reserved(current_device)
    print("Memory Allocated (bytes):", memory_allocated)
    print("Memory Cached (bytes):", memory_cached)

else:
    print("CUDA is not available. PyTorch is using CPU.")

In [ ]:
from ray.rllib.algorithms.impala import ImpalaConfig
from ray import air
from ray import tune
config = ImpalaConfig()
# Print out some default values.
print(config.vtrace)  
# Update the config object.
config = config.training(   
    lr=tune.grid_search([0.0001])
)
# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=5) 
config = config.environment(env="PowerSystemEnv")  
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "IMPALA",
    run_config=air.RunConfig(stop={"episode_reward_mean": 0.0001}),
    param_space=config.to_dict(),
).fit()

In [ ]:
from ray.rllib.algorithms.impala import ImpalaConfig
from ray import air
from ray import tune
config = ImpalaConfig()
# Print out some default values.
print(config.vtrace)  
# Update the config object.
config = config.training(   
    lr=tune.grid_search([0.00001])
)
# Set the config object's env.
config = config.resources(num_gpus=1)  
config = config.rollouts(num_rollout_workers=1) 
config = config.environment(env="PowerSystemEnv")  
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "IMPALA",
    run_config=air.RunConfig(stop={"episode_reward_mean": -0.1}),
    param_space=config.to_dict(),
).fit()

In [ ]:
from ray.rllib.algorithms.impala import ImpalaConfig
from ray import air
from ray import tune
config = ImpalaConfig()
# Print out some default values.
print(config.vtrace)  
# Update the config object.
config.training( 
    lr=tune.grid_search([ 0.0001,0.0005,0.0009]), train_batch_size=tune.grid_search([ 2500]), minibatch_buffer_size = tune.grid_search([1, 128]),
        num_sgd_iter = tune.grid_search([ 1,30]))
# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=5) 
config = config.environment(env="PowerSystemEnv")  
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "IMPALA",
    run_config=air.RunConfig(stop={"episodes_total": 100000}),
    param_space=config.to_dict(),
).fit()
# Build a Algorithm object from the config and run 1 training iteration.
algo = config.build(env="PowerSystemEnv")  


In [ ]:
from ray.rllib.algorithms.impala import ImpalaConfig
from ray import air
from ray import tune
config = ImpalaConfig()
# Print out some default values.
print(config.vtrace)  
# Update the config object.
config.training( 
    lr=tune.grid_search([ 0.0001,0.0002,0.0003,0.0004, 0.0005]), train_batch_size=tune.grid_search([ 2500]), minibatch_buffer_size = tune.grid_search([1, 128]),
        num_sgd_iter = tune.grid_search([ 1,30]))
# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=5) 
config = config.environment(env="PowerSystemEnv")  
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "IMPALA",
    run_config=air.RunConfig(stop={"episodes_total": 100000}),
    param_space=config.to_dict(),
).fit()
# Build a Algorithm object from the config and run 1 training iteration.
algo = config.build(env="PowerSystemEnv")  


In [ ]:
from ray.rllib.algorithms.impala import ImpalaConfig
from ray import air
from ray import tune
config = ImpalaConfig()
# Print out some default values.
print(config.vtrace)  
# Update the config object.
config.training( 
    lr=tune.grid_search([ 0.0005]), train_batch_size=tune.grid_search([ 2500]), minibatch_buffer_size = tune.grid_search([128]),
        num_sgd_iter = tune.grid_search([ 1]))
# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=8) 
config = config.environment(env="PowerSystemEnv")  
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "IMPALA",
    run_config=air.RunConfig(stop={"episodes_total": 100000}),
    param_space=config.to_dict(),
).fit()
# Build a Algorithm object from the config and run 1 training iteration.
algo = config.build(env="PowerSystemEnv")  


In [ ]:
print(os.getcwd())


In [ ]:
from ray.rllib.algorithms.a3c import A3CConfig
from ray import air
from ray import tune
config = A3CConfig()
# Print out some default values.
#print(config.sample_async)  
# Update the config object.
config = config.training( 
    lr=tune.grid_search([ 0.0001]))
# Set the config object's env.
config = config.environment(env="PowerSystemEnv") 
config = config.rollouts(num_rollout_workers=23) 
config = config.resources(num_gpus=1)  


# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "A3C",
    run_config=air.RunConfig(stop={"episodes_total": 50000}),
    param_space=config.to_dict(),
).fit()

In [ ]:
tune.run(
    "SAC",
    stop={"episodes_total": 100000},
    config={
        "env": "PowerSystemEnv",
        "num_gpus": 1,
        "num_workers": 23,
        "lr": 3e-3
       # "alpha": 0.1,          # This decreases the temperature
        # Optionally adjust the target entropy as well
        # "target_entropy": some_value
    },
   local_dir="SAC-Test",
    checkpoint_freq=10,  # Save a checkpoint every 10 training iterations
    checkpoint_at_end=True  # Also save a checkpoint at the end of training
)

# Access the last checkpoint path
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean"),
    metric="episode_reward_mean"
)


In [ ]:
tune.run(
    "SAC",
    stop={"episodes_total": 100000},
    config={
        "env": "PowerSystemEnv",
        "num_gpus": 0,
        "num_workers": 23,
        "lr": 3e-3
       # "alpha": 0.1,          # This decreases the temperature
        # Optionally adjust the target entropy as well
        # "target_entropy": some_value
    },
   local_dir="SAC-Test",
    checkpoint_freq=10,  # Save a checkpoint every 10 training iterations
    checkpoint_at_end=True  # Also save a checkpoint at the end of training
)

# Access the last checkpoint path
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean"),
    metric="episode_reward_mean"
)


In [ ]:
tune.run(
    "SAC",
    stop={"episodes_total": 100000},
    config={
        "env": "PowerSystemEnv",
        "num_gpus": 0,
        "num_workers": 11,
        "lr": 3e-3
       # "alpha": 0.1,          # This decreases the temperature
        # Optionally adjust the target entropy as well
        # "target_entropy": some_value
    },
   local_dir="SAC-Test",
    checkpoint_freq=10,  # Save a checkpoint every 10 training iterations
    checkpoint_at_end=True  # Also save a checkpoint at the end of training
)

# Access the last checkpoint path
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean"),
    metric="episode_reward_mean"
)


In [ ]:
tune.run(
    "SAC",
    stop={"episodes_total": 100000},
    config={
        "env": "PowerSystemEnv",
        "num_gpus": 0,
        "num_workers": 5,
        "lr": 3e-3
       # "alpha": 0.1,          # This decreases the temperature
        # Optionally adjust the target entropy as well
        # "target_entropy": some_value
    },
   local_dir="SAC-Test",
    checkpoint_freq=10,  # Save a checkpoint every 10 training iterations
    checkpoint_at_end=True  # Also save a checkpoint at the end of training
)

# Access the last checkpoint path
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean"),
    metric="episode_reward_mean"
)

In [ ]:
tune.run(
    "DDPG",
    stop={"episodes_total": 50000},
    config={
        "env": "PowerSystemEnv",
        "num_gpus": 0,
        "num_workers": 5,
        "lr": 3e-3
       # "alpha": 0.1,          # This decreases the temperature
        # Optionally adjust the target entropy as well
        # "target_entropy": some_value
    },
   local_dir="SAC-Test",
    checkpoint_freq=10,  # Save a checkpoint every 10 training iterations
    checkpoint_at_end=True  # Also save a checkpoint at the end of training
)

# Access the last checkpoint path
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean"),
    metric="episode_reward_mean"
)


In [ ]:
from ray import air
from ray import tune
from ray.rllib.algorithms.dreamer import DreamerConfig
config = DreamerConfig()
# Print out some default values.
#print(config.clip_param)  
# Update the config object.
config = config.training(  
    lr=tune.grid_search([0.0001]))
# Set the config object's env.
# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=5) 
config = config.environment(env="PowerSystemEnv") # Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "Dreamer",
    run_config=air.RunConfig(stop={"episodes_total": 50000}),
    param_space=config.to_dict(),
).fit()

In [ ]:
from ray.rllib.algorithms.ars import ARSConfig
from ray import air
from ray import tune
config = ARSConfig()
# Print out some default values.
print(config.action_noise_std)  
# Update the config object.
config = config.training(  
    rollouts_used=tune.grid_search([32, 64]), eval_prob=0.5)
# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=5) 
config = config.environment(env="PowerSystemEnv") 
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "ARS",
    run_config=air.RunConfig(stop={"episodes_total": 50000}),
    param_space=config.to_dict(),
).fit()

In [ ]:
from ray import train, tune
from ray.rllib.algorithms.a2c import A2CConfig
config = A2CConfig()
# Print out some default values.
print(config.sample_async)   
# Update the config object.
config = config.training(lr=tune.grid_search(  
    [0.001, 0.0001]), use_critic=False)
# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=5) 
config = config.environment(env="PowerSystemEnv")  
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "A3C",
    run_config=air.RunConfig(stop={"episode_reward_mean": 0}),
    param_space=config.to_dict(),
).fit()

In [ ]:
from ray.rllib.algorithms.maml import MAMLConfig
from ray import air
from ray import tune
config = MAMLConfig()
# Print out some default values.
print(config.lr)  
# Update the config object.
config = config.training(  
    grad_clip=tune.grid_search([10.0, 40.0]))
# Set the config object's env.
# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=5) 
config = config.environment(env="PowerSystemEnv") # Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "MAML",
    run_config=air.RunConfig(stop={"episode_reward_mean": 0}),
    param_space=config.to_dict(),
).fit()

In [ ]:
from ray import tune
from ray.rllib.agents.sac import SACTrainer

# ... (your other imports and code)

def train_and_evaluate():
    # Create a trainer instance
    trainer = SACTrainer(
        config={
            "env": "PowerSystemEnv",
            "num_gpus": 0,
            "num_workers": 19,
            "lr": 3e-3
            # ... (any other config options)
        }
    )

    # Train the policy for a specified number of iterations
    for i in range(10):  # Adjust the range as needed
        result = trainer.train()
        print(f"Iteration {i}: {result}")

    # Save a checkpoint
    checkpoint_path = trainer.save()
    print(f"Checkpoint saved at: {checkpoint_path}")

    # Perform evaluation
    evaluation_results = trainer.evaluate()
    print(evaluation_results)

# Run the training and evaluation
train_and_evaluate()


In [ ]:
from ray.rllib.algorithms.td3 import TD3Config
from ray import air
from ray import tune
config = TD3Config()
# Print out some default values.
print(config.lr)   
# Update the config object.
config = config.training(lr=tune.grid_search(  
    [0.001, 0.0001]))
config = config.rollouts(num_rollout_workers=17)
# Set the config object's env.
config.environment(env="PowerSystemEnv")  
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "TD3",
    run_config=air.RunConfig(stop={"episode_reward_mean": 200}),
    param_space=config.to_dict(),
).fit()

In [ ]:
pip install --upgrade pandas

In [ ]:
pip install gymnasium  --upgrade

In [ ]:
pip install tensorflow==2.8.0 ray[rllib]==2.2.0 protobuf==3.20

In [ ]:
pip install py-dss-interface==1.0.2

In [ ]:
pip install gymnasium>=0.28.0

In [ ]:


#Testing with partial observability of voltage violatiosn for each agent

import gymnasium as gym
from gymnasium import spaces
from gymnasium.spaces import Tuple, Dict

from ray.rllib.env.multi_agent_env import MultiAgentEnv
import numpy as np
from ray.rllib.algorithms.sac import SAC
from ray.rllib.algorithms.ppo import PPO
from ray.tune.registry import register_env
from ray import tune
import numpy as np

from py_dss_interface import DSSDLL
import stable_baselines3
#from stable_baselines3 import SAC
import matplotlib.pyplot as plt
import csv
import pandas as pd
#from stable_baselines3 import A2C, DQN, PPO, TD3, SAC
import torch
import warnings
warnings.filterwarnings('ignore')
import logging

logging.basicConfig(level=logging.ERROR)





device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import time

start_time = time.time()



# Initialize OpenDSS
dss = DSSDLL(r"C:\Program Files\OpenDSS")
dss_file = r"D:\Alaa_Selim\123Bus\IEEE123Master.dss"
dss.text(f"compile [{dss_file}]")

# defining lengths for each segment
PV_KVAR_ACTION_LEN = 30
PV_KW_ACTION_LEN = 30
BESS_KW_ACTION_LEN = 30
TRANSFORMER_TAPS_ACTION_LEN = 1
CAPACITOR_ACTION_LEN = 4

# defining values for each segment
PV_KVAR_ACTION_LOW = -80
PV_KVAR_ACTION_HIGH = 80

PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 100

BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 100

TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1

CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1


class PowerSystemEnv(MultiAgentEnv):
    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file):
        super(PowerSystemEnv, self).__init__()

        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        self.ranked_buses  = ['50', '49', '51', '47', '48', '44', '57', '54', '64', '63', '55', '62', '56', '65', '43', '60', '42', '52', '53', '61', '66', '36', '40', '67', '72', '86', '87', '76', '89', '97']
        self.capacitor_names = ["C83", "C88a", "C90b", "C92c"]
        self.KWrated=100
         # Apply actions to PV systems and batteries
        for i in range(30):
            bus = self.ranked_buses[i]
            self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=100 Pmpp=80")
            self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=100 kVAR=0")
        
        with open(irradiance_csv_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            self.irradiance_profile = [float(row[0]) for row in reader]
    
        with open(load_profile_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader, None)  # Skip the header
            self.load_profile = [float(row[0]) for row in reader]
            
            
        self.action_space = spaces.Box(
            low=np.array(
                [PV_KVAR_ACTION_LOW]*PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_LOW]*PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_LOW]*BESS_KW_ACTION_LEN 
            ),
            high=np.array(
                [PV_KVAR_ACTION_HIGH]*PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_HIGH]*PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_HIGH]*BESS_KW_ACTION_LEN 
               
            ),
            dtype=np.float32
        )
            

     
        # Assuming observation space is the voltage at each bus
        self.observation_space = spaces.Box(low=0, high=10, shape=(278,), dtype=np.float32)  # Modified shape
            
         
        # Update the action and observation spaces for each agent
        self.action_space_dict = {
            f"agent_{i}": spaces.Box(
                low=np.concatenate(([PV_KW_ACTION_LOW] * 6, [PV_KVAR_ACTION_LOW] * 6, [BESS_KW_ACTION_LOW] * 6)),
                high=np.concatenate(([PV_KW_ACTION_HIGH] * 6, [PV_KVAR_ACTION_HIGH] * 6, [BESS_KW_ACTION_HIGH] * 6)),
                shape=(18,), dtype=np.float32) for i in range(5)
        }



        self.observation_space_dict = {}

        for i in range(4):  # For the first four agents
            self.observation_space_dict[f'agent_{i}'] = spaces.Box(low=0, high=2, shape=(50,), dtype=np.float32)

        # For the last agent
        self.observation_space_dict['agent_4'] = spaces.Box(low=0, high=2, shape=(78,), dtype=np.float32)

        
        self.current_step = 0

        
         # Initialize control step counter
        self.control_steps = int(0)

        # Maximum control steps allowed in one episode
        self.max_control_steps =int(3)  # for example
        
        



          # Define the agent IDs
        self._agent_ids = [f'agent_{i}' for i in range(5)]


        
   

        
    def _take_action(self, action_dict):
        # Iterate through the action_dict
        for agent_id, action in action_dict.items():
            idx_offset = int(agent_id.split('_')[1]) * 6

            # Actions for PVs
            for idx in range(6):  
                kw_action_value = action[idx]
                kvar_action_value = action[idx + 6]
                
                # handle kW actions
                irradiance = self.irradiance_profile[(self.current_step) % 8760]
                scaled_pv_kw = kw_action_value * irradiance
                if scaled_pv_kw > irradiance* self.KWrated:  # clip to 100
                    scaled_pv_kw = irradiance* self.KWrated
                
                # handle kVAR actions
                pv_kvar = kvar_action_value
                    
                S_max = 100  # Maximum apparent power (example value)

                q_max1 = np.sqrt(S_max**2 - np.power(scaled_pv_kw, 2))
    
                pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)
                    
                
                self.controller.text(f"edit PVSystem.PV{idx + idx_offset + 1} phases=3 kV=4.16 kW={scaled_pv_kw} kVAR={pv_kvar}")

                

            for idx, action_value in enumerate(action[12:18]):  
                    self.controller.text(f"edit Storage.Battery{idx + idx_offset + 1} phases=3 kV=4.16 kW={action_value} kVAR=0")

   
    # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")


    def step(self, action_dict):
    # Execute the action
        self._take_action(action_dict)

    # Calculate the rewards
        losses = sum(self.controller.circuit_losses())
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        voltage_violations = sum(1 for v in all_bus_voltages if v <= 0.95 or v >= 1.05)

    # Calculate the sum of absolute voltage deviations from 1
        voltage_deviations = sum(abs(v - 1) for v in all_bus_voltages)

    # Combine losses, violations, and deviations in the reward
        reward = - voltage_violations

    # Gather observations for all agents
        observations = {agent_id: self.get_observation(agent_id) for agent_id in self._agent_ids}

    # Set the reward for all agents
        rewards = {agent_id: reward for agent_id in self._agent_ids}

    # Set termination status for all agents
        terminations = {agent_id: (self.control_steps >= self.max_control_steps) or (voltage_deviations < 0.5) or (voltage_violations == 0) for agent_id in self._agent_ids}
        terminations['__all__'] = any(terminations.values())

    # Assuming no truncation; modify as needed
        truncateds = {agent_id: False for agent_id in self._agent_ids}
        truncateds['__all__'] = any(truncateds.values())

    # Gather additional information if required
        infos = {agent_id: {} for agent_id in self._agent_ids}

    # After taking action, increment control steps
        self.control_steps += 1

        return observations, rewards, terminations, truncateds, infos


   

    def reset(self, *, seed=None, options=None):
    # If a seed is provided, set the random seed for numpy
        if seed is not None:
            np.random.seed(seed)

    # You can use options to customize the reset function if needed
    # For now, we'll just print the options
        if options is not None:
            print(f"Reset options: {options}")
        # Reset power system to initial state
        self.controller.text(f"compile [{dss_file}]")
        
        #self.current_step = int(np.clip(np.random.normal(loc=0, scale=8760), 0, 8760))
        self.current_step = np.random.randint(0, 8761)
       

 # assuming the profile has 8760 hours  #use gaussian distribution
        self.control_steps = 0


     
        
        
         # Load names
        load_names = [
            "S1a", "S2b", "S4c", "S5c", "S6c", "S7a", "S9a", "S10a", "S11a", "S12b",
            "S16c", "S17c", "S19a", "S20a", "S22b", "S24c", "S28a", "S29a", "S30c", "S31c",
            "S32c", "S33a", "S34c", "S35a", "S37a", "S38b", "S39b", "S41c", "S42a", "S43b",
            "S45a", "S46a", "S47", "S48", "S49a", "S49b", "S49c", "S50c", "S51a", "S52a",
            "S53a", "S55a", "S56b", "S58b", "S59b", "S60a", "S62c", "S63a", "S64b", "S65a",
            "S65b", "S65c", "S66c", "S68a", "S69a", "S70a", "S71a", "S73c", "S74c", "S75c",
            "S76a", "S76b", "S76c", "S77b", "S79a", "S80b", "S82a", "S83c", "S84c", "S85c",
            "S86b"
            ]
  
            



        # Read CSV file into list
        load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None, squeeze=True).tolist()

        # Create a dictionary with load names as keys and load powers as values
        load_dict = dict(zip(load_names, load_powers))

# Generate load scales using a Gaussian distribution
        load_scales = np.random.normal(loc=self.load_profile[self.current_step % 8760], scale=0.5, size=len(load_names))

        for load_name, load_scale in zip(load_names, load_scales):
    # Get the load power corresponding to load_name from the dictionary
            load_power = load_dict[load_name]
    
    # Multiply load power by load_scale
            result = load_power * load_scale
    
    # Use the result in your controller
            self.controller.text(f"edit Load.{load_name} kW={result}")

            
            
        # Load the irradiance for the current hour
        
        
        irradiance = self.irradiance_profile[self.current_step % 8760]
        

    
        

        # Initialize random states for controlled devices
        for i, bus in enumerate(self.ranked_buses):
            # Randomly initialize PV systems and batteries
            #kvar should be set to very small
            fixed_power_factor = 0.9  # Set a fixed power factor value (e.g., 0.9)
            pv_kw = 100 * self.irradiance_profile[self.current_step % 8760]  # Scale PV kW by irradiance
            power_factor_angle = np.arccos(fixed_power_factor)  # Calculate the angle corresponding to the power factor
            pv_kvar = pv_kw * np.tan(power_factor_angle)  # Calculate reactive power (kVAR) based on kW and power factor
            battery_kw = self.load_profile[self.current_step % 8760]*100  # Scale Battery kW by load
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0") #select charging or discharging
            #make a comparison study beween different modes
            #investigate batteries charging/dicharging pattern




        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        
        # Get the observations for all agents
        observations = {agent_id: self.get_observation(agent_id) for agent_id in self._agent_ids}

    # You can include any additional information here. If there's nothing, just return an empty dictionary.
        infos = {}

        return observations, infos


    def render(self, mode='human'):
        pass

    def close(self):
        pass

    def get_observation(self, agent_id):
    # Get the voltage at all buses
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
    # Flatten the list of bus voltages
        all_bus_voltages = np.array(all_bus_voltages).flatten()

    # Extract agent index from the agent_id
        agent_idx = int(agent_id.split('_')[1])

    # Determine the start and end index for the observation chunk for this agent
        if agent_idx < 4:
            start_idx = agent_idx * 50
            end_idx = start_idx + 50
        else:  # This is agent_4
            start_idx = 200
            end_idx = 278  # or you could use end_idx = len(all_bus_voltages) if the size is not guaranteed to be 278

    # Extract the observation chunk for this agent
        agent_observation = all_bus_voltages[start_idx:end_idx]

        return agent_observation


    def get_agent_observation(self):
        agent_obs = {}
        for agent_id in self.agents:
            agent_obs[agent_id] = self.get_observation()
        return agent_obs

    def _end_of_episode(self):
        return self.current_step >= len(self.irradiance_profile)
    

irradiance_csv_file = r"D:\Alaa_Selim\Irradiance_Profile_Santa_Clara.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"

    
    
original_env = PowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)


from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility

env = MultiAgentEnvCompatibility(original_env)

def env_creator(env_config):
    return PowerSystemEnv(
                            dss_path=r"C:\Program Files\OpenDSS",
                            dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
                            irradiance_csv_file=irradiance_csv_file,
                            load_profile_file=load_profile_file
                            )

register_env("PowerSystemEnv", env_creator)



# Suppose you have N agents
N = 5  # adjust this as per your actual number of agents

policies = {
    f"policy_{i}": (None, 
                    original_env.observation_space_dict[f"agent_{i}"], 
                    original_env.action_space_dict[f"agent_{i}"], 
                    {}) 
    for i in range(N)
}


def policy_mapping_fn(agent_id, episode, **kwargs):
    if "agent_" in agent_id:
        # Extract agent number and return the corresponding policy
        agent_num = int(agent_id.split("_")[1])
        return f"policy_{agent_num}"
    else:
        raise ValueError(f"Unknown agent_id: {agent_id}")

def evaluate(agent, env_creator):
    env = env_creator({})
    obs_dict = env.reset()  
    done = {"__all__": False}
    total_rewards = {agent_id: 0 for agent_id in obs_dict.keys()}
    
    all_obs = []  # List to store observations for all agents at each time step
    all_actions = []  # List to store actions for all agents at each time step
    
    while not done["__all__"]:
        actions_dict = {}
        current_obs = []
        current_actions = []
        for agent_id, obs in obs_dict.items():
            policy_id = policy_mapping_fn(agent_id, None)
            action = agent.compute_action(obs, policy_id=policy_id)
            actions_dict[agent_id] = action
            
            current_obs.append(obs)
            current_actions.append(action)
        
        all_obs.append(current_obs)
        all_actions.append(current_actions)
        
        next_obs_dict, reward_dict, done, _ = env.step(actions_dict)
        
        for agent_id in obs_dict.keys():
            total_rewards[agent_id] += reward_dict.get(agent_id, 0)
            obs_dict[agent_id] = next_obs_dict[agent_id]
        
        done["__all__"] = all(done.values())

    avg_reward = np.mean(list(total_rewards.values()))
    
    return avg_reward, all_obs, all_actions


import os

local_dir = r'C:\Users\Alaa\ray_results\SAC'
os.makedirs(local_dir, exist_ok=True)

from ray.tune import Callback
import numpy as np



from ray.tune import Callback

class CustomEvalCallback(Callback):
    def __init__(self, eval_num_episodes=10):
        self.eval_num_episodes = eval_num_episodes
        self.eval_rewards = []
        self.eval_reward_maxes = []
        self.eval_reward_mins = []
        self.all_eval_obs = []
        self.all_eval_actions = []

    def on_training_iteration(self, iteration, trials, trial, result, **info):
        agent = trial.get_trainable()
        env_name = trial.config["env"]

        # Lists to capture episode-wise observations and actions
        episode_obs = []
        episode_actions = []

        # Evaluate the agent
        rewards = []
        for _ in range(self.eval_num_episodes):
            obs_dict, actions_dict = self._single_evaluation_episode(agent, env_name)
            rewards.append(np.mean(list(obs_dict.values())))
            episode_obs.append(obs_dict)
            episode_actions.append(actions_dict)

        mean_reward = np.mean(rewards)
        max_reward = np.max(rewards)
        min_reward = np.min(rewards)

        # Store results
        self.eval_rewards.append(mean_reward)
        self.eval_reward_maxes.append(max_reward)
        self.eval_reward_mins.append(min_reward)
        self.all_eval_obs.append(episode_obs)
        self.all_eval_actions.append(episode_actions)
        print("Mean Reward for current iteration:", mean_reward)
        print("Max Reward for current iteration:", max_reward)
        print("Min Reward for current iteration:", min_reward)


    def _single_evaluation_episode(self, agent, env_name):
        """Runs a single evaluation episode and returns observations and actions."""
        env = env_creator({})
        obs_dict = env.reset()
        done = {"__all__": False}
        episode_rewards = {agent_id: 0 for agent_id in obs_dict.keys()}
        episode_obs = {}
        episode_actions = {}

        while not done["__all__"]:
            actions_dict = {}
            for agent_id, obs in obs_dict.items():
                policy_id = policy_mapping_fn(agent_id, None)
                actions_dict[agent_id] = agent.compute_action(obs, policy_id=policy_id)

            next_obs_dict, reward_dict, done, _ = env.step(actions_dict)
            for agent_id in obs_dict.keys():
                episode_rewards[agent_id] += reward_dict.get(agent_id, 0)
                obs_dict[agent_id] = next_obs_dict[agent_id]
                
                episode_obs.setdefault(agent_id, []).append(obs)
                episode_actions.setdefault(agent_id, []).append(actions_dict[agent_id])

            done["__all__"] = all(done.values())

        print("Single Evaluation Episode Completed!")  # Debugging print
        print("Observations:", obs_dict)
        print("Actions:", actions_dict)
        print("Episode Rewards:", episode_rewards)

        return episode_rewards, episode_obs, episode_actions






eval_callback = CustomEvalCallback()

tune.run(
    "SAC",
    stop={"episode_reward_mean": 0},
    config={
        "env": "PowerSystemEnv",
        "environment": {
            "disable_env_checking": True
        },
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn
        },
        "num_gpus": 0,
        "num_workers": 17,
        "lr": 3e-4,
    },
    local_dir=local_dir,
    callbacks=[eval_callback]
)



print("Evaluation Rewards:", eval_callback.eval_rewards)
print("Max Evaluation Rewards:", eval_callback.eval_reward_maxes)
print("Min Evaluation Rewards:", eval_callback.eval_reward_mins)

import matplotlib.pyplot as plt

plt.fill_between(
    range(len(eval_callback.eval_rewards)), 
    eval_callback.eval_reward_mins, 
    eval_callback.eval_reward_maxes, 
    color='gray', alpha=0.4
)
plt.plot(eval_callback.eval_rewards, label='Mean Reward')
plt.xlabel('Training Iteration')
plt.ylabel('Evaluation Reward')
plt.title('Evaluation Rewards over Training')
plt.legend()
plt.grid(True)
plt.show()


if eval_callback.all_eval_obs:
    for i in range(N):  # Assuming N agents
        agent_obs = eval_callback.all_eval_obs[0].get(f"agent_{i}", [])
        
        plt.figure(figsize=(12, 6))
        plt.plot(agent_obs, label=f'Agent_{i} State')
        
        plt.legend()
        plt.xlabel('Time Steps')
        plt.ylabel('State Value')
        plt.title(f'Agent_{i} States over Time')
        plt.show()


if eval_callback.all_eval_actions:
    for i in range(N):  # Assuming N agents
        agent_actions = eval_callback.all_eval_actions[0].get(f"agent_{i}", [])
        
        plt.figure(figsize=(12, 6))
        plt.plot(agent_actions, label=f'Agent_{i} Action')
        
        plt.legend()
        plt.xlabel('Time Steps')
        plt.ylabel('Action Value')
        plt.title(f'Agent_{i} Actions over Time')
        plt.show()



In [ ]:
pip install tensorflow  protobuf==3.20

In [ ]:
pip install gymnasium

In [ ]:
pip install opencv-python

In [ ]:
pip install pyyaml

In [ ]:
pip install msgpack

In [ ]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'


In [ ]:


#Testing with partial observability of voltage violatiosn for each agent

import gymnasium as gym
from gymnasium import spaces
from gymnasium.spaces import Tuple, Dict

from ray.rllib.env.multi_agent_env import MultiAgentEnv
import numpy as np
from ray.rllib.algorithms.sac import SAC
from ray.rllib.algorithms.ppo import PPO
from ray.tune.registry import register_env
from ray import tune
import numpy as np

from py_dss_interface import DSSDLL
#import stable_baselines3
#from stable_baselines3 import SAC
import matplotlib.pyplot as plt
import csv
import pandas as pd
#from stable_baselines3 import A2C, DQN, PPO, TD3, SAC
import torch
import warnings
warnings.filterwarnings('ignore')
import logging

logging.basicConfig(level=logging.ERROR)





device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import time

start_time = time.time()



# Initialize OpenDSS
dss = DSSDLL(r"C:\Program Files\OpenDSS")
dss_file = r"D:\Alaa_Selim\123Bus\IEEE123Master.dss"
dss.text(f"compile [{dss_file}]")

# defining lengths for each segment
PV_KVAR_ACTION_LEN = 30
PV_KW_ACTION_LEN = 30
BESS_KW_ACTION_LEN = 30
TRANSFORMER_TAPS_ACTION_LEN = 1
CAPACITOR_ACTION_LEN = 4

# defining values for each segment
PV_KVAR_ACTION_LOW = -80
PV_KVAR_ACTION_HIGH = 80

PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 100

BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 100

TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1

CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1


class PowerSystemEnv(MultiAgentEnv):
    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file):
        super(PowerSystemEnv, self).__init__()

        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        self.ranked_buses  = ['50', '49', '51', '47', '48', '44', '57', '54', '64', '63', '55', '62', '56', '65', '43', '60', '42', '52', '53', '61', '66', '36', '40', '67', '72', '86', '87', '76', '89', '97']
        self.capacitor_names = ["C83", "C88a", "C90b", "C92c"]
        self.KWrated=100
        self.previous_reward = 0.0
        self.alpha = 0.1  #
         # Apply actions to PV systems and batteries
        for i in range(30):
            bus = self.ranked_buses[i]
            self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=100 Pmpp=80")
            self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=100 kVAR=0")
        
        with open(irradiance_csv_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            self.irradiance_profile = [float(row[0]) for row in reader]
    
        with open(load_profile_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader, None)  # Skip the header
            self.load_profile = [float(row[0]) for row in reader]
            
            
        self.action_space = spaces.Box(
            low=np.array(
                [PV_KVAR_ACTION_LOW]*PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_LOW]*PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_LOW]*BESS_KW_ACTION_LEN 
            ),
            high=np.array(
                [PV_KVAR_ACTION_HIGH]*PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_HIGH]*PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_HIGH]*BESS_KW_ACTION_LEN 
               
            ),
            dtype=np.float32
        )
            

     
        # Assuming observation space is the voltage at each bus
        self.observation_space = spaces.Box(low=0, high=2, shape=(278,), dtype=np.float32)  # Modified shape
            
         
        # Update the action and observation spaces for each agent
        self.action_space_dict = {
            f"agent_{i}": spaces.Box(
                low=np.concatenate(([PV_KW_ACTION_LOW] * 6, [PV_KVAR_ACTION_LOW] * 6, [BESS_KW_ACTION_LOW] * 6)),
                high=np.concatenate(([PV_KW_ACTION_HIGH] * 6, [PV_KVAR_ACTION_HIGH] * 6, [BESS_KW_ACTION_HIGH] * 6)),
                shape=(18,), dtype=np.float32) for i in range(5)
        }



        self.observation_space_dict = {}
        
        #self.observation_space_dict = {
              # agent_id: spaces.Box(low=0, high=2, shape=(278,), dtype=np.float32) for agent_id in self._agent_ids
              #  }

        
        

        for i in range(5):  # For the first four agents
            self.observation_space_dict[f'agent_{i}'] = spaces.Box(low=0, high=2, shape=(278,), dtype=np.float32)

        # For the last agent
        #self.observation_space_dict['agent_4'] = spaces.Box(low=0, high=2, shape=(78,), dtype=np.float32)

        
        self.current_step = 0

        
         # Initialize control step counter
        self.control_steps = 0

        # Maximum control steps allowed in one episode
        self.max_control_steps =int(2)  # for example
        
        



          # Define the agent IDs
        self._agent_ids = [f'agent_{i}' for i in range(5)]


        
   

        
    def _take_action(self, action_dict):
    # Define total actions per agent
        TOTAL_ACTIONS_PER_AGENT = 6 + 6 + 6

    # Iterate through the action_dict
        for agent_id, action in action_dict.items():
            agent_idx = int(agent_id.split('_')[1])

        # Determine action segments for this agent
            pv_kw_actions = action[:6]
            pv_kvar_actions = action[6:12]
            bess_kw_actions = action[12:18]

        # Index offset based on agent_idx for PV and battery control
            pv_idx_offset = agent_idx * 6
            battery_idx_offset = agent_idx * 6

        # Actions for PVs
            for idx, (kw_action_value, kvar_action_value) in enumerate(zip(pv_kw_actions, pv_kvar_actions)):
            # handle kW actions
                irradiance = self.irradiance_profile[(self.current_step) % 8760]
                scaled_pv_kw = kw_action_value * irradiance
                if scaled_pv_kw > irradiance * self.KWrated:  # clip to max
                    scaled_pv_kw = irradiance * self.KWrated

            # handle kVAR actions
                pv_kvar = kvar_action_value
                S_max = 100  # Maximum apparent power (example value)
                q_max1 = np.sqrt(S_max**2 - np.power(scaled_pv_kw, 2))
                pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)
                self.controller.text(f"edit PVSystem.PV{idx + pv_idx_offset + 1} phases=3 kV=4.16 Pmpp={scaled_pv_kw} kVAR={pv_kvar}")

            for idx, action_value in enumerate(bess_kw_actions):
                self.controller.text(f"edit Storage.Battery{idx + battery_idx_offset + 1} phases=3 kV=4.16 kW={action_value} kVAR=0")

        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")




    def step(self, action_dict):
    # Execute the action
        self._take_action(action_dict)

    # Calculate the rewards
        losses = sum(self.controller.circuit_losses())
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        voltage_violations = sum(1 for v in all_bus_voltages if v <= 0.95 or v >= 1.05)




        
        # Calculate the sum of squared voltage deviations from 1. This penalizes larger deviations more heavily.
        voltage_deviations = sum((v - 1)**2 for v in all_bus_voltages)



        # Define penalty weights for different components
        w_deviation = 1.0
        w_violation = 5.0
        w_loss = 0.001

        # Combine losses, violations, and deviations in the reward
        #reward = - w_deviation * voltage_deviations - w_violation * voltage_violations
        reward= - voltage_violations 
        
        #-  self.control_steps

        


    # Gather observations for all agents
        observations = {agent_id: self.get_observation(agent_id) for agent_id in self._agent_ids}

    # Set the reward for all agents
        rewards = {agent_id: reward for agent_id in self._agent_ids}

    # Set termination status for all agents
        terminations = {agent_id: (self.control_steps >= self.max_control_steps) or (voltage_violations == 0) for agent_id in self._agent_ids}
        terminations['__all__'] = any(terminations.values())

    # Assuming no truncation; modify as needed
        truncateds = {agent_id: False for agent_id in self._agent_ids}
        truncateds['__all__'] = any(truncateds.values())

    # Gather additional information if required
        infos = {agent_id: {} for agent_id in self._agent_ids}

    # After taking action, increment control steps
        self.control_steps += 1

        return observations, rewards, terminations, truncateds, infos


   

    def reset(self, *, seed=None, options=None):
    # If a seed is provided, set the random seed for numpy
        if seed is not None:
            np.random.seed(seed)

    # You can use options to customize the reset function if needed
    # For now, we'll just print the options
        if options is not None:
            print(f"Reset options: {options}")
        # Reset power system to initial state
        self.controller.text(f"compile [{dss_file}]")
        
        #self.current_step = int(np.clip(np.random.normal(loc=0, scale=8760), 0, 8760))
        self.current_step = np.random.randint(0, 8761)
       

 # assuming the profile has 8760 hours  #use gaussian distribution
        self.control_steps = 0


     
        
        
         # Load names
        load_names = [
            "S1a", "S2b", "S4c", "S5c", "S6c", "S7a", "S9a", "S10a", "S11a", "S12b",
            "S16c", "S17c", "S19a", "S20a", "S22b", "S24c", "S28a", "S29a", "S30c", "S31c",
            "S32c", "S33a", "S34c", "S35a", "S37a", "S38b", "S39b", "S41c", "S42a", "S43b",
            "S45a", "S46a", "S47", "S48", "S49a", "S49b", "S49c", "S50c", "S51a", "S52a",
            "S53a", "S55a", "S56b", "S58b", "S59b", "S60a", "S62c", "S63a", "S64b", "S65a",
            "S65b", "S65c", "S66c", "S68a", "S69a", "S70a", "S71a", "S73c", "S74c", "S75c",
            "S76a", "S76b", "S76c", "S77b", "S79a", "S80b", "S82a", "S83c", "S84c", "S85c",
            "S86b"
            ]
  
            



        # Read CSV file into list
        #load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).tolist()
        # Convert the first column to a list
        load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).iloc[:, 0].tolist()


        # Create a dictionary with load names as keys and load powers as values
        load_dict = dict(zip(load_names, load_powers))

# Generate load scales using a Gaussian distribution
        load_scales = np.random.normal(loc=self.load_profile[self.current_step % 8760], scale=0.5, size=len(load_names))

        for load_name, load_scale in zip(load_names, load_scales):
    # Get the load power corresponding to load_name from the dictionary
            load_power = load_dict[load_name]
    
    # Multiply load power by load_scale
            result = load_power * load_scale
    
    # Use the result in your controller
            self.controller.text(f"edit Load.{load_name} kW={result}")

            
            
        # Load the irradiance for the current hour
        
        
        irradiance = self.irradiance_profile[self.current_step % 8760]
        

    
        

        # Initialize random states for controlled devices
        for i, bus in enumerate(self.ranked_buses):
            # Randomly initialize PV systems and batteries
            #kvar should be set to very small
            fixed_power_factor = 0.9  # Set a fixed power factor value (e.g., 0.9)
            pv_kw = 100 * self.irradiance_profile[self.current_step % 8760]  # Scale PV kW by irradiance
            power_factor_angle = np.arccos(fixed_power_factor)  # Calculate the angle corresponding to the power factor
            pv_kvar = pv_kw * np.tan(power_factor_angle)  # Calculate reactive power (kVAR) based on kW and power factor
            battery_kw = self.load_profile[self.current_step % 8760]*100  # Scale Battery kW by load
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0") #select charging or discharging
            #make a comparison study beween different modes
            #investigate batteries charging/dicharging pattern




        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        
        # Get the observations for all agents
        observations = {agent_id: self.get_observation(agent_id) for agent_id in self._agent_ids}

    # You can include any additional information here. If there's nothing, just return an empty dictionary.
        infos = {}

        return observations, infos


    def render(self, mode='human'):
        pass

    def close(self):
        pass

    def get_observation(self, agent_id):
    # Get the voltage at all buses
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
    
    # Flatten the list of bus voltages
        all_bus_voltages = np.array(all_bus_voltages).flatten()
    
        return all_bus_voltages

    def get_agent_observation(self):
        agent_obs = {}
        for agent_id in self.agents:
            agent_obs[agent_id] = self.get_observation(agent_id)  # Note: We're still passing agent_id even if it's unused. This keeps the interface consistent.
        return agent_obs

    def _end_of_episode(self):
        return self.current_step >= len(self.irradiance_profile)

    

irradiance_csv_file = r"D:\Alaa_Selim\Irradiance_Profile_Santa_Clara.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"

    
    
original_env = PowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)


from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility

env = MultiAgentEnvCompatibility(original_env)

def env_creator(env_config):
    return PowerSystemEnv(
                            dss_path=r"C:\Program Files\OpenDSS",
                            dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
                            irradiance_csv_file=irradiance_csv_file,
                            load_profile_file=load_profile_file
                            )

register_env("PowerSystemEnv", env_creator)



# Suppose you have N agents
N = 5  # adjust this as per your actual number of agents

policies = {
    f"policy_{i}": (None, 
                    original_env.observation_space_dict[f"agent_{i}"], 
                    original_env.action_space_dict[f"agent_{i}"], 
                    {"agent_id": i})  # Set 'agent_id' key here
    for i in range(N)
}


def policy_mapping_fn(agent_id, episode, **kwargs):
    if "agent_" in agent_id:
        # Extract agent number and return the corresponding policy
        agent_num = int(agent_id.split("_")[1])
        return f"policy_{agent_num}"
    else:
        raise ValueError(f"Unknown agent_id: {agent_id}")






In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import csv
from py_dss_interface import DSSDLL
import pandas as pd

# defining values and lengths as before
PV_KVAR_ACTION_LEN = 30
PV_KW_ACTION_LEN = 30
BESS_KW_ACTION_LEN = 30

PV_KVAR_ACTION_LOW = -80
PV_KVAR_ACTION_HIGH = 80
PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 100
BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 100

class CentralizedPowerSystemEnv(gym.Env):
    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file):
        super(CentralizedPowerSystemEnv, self).__init__()

        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        self.ranked_buses  = ['50', '49', '51', '47', '48', '44', '57', '54', '64', '63', '55', '62', '56', '65', '43', '60', '42', '52', '53', '61', '66', '36', '40', '67', '72', '86', '87', '76', '89', '97']
        self.capacitor_names = ["C83", "C88a", "C90b", "C92c"]
        self.KWrated=100
        self.previous_reward = 0.0
        self.alpha = 0.1

        # Apply actions to PV systems and batteries
        for i in range(30):
            bus = self.ranked_buses[i]
            self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=100 Pmpp=80")
            self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=100 kVAR=0")

        with open(irradiance_csv_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            self.irradiance_profile = [float(row[0]) for row in reader]

        with open(load_profile_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader, None)
            self.load_profile = [float(row[0]) for row in reader]

        total_action_shape = PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN
        self.action_space = spaces.Box(low=-100, high=100, shape=(total_action_shape,), dtype=np.float32)

        total_observation_shape = 278  # Assuming observation space is the voltage at each bus
        self.observation_space = spaces.Box(low=0, high=2, shape=(total_observation_shape,), dtype=np.float32)
        
        self.current_step = 0
        self.control_steps = 0
        self.max_control_steps = 2

    def _take_action(self, action):
        TOTAL_ACTIONS_PER_AGENT = 6 + 6 + 6
        agent_actions = np.split(action, 5)
        
        for agent_idx, agent_action in enumerate(agent_actions):
            pv_kw_actions = agent_action[:6]
            pv_kvar_actions = agent_action[6:12]
            bess_kw_actions = agent_action[12:18]

            pv_idx_offset = agent_idx * 6
            battery_idx_offset = agent_idx * 6

            for idx, (kw_action_value, kvar_action_value) in enumerate(zip(pv_kw_actions, pv_kvar_actions)):
                irradiance = self.irradiance_profile[self.current_step % 8760]
                scaled_pv_kw = kw_action_value * irradiance
                if scaled_pv_kw > irradiance * self.KWrated:
                    scaled_pv_kw = irradiance * self.KWrated

                pv_kvar = kvar_action_value
                S_max = 100
                q_max1 = np.sqrt(S_max**2 - np.power(scaled_pv_kw, 2))
                pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)
                self.controller.text(f"edit PVSystem.PV{idx + pv_idx_offset + 1} phases=3 kV=4.16 Pmpp={scaled_pv_kw} kVAR={pv_kvar}")

            for idx, action_value in enumerate(bess_kw_actions):
                self.controller.text(f"edit Storage.Battery{idx + battery_idx_offset + 1} phases=3 kV=4.16 kW={action_value} kVAR=0")

        self.controller.text("set controlmode=off")
        self.controller.text("solve")

    def step(self, action):
        # Execute the action
        self._take_action(action)

        # Calculate the rewards
        losses = sum(self.controller.circuit_losses())
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        voltage_violations = sum(1 for v in all_bus_voltages if v <= 0.95 or v >= 1.05)
        voltage_deviations = sum((v - 1)**2 for v in all_bus_voltages)

        # Define penalty weights for different components
        w_deviation = 1.0
        w_violation = 5.0
        w_loss = 0.001

        reward = - w_deviation * voltage_deviations - w_violation * voltage_violations - w_loss * losses
        done = self.control_steps >= self.max_control_steps or voltage_violations == 0
            # Set termination status for all agents
        terminations = done
    # Assuming no truncation; modify as needed
        truncateds = False

    # Gather additional information if required

    # After taking action, increment control steps
        self.control_steps += 1


   
        observations = self.get_central_observation()

        # Assuming no additional info for now.
        info = {}

        return observations, reward, terminations, truncateds, info
        
    def reset(self, *, seed=None, options=None):
        # If a seed is provided, set the random seed for numpy
        if seed is not None:
            np.random.seed(seed)

        # You can use options to customize the reset function if needed
        # For now, we'll just print the options
        if options is not None:
            print(f"Reset options: {options}")

        # Reset power system to initial state
        self.controller.text(f"compile [{dss_file}]")
        self.current_step = np.random.randint(0, 8761)
        self.control_steps = 0

        
        
         # Load names
        load_names = [
            "S1a", "S2b", "S4c", "S5c", "S6c", "S7a", "S9a", "S10a", "S11a", "S12b",
            "S16c", "S17c", "S19a", "S20a", "S22b", "S24c", "S28a", "S29a", "S30c", "S31c",
            "S32c", "S33a", "S34c", "S35a", "S37a", "S38b", "S39b", "S41c", "S42a", "S43b",
            "S45a", "S46a", "S47", "S48", "S49a", "S49b", "S49c", "S50c", "S51a", "S52a",
            "S53a", "S55a", "S56b", "S58b", "S59b", "S60a", "S62c", "S63a", "S64b", "S65a",
            "S65b", "S65c", "S66c", "S68a", "S69a", "S70a", "S71a", "S73c", "S74c", "S75c",
            "S76a", "S76b", "S76c", "S77b", "S79a", "S80b", "S82a", "S83c", "S84c", "S85c",
            "S86b"
            ]
  
            



        # Read CSV file into list
        #load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).tolist()
        # Convert the first column to a list
        load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).iloc[:, 0].tolist()


        # Create a dictionary with load names as keys and load powers as values
        load_dict = dict(zip(load_names, load_powers))

# Generate load scales using a Gaussian distribution
        load_scales = np.random.normal(loc=self.load_profile[self.current_step % 8760], scale=0.5, size=len(load_names))

        for load_name, load_scale in zip(load_names, load_scales):
    # Get the load power corresponding to load_name from the dictionary
            load_power = load_dict[load_name]
    
    # Multiply load power by load_scale
            result = load_power * load_scale
    
    # Use the result in your controller
            self.controller.text(f"edit Load.{load_name} kW={result}")

            
            
        # Load the irradiance for the current hour
        
        
        irradiance = self.irradiance_profile[self.current_step % 8760]
        

    
        

        # Initialize random states for controlled devices
        for i, bus in enumerate(self.ranked_buses):
            # Randomly initialize PV systems and batteries
            #kvar should be set to very small
            fixed_power_factor = 0.9  # Set a fixed power factor value (e.g., 0.9)
            pv_kw = 100 * self.irradiance_profile[self.current_step % 8760]  # Scale PV kW by irradiance
            power_factor_angle = np.arccos(fixed_power_factor)  # Calculate the angle corresponding to the power factor
            pv_kvar = pv_kw * np.tan(power_factor_angle)  # Calculate reactive power (kVAR) based on kW and power factor
            battery_kw = self.load_profile[self.current_step % 8760]*100  # Scale Battery kW by load
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0") #select charging or discharging
            #make a comparison study beween different modes
            #investigate batteries charging/dicharging pattern



        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")

        observation = self.get_central_observation()
        return observation, {}  # Adding the additional reset info (empty dictionary)

    def get_central_observation(self):
        # Get the voltage at all buses
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()

        # Flatten the list of bus voltages
        all_bus_voltages = np.array(all_bus_voltages).flatten()

        return all_bus_voltages

    def render(self, mode='human'):
        pass

    def close(self):
        pass

    def _end_of_episode(self):
        return self.current_step >= len(self.irradiance_profile)




In [ ]:
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv

# Assuming your environment class is loaded and properly set up.
env = DummyVecEnv([lambda: CentralizedPowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)])

model = SAC("MlpPolicy", env, verbose=1, buffer_size=50000)
model.learn(total_timesteps=10000)
model.save("sac_centralized_power_system")

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import csv
from py_dss_interface import DSSDLL
import pandas as pd

# defining values and lengths as before
PV_KVAR_ACTION_LEN = 30
PV_KW_ACTION_LEN = 30
BESS_KW_ACTION_LEN = 30

PV_KVAR_ACTION_LOW = -80
PV_KVAR_ACTION_HIGH = 80
PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 100
BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 100

class CentralizedPowerSystemEnv(gym.Env):
    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file):
        super(CentralizedPowerSystemEnv, self).__init__()

        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        self.ranked_buses  = ['50', '49', '51', '47', '48', '44', '57', '54', '64', '63', '55', '62', '56', '65', '43', '60', '42', '52', '53', '61', '66', '36', '40', '67', '72', '86', '87', '76', '89', '97']
        self.capacitor_names = ["C83", "C88a", "C90b", "C92c"]
        self.KWrated=100
        self.previous_reward = 0.0
        self.alpha = 0.1
        self.current_step = 0
        self.control_steps = 0
        self.max_control_steps = 2
        self.num_agents = 5
        self.agent_observation_lens = [50, 50, 50, 50, 50]

        # Apply actions to PV systems and batteries
        for i in range(30):
            bus = self.ranked_buses[i]
            self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=1000 Pmpp=800")
            self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=1000 kVAR=0")

        with open(irradiance_csv_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            self.irradiance_profile = [float(row[0]) for row in reader]

        with open(load_profile_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader, None)
            self.load_profile = [float(row[0]) for row in reader]

        total_action_shape = PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN
        #self.action_space = spaces.Box(low=-100, high=100, shape=(total_action_shape,), dtype=np.float32)

        total_observation_shape = 283  # Assuming observation space is the voltage at each bus
       # Modify the action and observation spaces to have consistent dimensions for each agent
        # Combine all agent action spaces into one flat space
        total_action_shape = sum([18 for _ in range(self.num_agents)])
        self.action_space = spaces.Box(low=-100, high=100, shape=(total_action_shape,), dtype=np.float32)
        
        # Combine all agent observation spaces into one flat space
        total_observation_shape = sum(self.agent_observation_lens)
        self.observation_space = spaces.Box(low=0, high=2, shape=(total_observation_shape,), dtype=np.float32)

    def local_reward(self, agent_idx, voltages):
        # Define the start and end index for each agent's subset of voltages
        start_idx = sum(self.agent_observation_lens[:agent_idx])
        end_idx = start_idx + self.agent_observation_lens[agent_idx]

        # Extract the subset of voltages for the agent
        agent_voltages = voltages[start_idx:end_idx]

        # Calculate the number of voltage violations for the agent's subset of voltages
        voltage_violations = sum(1 for v in agent_voltages if v <= 0.95 or v >= 1.05)

        # Calculate the agent's reward based on voltage violations
        # We use a negative reward for violations, so agents are encouraged to reduce them.
        w_violation = -5.0
        agent_reward = w_violation * voltage_violations

        return agent_reward
    
    
    def get_agent_observation(self, agent_idx, voltages):
        start_idx = sum(self.agent_observation_lens[:agent_idx])
        end_idx = start_idx + self.agent_observation_lens[agent_idx]
        local_obs = voltages[start_idx:end_idx]
        return local_obs



    def _take_action_for_agent(self, agent_idx, action):
        # Define total actions per agent
        TOTAL_ACTIONS_PER_AGENT = 6 + 6 + 6

        # Index offset based on agent_idx for PV and battery control
        pv_idx_offset = agent_idx * 6
        battery_idx_offset = agent_idx * 6

        # Decompose the action for this agent
        pv_kw_actions = action[:6]
        pv_kvar_actions = action[6:12]
        bess_kw_actions = action[12:18]

        # Apply PV actions
        for idx, (kw_action_value, kvar_action_value) in enumerate(zip(pv_kw_actions, pv_kvar_actions)):
            # handle kW actions
            irradiance = self.irradiance_profile[self.current_step % 10]
            scaled_pv_kw = kw_action_value * irradiance
            if scaled_pv_kw > irradiance * self.KWrated:  # clip to max
                scaled_pv_kw = irradiance * self.KWrated

            # handle kVAR actions
            pv_kvar = kvar_action_value
            S_max = 100  # Maximum apparent power (example value)
            q_max1 = np.sqrt(S_max**2 - np.power(scaled_pv_kw, 2))
            pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)
            self.controller.text(f"edit PVSystem.PV{idx + pv_idx_offset + 1} phases=3 kV=4.16 kW={scaled_pv_kw} kVAR={pv_kvar}")

        # Apply battery actions
        for idx, action_value in enumerate(bess_kw_actions):
            self.controller.text(f"edit Storage.Battery{idx + battery_idx_offset + 1} phases=3 kV=4.16 kW={action_value} kVAR=0")

        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")

        


    def step(self, actions):
        # Each agent takes its action
        for agent_idx, action in enumerate(actions):
            self._take_action_for_agent(agent_idx, action)

        # Calculate the global rewards and other metrics
        losses = sum(self.controller.circuit_losses())
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()

        # Calculate local rewards
        rewards = [self.local_reward(i, all_bus_voltages) for i in range(self.num_agents)]

        # Check if episode should terminate
        done = self.control_steps >= self.max_control_steps
        terminations = [done] * self.num_agents
        truncateds = False

        # Update control steps
        self.control_steps += 1

        # Get observations for each agent
        observations = [self.get_agent_observation(i, all_bus_voltages) for i in range(self.num_agents)]
        print("Observations after actions:", observations)
        print("Observations Shape:", np.array(observations).shape)

        # Prepare empty info dicts for each agent
        infos = [{} for _ in range(self.num_agents)]

        return observations, rewards, terminations, truncateds, infos

        
    
    def reset(self, *, seed=None, options=None):
        # If a seed is provided, set the random seed for numpy
        if seed is not None:
            np.random.seed(seed)

        # You can use options to customize the reset function if needed
        # For now, we'll just print the options
        if options is not None:
            print(f"Reset options: {options}")

        # Reset power system to initial state
        self.controller.text(f"compile [{dss_file}]")
        self.current_step = np.random.randint(0, 11)
        self.control_steps = 0

        
        
         # Load names
        load_names = [
            "S1a", "S2b", "S4c", "S5c", "S6c", "S7a", "S9a", "S10a", "S11a", "S12b",
            "S16c", "S17c", "S19a", "S20a", "S22b", "S24c", "S28a", "S29a", "S30c", "S31c",
            "S32c", "S33a", "S34c", "S35a", "S37a", "S38b", "S39b", "S41c", "S42a", "S43b",
            "S45a", "S46a", "S47", "S48", "S49a", "S49b", "S49c", "S50c", "S51a", "S52a",
            "S53a", "S55a", "S56b", "S58b", "S59b", "S60a", "S62c", "S63a", "S64b", "S65a",
            "S65b", "S65c", "S66c", "S68a", "S69a", "S70a", "S71a", "S73c", "S74c", "S75c",
            "S76a", "S76b", "S76c", "S77b", "S79a", "S80b", "S82a", "S83c", "S84c", "S85c",
            "S86b"
            ]
  
            



        # Read CSV file into list
        #load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).tolist()
        # Convert the first column to a list
        load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).iloc[:, 0].tolist()


        # Create a dictionary with load names as keys and load powers as values
        load_dict = dict(zip(load_names, load_powers))

# Generate load scales using a Gaussian distribution
        load_scales = np.random.normal(loc=self.load_profile[self.current_step % 10], scale=0.5, size=len(load_names))

        for load_name, load_scale in zip(load_names, load_scales):
    # Get the load power corresponding to load_name from the dictionary
            load_power = load_dict[load_name]
    
    # Multiply load power by load_scale
            result = load_power * load_scale
    
    # Use the result in your controller
            self.controller.text(f"edit Load.{load_name} kW={result}")

            
            
        # Load the irradiance for the current hour
        
        
        irradiance = self.irradiance_profile[self.current_step % 10]
        

    
        

        # Initialize random states for controlled devices
        for i, bus in enumerate(self.ranked_buses):
            # Randomly initialize PV systems and batteries
            #kvar should be set to very small
            fixed_power_factor = 0.9  # Set a fixed power factor value (e.g., 0.9)
            pv_kw = 100 * self.irradiance_profile[self.current_step % 10]  # Scale PV kW by irradiance
            power_factor_angle = np.arccos(fixed_power_factor)  # Calculate the angle corresponding to the power factor
            pv_kvar = pv_kw * np.tan(power_factor_angle)  # Calculate reactive power (kVAR) based on kW and power factor
            battery_kw = self.load_profile[self.current_step % 10]*100  # Scale Battery kW by load
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0") #select charging or discharging
            #make a comparison study beween different modes
            #investigate batteries charging/dicharging pattern



        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")

        # Aggregate the observations
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()

        # Get individual observations for each agent
        all_observations = [self.get_agent_observation(i, all_bus_voltages) for i in range(self.num_agents)]

        # Convert the list of observations into a 2D numpy array
        obs_array = np.stack(all_observations)

        return obs_array, {} 

    def get_central_observation(self):
        # Get the voltage at all buses
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()

        # Flatten the list of bus voltages
        all_bus_voltages = np.array(all_bus_voltages).flatten()

        return all_bus_voltages

    def render(self, mode='human'):
        pass

    def close(self):
        pass

    def _end_of_episode(self):
        return self.current_step >= len(self.irradiance_profile)






In [ ]:
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv

# Convert your environment to vectorized format
vec_env = DummyVecEnv([lambda: CentralizedPowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)])
# Initialize SAC agents
agents = [SAC("MlpPolicy", vec_env, verbose=1) for _ in range(5)]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3.common.monitor import load_results
from stable_baselines3.common.results_plotter import ts2xy

from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

def make_env(dss_path, dss_file, irradiance_csv_file, load_profile_file):
    def _init():
        return CentralizedPowerSystemEnv(
            dss_path=dss_path,
            dss_file=dss_file,
            irradiance_csv_file=irradiance_csv_file,
            load_profile_file=load_profile_file
        )
    return _init

# Paths for your setup
dss_path = r"C:\Program Files\OpenDSS"
dss_file = r"D:\Alaa_Selim\123Bus\IEEE123Master.dss"
irradiance_csv_file = r"D:\Alaa_Selim\Irradiance_Profile_Santa_Clara.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"

# Create a single environment (since we are training sequentially)
env = make_env(dss_path, dss_file, irradiance_csv_file, load_profile_file)()
vec_env = DummyVecEnv([lambda: env])
vec_env = VecNormalize(vec_env)

# Initialize SAC agents for the environment
num_agents = 5
agents = [SAC("MlpPolicy", vec_env, verbose=1) for _ in range(num_agents)]

# Train each agent sequentially
total_timesteps = 10000  # Example value, adjust as needed
for agent in agents:
    agent.learn(total_timesteps=total_timesteps)

# Save the agents
model_save_path = "models/"
for idx, agent in enumerate(agents):
    agent.save(f"{model_save_path}agent_{idx}")

# Plot learning curves
def moving_average(values, window):
    """
    Smooth values by doing a moving average
    """
    weights = np.repeat(1.0, window) / window
    return np.convolve(values, weights, 'valid')

def plot_results():
    fig, ax = plt.subplots(figsize=(9, 4))
    
    for idx in range(num_agents):
        results = load_results(f"{model_save_path}agent_{idx}")
        x, y = ts2xy(results, 'timesteps')
        y = moving_average(y, window=50)  # Window of 50 for smoothing
        ax.plot(x[:-49], y, label=f"Agent_{idx}")
    
    ax.set_xlabel("Timesteps")
    ax.set_ylabel("Rewards")
    plt.legend()
    plt.tight_layout()
    # Save the figure
    plt.savefig('SAC_Learning.pdf', format='pdf')
    plt.show()

plot_results()


In [ ]:
from stable_baselines3.common.callbacks import BaseCallback
import matplotlib.pyplot as plt

class RewardLoggerCallback(BaseCallback):
    def __init__(self, check_freq: int):
        super(RewardLoggerCallback, self).__init__()
        self.check_freq = check_freq
        self.rewards = []
        self.mean_rewards = []

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            # Get the episode rewards from the environment
            episode_rewards = self.locals.get('episode_rewards', [])
            self.rewards.extend(episode_rewards)
            
            if len(self.rewards) > 0:
                mean_reward = np.mean(self.rewards[-100:])  # Mean of last 100 rewards
                self.mean_rewards.append(mean_reward)
                print(f"Mean reward over last 100 episodes: {mean_reward}")

        return True

    def plot_rewards(self):
        plt.plot(self.mean_rewards)
        plt.ylabel('Mean reward (last 100 episodes)')
        plt.xlabel('Number of timesteps')
        plt.title('Training progress')
        plt.show()


In [ ]:
from stable_baselines3.common.callbacks import EvalCallback
import pandas as pd

log_dir = './logs/'
eval_env = DummyVecEnv([lambda: CentralizedPowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)])

eval_callback = EvalCallback(eval_env, best_model_save_path=log_dir, log_path=log_dir, eval_freq=1000)

model = SAC("MlpPolicy", env, verbose=1, buffer_size=50000)
model.learn(total_timesteps=10000, callback=eval_callback)

# Save the model
model.save("sac_centralized_power_system")
import os

if not os.path.exists('./logs/'):
    os.makedirs('./logs/')

# Load the results using pandas
results = pd.read_csv(log_dir + 'evaluations.csv')
mean_rewards = results['mean_reward']

# Plot the mean rewards
plt.plot(mean_rewards)
plt.ylabel('Mean reward')
plt.xlabel('Number of timesteps')
plt.title('Training progress')
plt.show()



In [ ]:
import numpy as np
import gymnasium as gym
from stable_baselines3 import SAC, TD3
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.results_plotter import load_results, ts2xy
import matplotlib.pyplot as plt
from stable_baselines3.common.monitor import Monitor
import torch
import random
import time

SEED = 0
np.random.seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)

start_time = time.time()

# First, wrap the custom environment with Monitor
monitored_env = Monitor(CentralizedPowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
), './models/')

# Then, wrap the monitored environment with DummyVecEnv
env = DummyVecEnv([lambda: monitored_env])

# Save a checkpoint every 1000 steps
checkpoint_callback = CheckpointCallback(save_freq=1000, save_path='./models/')

# Training configurations
num_timesteps = 100000
agents = {
    "SAC": SAC("MlpPolicy", env, verbose=1),
}

# Train each agent and store results
results = {}
for name, agent in agents.items():
    print(f"\nTraining {name}...\n")
    agent.learn(total_timesteps=num_timesteps, callback=checkpoint_callback)
    results[name] = load_results('./models/')

end_time = time.time()
print(f"Total optimization time: {end_time - start_time} seconds")

# Plot learning curves
def moving_average(values, window):
    """
    Smooth values by doing a moving average
    """
    weights = np.repeat(1.0, window) / window
    return np.convolve(values, weights, 'valid')

def plot_results():
    fig, ax = plt.subplots(figsize=(9, 4))
    
    for name, result in results.items():
        x, y = ts2xy(result, 'timesteps')
        y = moving_average(y, window=50)  # Window of 50 for smoothing
        ax.plot(x[:-49], y, label=name)
    
    ax.set_xlabel("Timesteps")
    ax.set_ylabel("Rewards")
    plt.tight_layout()
    # Save the figure
    plt.savefig('SAC_Learning.pdf', format='pdf')
    plt.show()

plot_results()


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback

# Assuming your environment class CentralizedPowerSystemEnv is defined and properly set up as given in the snippets

class RewardLoggerCallback(BaseCallback):
    def __init__(self, check_freq: int):
        super(RewardLoggerCallback, self).__init__()
        self.check_freq = check_freq
        self.rewards = []
        self.mean_rewards = []

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            print("RewardLoggerCallback triggered!")  # Debug print
            episode_rewards = self.locals.get('episode_rewards', [])
            self.rewards.extend(episode_rewards)
            
            if len(self.rewards) > 0:
                mean_reward = np.mean(self.rewards[-100:])  
                self.mean_rewards.append(mean_reward)
                print(f"Mean reward over last 100 episodes: {mean_reward}")

        return True

# Set up the environment
env = DummyVecEnv([lambda: CentralizedPowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)])

reward_logger = RewardLoggerCallback(check_freq=1000)

# Ensure the logging directory exists
log_dir = './logs/'
if not os.path.exists(log_dir):
    print("Creating logging directory...")  # Debug print
    os.makedirs(log_dir)

# Set up the EvalCallback
eval_callback = EvalCallback(env, log_path=log_dir, eval_freq=5000)

# Train the model
model = SAC("MlpPolicy", env, verbose=1, buffer_size=50000)
print("Starting model training...")  # Debug print
model.learn(total_timesteps=10000, callback=[reward_logger, eval_callback])
print("Training completed!")  # Debug print

# Save the model
model.save("sac_centralized_power_system")

# Plot rewards using RewardLoggerCallback data
print("Plotting rewards using RewardLoggerCallback data...")  # Debug print
plt.plot(reward_logger.mean_rewards)
plt.ylabel('Mean reward (last 100 episodes)')
plt.xlabel('Number of timesteps')
plt.title('Training progress using RewardLogger')
plt.show()

# Optional: Plot rewards using evaluations.csv (from EvalCallback)
print("Attempting to plot rewards using evaluations.csv...")  # Debug print
try:
    results = pd.read_csv(log_dir + 'evaluations.csv')
    mean_rewards = results['mean_reward']
    plt.plot(mean_rewards)
    plt.ylabel('Mean reward')
    plt.xlabel('Number of timesteps')
    plt.title('Training progress using EvalCallback')
    plt.show()
except FileNotFoundError:
    print("evaluations.csv not found. Perhaps the EvalCallback didn't execute or finish logging.")


In [ ]:
import os

if os.path.exists('./logs/evaluations.csv'):
    print("The file exists!")
else:
    print("The file does not exist!")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def decentralized_execution(env, model):
    obs = env.reset()
    done = False
    all_agent_actions = [[] for _ in range(env.num_agents)]  # Store actions for plotting later
    
    while not done:
        central_action = model.predict(obs, deterministic=True)[0]
        
        # Split central action into individual agent actions
        agent_actions = np.split(central_action, env.num_agents)
        
        # Store each agent's actions for plotting
        for idx, agent_action in enumerate(agent_actions):
            all_agent_actions[idx].append(agent_action)
            
        obs, reward, done, _ = env.step(central_action)
        env.render()  # If you have a render method

    # Plot the actions for each agent
    for idx, agent_actions in enumerate(all_agent_actions):
        agent_actions = np.array(agent_actions)  # Convert to numpy array for easier plotting
        plt.figure()
        
        # Assuming the sum of actions is a meaningful metric for your case
        # Adjust this as needed
        summed_actions = agent_actions.sum(axis=1)
        
        plt.plot(summed_actions)
        plt.title(f"Agent {idx+1} Actions over Time")
        plt.xlabel("Timestep")
        plt.ylabel(f"Summed Actions of Agent {idx+1}")
        plt.show()

# Load trained model as before
loaded_model = SAC.load("sac_centralized_power_system")

# Perform decentralized execution with the above plots
decentralized_execution(env, loaded_model)


In [ ]:
pip install learn2learn

In [ ]:
pip install Habitat-Lab

In [ ]:
pip install pettingzoo

In [ ]:
import numpy as np
import csv
from py_dss_interface import DSSDLL
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from pettingzoo import AECEnv
from pettingzoo.utils import agent_selector
from pettingzoo import ParallelEnv


class CentralizedPowerSystemEnv(ParallelEnv):
    metadata = {'render.modes': ['human'], 'is_parallelizable': True,}

    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file):
        super(CentralizedPowerSystemEnv, self).__init__()

        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        self.ranked_buses  = [
                                '1', '7', '8', '13', '21', '23', '29', '250', '35', '40', '42', '55',
                                '56', '65', '76', '78', '66', '79', '81', '83', '91', '95', '100',
                                '197', '300', '110', '135', '160', '152', '610'
                            ]
        self.capacitor_names = ["C83", "C88a", "C90b", "C92c"]
        self.KWrated=100
        self.previous_reward = 0.0
        self.alpha = 0.1
        self.current_step = 0
        self.control_steps = 0
        self.max_control_steps = 10
        self.n_agents = 5
        self.agent_observation_lens = [56, 56, 56, 55,55]

        # Distribute the extra 28 observations
        baseline_extra = 28 // 5  # = 5
        remainder = 28 % 5  # = 3

        # Add baseline extra observations to all agents
        self.agent_observation_lens = [x + baseline_extra for x in self.agent_observation_lens]

        # Distribute the remainder to the first few agents
        for i in range(remainder):
            self.agent_observation_lens[i] += 1


        

        # Apply actions to PV systems and batteries
        for i in range(30):
            bus = self.ranked_buses[i]
            self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=1000 Pmpp=800")
            self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=1000 kVAR=0")

        with open(irradiance_csv_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            self.irradiance_profile = [float(row[0]) for row in reader]

        with open(load_profile_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader, None)
            self.load_profile = [float(row[0]) for row in reader]
        
        self.agents = [f"agent_{i}" for i in range(self.n_agents)]
        self.possible_agents = self.agents.copy()
        self.agent_name_to_agent_num = {name: num for num, name in enumerate(self.agents)}
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self._cumulative_rewards = {agent: 0.0 for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        self.rewards = {agent: 0 for agent in self.agents}






        # Setting up for PettingZoo's agent iteration
        #self._agent_selector = agent_selector(self.agents)

        # Agent spaces definition
        # Observations and Action spaces
        low_bound = np.zeros(18)
        high_bound = np.ones(18) * 1000

# Modify the bounds for indices from 12 to 18
        low_bound[6:12] = -1000
        high_bound[6:12] = 1000

        self.action_spaces = {agent: spaces.Box(low=low_bound, high=high_bound, dtype=np.float32) for agent in self.agents}
        self.observation_spaces = {agent: spaces.Box(low=-np.inf, high=np.inf, shape=(self.agent_observation_lens[idx],), dtype=np.float32) for idx, agent in enumerate(self.agents)}
    def local_reward(self, agent_idx, voltages):
        # Define the start and end index for each agent's subset of voltages
        start_idx = sum(self.agent_observation_lens[:agent_idx])
        end_idx = start_idx + self.agent_observation_lens[agent_idx]

        # Extract the subset of voltages for the agent
        agent_voltages = voltages[start_idx:end_idx]

        # Calculate the number of voltage violations for the agent's subset of voltages
        voltage_violations = sum(1 for v in agent_voltages if v <= 0.95 or v >= 1.05)

        # Calculate the agent's reward based on voltage violations
        # We use a negative reward for violations, so agents are encouraged to reduce them.
        w_violation = -5.0
        agent_reward = w_violation * voltage_violations

        return agent_reward
    
    
    def get_agent_observation(self, agent_idx, voltages):
        start_idx = sum(self.agent_observation_lens[:agent_idx])
        end_idx = start_idx + self.agent_observation_lens[agent_idx]
        local_obs = voltages[start_idx:end_idx]
        return local_obs

    def reset(self, *, seed=None, options=None):
        # If a seed is provided, set the random seed for numpy
        if seed is not None:
            np.random.seed(seed)

        # You can use options to customize the reset function if needed
        # For now, we'll just print the options
        if options is not None:
            print(f"Reset options: {options}")

        # Reset power system to initial state
        self.controller.text(f"compile [{dss_file}]")
        self.current_step = np.random.randint(0, 11)
        self.control_steps = 0
        self.terminations = {agent: False for agent in self.agents}


        
        
         # Load names
        load_names = [
            "S1a", "S2b", "S4c", "S5c", "S6c", "S7a", "S9a", "S10a", "S11a", "S12b",
            "S16c", "S17c", "S19a", "S20a", "S22b", "S24c", "S28a", "S29a", "S30c", "S31c",
            "S32c", "S33a", "S34c", "S35a", "S37a", "S38b", "S39b", "S41c", "S42a", "S43b",
            "S45a", "S46a", "S47", "S48", "S49a", "S49b", "S49c", "S50c", "S51a", "S52a",
            "S53a", "S55a", "S56b", "S58b", "S59b", "S60a", "S62c", "S63a", "S64b", "S65a",
            "S65b", "S65c", "S66c", "S68a", "S69a", "S70a", "S71a", "S73c", "S74c", "S75c",
            "S76a", "S76b", "S76c", "S77b", "S79a", "S80b", "S82a", "S83c", "S84c", "S85c",
            "S86b"
            ]
  
            



        # Read CSV file into list
        #load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).tolist()
        # Convert the first column to a list
        load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).iloc[:, 0].tolist()


        # Create a dictionary with load names as keys and load powers as values
        load_dict = dict(zip(load_names, load_powers))

# Generate load scales using a Gaussian distribution
        load_scales = np.random.normal(loc=self.load_profile[self.current_step %10], scale=0.5, size=len(load_names))

        for load_name, load_scale in zip(load_names, load_scales):
    # Get the load power corresponding to load_name from the dictionary
            load_power = load_dict[load_name]
    
    # Multiply load power by load_scale
            result = load_power * load_scale
    
    # Use the result in your controller
            self.controller.text(f"edit Load.{load_name} kW={result}")

            
            
        # Load the irradiance for the current hour
        
        
        irradiance = self.irradiance_profile[self.current_step % 10]
        

    
        

        # Initialize random states for controlled devices
        for i, bus in enumerate(self.ranked_buses):
            # Randomly initialize PV systems and batteries
            #kvar should be set to very small
            fixed_power_factor = 0.9  # Set a fixed power factor value (e.g., 0.9)
            pv_kw = 100 * self.irradiance_profile[self.current_step % 10]  # Scale PV kW by irradiance
            power_factor_angle = np.arccos(fixed_power_factor)  # Calculate the angle corresponding to the power factor
            pv_kvar = pv_kw * np.tan(power_factor_angle)  # Calculate reactive power (kVAR) based on kW and power factor
            battery_kw = self.load_profile[self.current_step % 10]*100  # Scale Battery kW by load
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0") #select charging or discharging
            #make a comparison study beween different modes
            #investigate batteries charging/dicharging pattern



        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")

        # Aggregate the observations
        all_bus_voltages = np.clip(self.controller.circuit_all_bus_vmag_pu(), 1e-10, 10.0)

        # Get individual observations for each agent
        all_observations = [self.get_agent_observation(i, all_bus_voltages) for i in range(self.n_agents)]

        # Convert the list of observations into a 2D numpy array
        #obs_array = np.stack(all_observations)

        self.agents = self.possible_agents.copy()
        #self._agent_selector.reinit(self.agents)
        #self.agent_selection = self._agent_selector.next()
        self._cumulative_rewards = {agent: 0.0 for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        observations = self.get_obs()
        
       # if return_info:
           # return self.get_obs(), self.get_infos()
       # else:
           # return self.get_obs()
#





        return self.get_obs(), self.get_infos()
    
    
    def _take_action_for_agent(self, agent_idx, action):
        # Define total actions per agent
        TOTAL_ACTIONS_PER_AGENT = 6 + 6 + 6

        # Index offset based on agent_idx for PV and battery control
        pv_idx_offset = agent_idx * 6
        battery_idx_offset = agent_idx * 6

        # Decompose the action for this agent
        pv_kw_actions = action[:6]
        pv_kvar_actions = action[6:12]
        bess_kw_actions = action[12:18]

        # Apply PV actions
        for idx, (kw_action_value, kvar_action_value) in enumerate(zip(pv_kw_actions, pv_kvar_actions)):
            # handle kW actions
            irradiance = self.irradiance_profile[self.current_step % 10]
            scaled_pv_kw = kw_action_value * irradiance
            if scaled_pv_kw > irradiance * self.KWrated:  # clip to max
                scaled_pv_kw = irradiance * self.KWrated

            # handle kVAR actions
            pv_kvar = kvar_action_value
            S_max = 100  # Maximum apparent power (example value)
            q_max1 = np.sqrt(S_max**2 - np.power(scaled_pv_kw, 2))
                        # Set the reactive power based on the maximum possible Q and the desired Q
            if scaled_pv_kw == 0:  # If active power is zero
                pv_kvar = 0
            else:
                pv_kvar = np.clip(kvar_action_value, -q_max1, q_max1)  # Ensure desired Q is within feasible limits
            self.controller.text(f"edit PVSystem.PV{idx + pv_idx_offset + 1} phases=3 kV=4.16 kW={scaled_pv_kw} kVAR={pv_kvar}")
            action_str = f"Agent agent_{agent_idx} sets PVSystem.PV{idx + pv_idx_offset + 1} to kW={scaled_pv_kw} kVAR={pv_kvar}"
            print(action_str)
        # Apply battery actions
        for idx, action_value in enumerate(bess_kw_actions):
            self.controller.text(f"edit Storage.Battery{idx + battery_idx_offset + 1} phases=3 kV=4.16 kW={action_value} kVAR=0")
            action_str = f"Agent agent_{agent_idx} sets Storage.Battery{idx + battery_idx_offset + 1} to kW={action_value}"
            print(action_str)
        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        print(f"Agent {agent_idx} Action: {action}")


        




    def step(self, actions):
        # Apply actions for all agents
        for agent_name, action in actions.items():
            agent_idx = self.agent_name_to_agent_num[agent_name]
            self._take_action_for_agent(agent_idx, action)

        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")

        all_bus_voltages = np.clip(self.controller.circuit_all_bus_vmag_pu(), 1e-10, 10.0)
        rewards = {agent: self.local_reward(self.agent_name_to_agent_num[agent], all_bus_voltages) for agent in self.agents}
        if rewards ==0:
            rewards+=1000
        self.control_steps += 1
        done = self.control_steps >= 10
        #terminations = {agent: done for agent in self.agents}
        #truncateds = {agent: False for agent in self.agents}
        terminations = done 
        truncateds = False 
        infos = {agent: {} for agent in self.agents}

        # Observations for all agents
        obs = {agent: self.observe(agent) for agent in self.agents}
        for agent in self.agents:
            observation = obs[agent]
            print(f"Agent {agent} Observation:", observation)
            print(f"Length of {agent}'s Observation:", len(observation))



        return obs, rewards, terminations, truncateds, infos

    def get_obs(self):
        all_bus_voltages = np.clip(self.controller.circuit_all_bus_vmag_pu(), 1e-10, 10.0)
        obs = {agent: np.array(self.get_agent_observation(self.agent_name_to_agent_num[agent], all_bus_voltages)) for agent in self.agents}
        #print(f"Agent {agent} Observation: {obs[agent]}")
        return obs


    def get_rewards(self):
        all_bus_voltages = np.clip(self.controller.circuit_all_bus_vmag_pu(), 1e-10, 10.0)
        rewards = {agent: self.local_reward(self.agent_name_to_agent_num[agent], all_bus_voltages) for agent in self.agents}
        return rewards

    def get_dones(self):
        done = self.control_steps >= 10
        return {agent: done for agent in self.agents}

    def get_infos(self):
        return {agent: {} for agent in self.agents}

    def observation_space(self, agent):
        return self.observation_spaces[agent]

    def action_space(self, agent):
        return self.action_spaces[agent]

    def render(self, mode='human'):
        # Stub for now. Later, you can extend it to visualize the power system, if necessary.
        pass

    def close(self):
        # You might want to close the controller or perform cleanup here in the future.
        pass

    def seed(self, seed=None):
        # For reproducibility. You might want to seed other random processes, if any.
        if seed is not None:
            np.random.seed(seed)
    def observe(self, agent):
        all_bus_voltages = np.clip(self.controller.circuit_all_bus_vmag_pu(), 1e-10, 10.0)
        return np.array(self.get_agent_observation(self.agent_name_to_agent_num[agent], all_bus_voltages))








In [ ]:
# Instantiate the environment
env = CentralizedPowerSystemEnv(dss_path, dss_file, irradiance_csv_file, load_profile_file)
# Reset the environment
observations = env.reset()

# Take random actions for each agent for 5 steps
for _ in range(5):
    actions = {agent: env.action_space(agent).sample() for agent in env.agents}
    obs, rewards, terminations, truncateds, infos = env.step(actions)

env.close()


In [ ]:
all_bus_voltages = dss.circuit_all_bus_vmag_pu()
print(all_bus_voltages)


In [ ]:
print(f'dss.circuit_all_node_names(): {dss.circuit_all_node_names()}')
z= dss.circuit_all_node_names()
print(len(dss.circuit_all_node_names()))

In [ ]:
#from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility

#env = MultiAgentEnvCompatibility(CentralizedPowerSystemEnv(dss_path, dss_file, irradiance_csv_file, load_profile_file))


from ray.rllib.env import PettingZooEnv

env = CentralizedPowerSystemEnv(dss_path, dss_file, irradiance_csv_file, load_profile_file)
converted_env = PettingZooEnv(env)





def env_creator(env_config):
    return converted_env(
                            dss_path=r"C:\Program Files\OpenDSS",
                            dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
                            irradiance_csv_file=irradiance_csv_file,
                            load_profile_file=load_profile_file
                            )

register_env("CentralizedPowerSystemEnv", env_creator)
# Suppose you have N agents
N = 5  # adjust this as per your actual number of agents

policies = {
    f"policy_{i}": (None, 
                    original_env.observation_space_dict[f"agent_{i}"], 
                    original_env.action_space_dict[f"agent_{i}"], 
                    {"agent_id": i})  # Set 'agent_id' key here
    for i in range(N)
}


def policy_mapping_fn(agent_id, episode, **kwargs):
    if "agent_" in agent_id:
        # Extract agent number and return the corresponding policy
        agent_num = int(agent_id.split("_")[1])
        return f"policy_{agent_num}"
    else:
        raise ValueError(f"Unknown agent_id: {agent_id}")





In [ ]:
from ray.tune.registry import register_env
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv
from pettingzoo.utils.wrappers import pad_observations_v0, pad_action_space_v0

def env_creator(_):
    env = CentralizedPowerSystemEnv(dss_path, dss_file, irradiance_csv_file, load_profile_file)
    env = pad_observations_v0(env)
    env = pad_action_space_v0(env)
    return PettingZooEnv(env)

register_env("CentralizedPowerSystemEnv", env_creator)


In [ ]:
tune.run(
    "SAC",
    stop={"episodes_total": 300000},
    config={
        "env": "CentralizedPowerSystemEnv",
        "environment": {
            "disable_env_checking": True
        },
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn
        },
        "num_gpus": 0,
        "num_workers": 19,
        "lr": 3e-3,
    },
    local_dir="SAC-Test1",
    # callbacks=[eval_callback]
)


In [ ]:
pip install supergym

In [ ]:
from supersuit import pad_observations_v0, pad_action_space_v0
from supersuit import pettingzoo_env_to_vec_env_v1

env = CentralizedPowerSystemEnv(dss_path, dss_file, irradiance_csv_file, load_profile_file)

# Pad the observations to ensure they're all the same size
# Ensure all agents have consistent observation and action spaces
env = pad_observations_v0(env)
env = pad_action_space_v0(env)

# Convert to a vectorized gym environment
gym_env = pettingzoo_env_to_vec_env_v1(padded_env)


In [ ]:
# SB3 expects vectorized environments, so wrap it with DummyVecEnv
vec_env = DummyVecEnv([lambda: gym_env])

# Initialize and train the PPO agent
model = SAC("MlpPolicy", vec_env, verbose=1)
model.learn(total_timesteps=100000)  # Adjust timesteps as needed

# Save the model
model.save("power_system_ppo")

In [ ]:
# Necessary imports
from pettingzoo.utils.conversions import from_parallel
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from pettingzoo.utils import aec_to_parallel

# Your custom environment
env = CentralizedPowerSystemEnv(dss_path, dss_file, irradiance_csv_file, load_profile_file)

# Convert your environment to parallel
parallel_env = aec_to_parallel(env)

# Make a function that produces environments
def make_env():
    env = CentralizedPowerSystemEnv(dss_path, dss_file, irradiance_csv_file, load_profile_file)
    env = aec_to_parallel(env)
    env = from_parallel(env)  # This converts it to the format usable by Stable Baselines3
    return env

# Vectorize the environment
vec_env = DummyVecEnv([make_env])

# If your observations are images (e.g., Atari games), Stable Baselines3 expects the channels to come first.
# If not, you can ignore this line.
# vec_env = VecTransposeImage(vec_env)

# Initialize and train a PPO agent
model = PPO("MlpPolicy", vec_env, verbose=1)
model.learn(total_timesteps=10000)

# Evaluate the trained model
obs = vec_env.reset()
for _ in range(1000):
    action, _ = model.predict(obs)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render()

vec_env.close()


In [ ]:
from pettingzoo.utils.conversions import from_parallel


In [ ]:
# Reset the environment
obs = env.reset()

# Check the observation
print("Initial observation:", obs)

# Manually step through the environment for a number of steps
for step in range(5):
    for agent in env.agents:
        env.agent_selection = agent
        random_action = env.action_space(agent).sample()  # Taking random action
        obs, reward, done, info = env.step(random_action)
        print(f"Step {step + 1} - Agent {agent}: Reward = {reward}")


In [ ]:
result = env.step(random_action)
print(result)
print(type(result))


In [ ]:
pip install pettingzoo[test]


In [ ]:

def env_creator(env_config):
    return  CentralizedPowerSystemEnv(dss_path, dss_file, irradiance_csv_file, load_profile_file)


register_env("CentralizedPowerSystemEnv", env_creator)


In [ ]:
from ray.tune.registry import register_env
from ray.rllib.env import PettingZooEnv

def env_creator(config):
    # You might want to extract some parameters from the config here if you plan to parametrize your env
    return CentralizedPowerSystemEnv(dss_path, dss_file, irradiance_csv_file, load_profile_file)

register_env('CentralizedPowerSystem', lambda config: PettingZooEnv(env_creator(config)))


In [ ]:
from ray.tune.registry import register_env
from ray.rllib.env import PettingZooEnv

def env_creator(config):
    raw_env = CentralizedPowerSystemEnv(dss_path, dss_file, irradiance_csv_file, load_profile_file)
    # Make sure to wrap your environment with the PettingZoo wrapper from RLlib
    env = PettingZooEnv(raw_env)
    return env

register_env('CentralizedPowerSystem', env_creator)


In [ ]:
from ray.tune.registry import register_env
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv

def env_creator(_):
    env = CentralizedPowerSystemEnv(dss_path, dss_file, irradiance_csv_file, load_profile_file)
    env = pad_observations_v0(env)
    env = pad_action_space_v0(env)
    return PettingZooEnv(env)

register_env("CentralizedPowerSystemEnv", env_creator)


In [ ]:
tune.run(
    "SAC",
    stop={"episodes_total": 300000},
    config={
        "env": "CentralizedPowerSystemEnv",
        "environment": {
            "disable_env_checking": True
        },
        
        "num_gpus": 0,
        "num_workers": 19,
        "lr": 3e-3,
    },
    local_dir="SAC-Test",
    # callbacks=[eval_callback]
)


In [ ]:
pip install supersuit


In [ ]:
from supersuit import pettingzoo_env_to_vec_env_v1
from pettingzoo.utils import aec_to_parallel

def env_creator(_):
    env = CentralizedPowerSystemEnv(dss_path, dss_file, irradiance_csv_file, load_profile_file)
    env = aec_to_parallel(env)  # Convert to parallel API
    env = pettingzoo_env_to_vec_env_v1(env)  # Convert to vec env
    return env

register_env("CentralizedPowerSystem", env_creator)


In [ ]:
import pettingzoo.utils
print(dir(pettingzoo.utils))


In [ ]:
from supersuit import pettingzoo_env_to_vec_env_v1

def env_creator(_):
    env = CentralizedPowerSystemEnv(dss_path, dss_file, irradiance_csv_file, load_profile_file)
    env = env.parallel_env()  # Convert AEC environment to parallel API
    env = pettingzoo_env_to_vec_env_v1(env)  # Convert to vec env
    return env

register_env("CentralizedPowerSystem", env_creator)


In [ ]:
    def step(self, action):
        #agent_idx = self.agent_name_to_agent_num[self.agent_selection]
        #self._take_action_for_agent(agent_idx, action)
        for agent_name, action in actions.items():
            agent_idx = self.agent_name_to_agent_num[agent_name]
            self._take_action_for_agent(agent_idx, action)

        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")

        all_bus_voltages = np.clip(self.controller.circuit_all_bus_vmag_pu(), 1e-10, 10.0)

        reward = self.local_reward(agent_idx, all_bus_voltages)
        self._cumulative_rewards[self.agent_selection] += reward
        self.rewards[self.agent_selection] = reward

        for agent in self.agents:
            observation = self.observe(agent)
            print(f"Observation for {agent}:", observation)

        self.control_steps += 1
        done = self.control_steps >= self.max_control_steps
        self.terminations = {agent: done for agent in self.agents}  # Update termination status

        # Set truncateds to False for simplicity
        truncateds = {agent: False for agent in self.agents}
        infos = {}  # You can add additional information here if needed

        # Get the observation for the agent
        observation = self.get_agent_observation(agent_idx, all_bus_voltages)

        # Since it's AECEnv, we'll handle agent iteration within the environment
        #self.agent_selection = self._agent_selector.next()

        return observation, reward, self.terminations, truncateds, infos

In [ ]:
import os
import gymnasium as gym
from gymnasium import spaces
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import numpy as np

from dss import DSS
import csv
import pandas as pd


# defining lengths for each segment
PV_KVAR_ACTION_LEN = 30
PV_KW_ACTION_LEN = 30
BESS_KW_ACTION_LEN = 30
TRANSFORMER_TAPS_ACTION_LEN = 1
CAPACITOR_ACTION_LEN = 4

# defining values for each segment
PV_KVAR_ACTION_LOW = -80
PV_KVAR_ACTION_HIGH = 80

PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 100

BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 100

TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1

CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1

class PowerSystemEnv(MultiAgentEnv):
    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file):
        super(PowerSystemEnv, self).__init__()
        self.home_dir = os.getcwd()
        self.dss_file = os.path.abspath(dss_file)
        self.controller = DSS #DSSDLL(dss_path)
        self.controller.Text.Command = f"compile [{self.dss_file}]"
        self.dssCircuit = DSS.ActiveCircuit
        self.AllNodeNum = len(self.dssCircuit.AllNodeNames)
        self.ranked_buses = ['57', '54', '64', '63', '55', '62',
                             '56', '65', '60', '52', '53', '61',
                             '66', '67', '72', '86', '87', '76',
                             '89', '97', '36', '40', '42', '43',
                             '44', '47', '48','49','50','51']
        self.capacitor_names = ["C83", "C88a", "C90b", "C92c"]
        self.KWrated = 100
        self.previous_reward = 0.0
        self.alpha = 0.1  #
        # Apply actions to PV systems and batteries

        if self.home_dir != os.getcwd():
            os.chdir(self.home_dir)
        with open(irradiance_csv_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            self.irradiance_profile = [float(row[0]) for row in reader]

        with open(load_profile_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader, None)  # Skip the header
            self.load_profile = [float(row[0]) for row in reader]

        self.load_names = [
            "S1a", "S2b", "S4c", "S5c", "S6c", "S7a", "S9a", "S10a", "S11a", "S12b",
            "S16c", "S17c", "S19a", "S20a", "S22b", "S24c", "S28a", "S29a", "S30c", "S31c",
            "S32c", "S33a", "S34c", "S35a", "S37a", "S38b", "S39b", "S41c", "S42a", "S43b",
            "S45a", "S46a", "S47", "S48", "S49a", "S49b", "S49c", "S50c", "S51a", "S52a",
            "S53a", "S55a", "S56b", "S58b", "S59b", "S60a", "S62c", "S63a", "S64b", "S65a",
            "S65b", "S65c", "S66c", "S68a", "S69a", "S70a", "S71a", "S73c", "S74c", "S75c",
            "S76a", "S76b", "S76c", "S77b", "S79a", "S80b", "S82a", "S83c", "S84c", "S85c",
            "S86b"
        ]

        # Read CSV file into list
        self.load_powers = pd.read_csv('.\Loadpowers.csv', header=None).iloc[:, 0].tolist()

        # Create a dictionary with load names as keys and load powers as values
        self.load_dict = dict(zip(self.load_names, self.load_powers))

        # Update the action and observation spaces for each agent

        self.actionspace_dict = {}
        self.actionlist = [13,7,10]     # the no of PV/BESs in each agent
        self.agentno = len(self.actionlist)
        self.actionspace_dict = {
            f"agent_{i}": spaces.Box(
                low=np.concatenate(([PV_KVAR_ACTION_LOW] * self.actionlist[i], [BESS_KW_ACTION_LOW] * self.actionlist[i])), # PV Q control
                high=np.concatenate(([PV_KVAR_ACTION_HIGH] * self.actionlist[i], [BESS_KW_ACTION_HIGH] * self.actionlist[i])),  #BES P control
                shape=(self.actionlist[i]*2,), dtype=np.float32) for i in range(self.agentno)
        }


        self.observation_space_dict = {}
        for i in range(self.agentno):               # each agent has whole-system observation
            self.observation_space_dict[f'agent_{i}'] = spaces.Box(low=0, high=2, shape=(self.AllNodeNum,), dtype=np.float32)

        self.action_space = gym.spaces.Dict(self.actionspace_dict)
        self.observation_space = gym.spaces.Dict(self.observation_space_dict)
        self.current_step = 0

        # Initialize control step counter
        self.control_steps = 0

        # Maximum control steps allowed in one episode
        self.max_control_steps = int(5)  # for example

        # Define the agent IDs
        self.agents = [f'agent_{i}' for i in range(self.agentno)]
        self._agent_ids = [f'agent_{i}' for i in range(self.agentno)]

    def _take_action(self, action_dict):
        # Define total actions per agent

        # Iterate through the action_dict
        for agent_id, action in action_dict.items():
            agent_idx = int(agent_id.split('_')[1])

            # Determine action segments for this agent
            #pv_kw_actions = action[start_idx:start_idx + 6]
            #pv_kvar_actions = action[start_idx + 6:start_idx + 12]
            #bess_kw_actions = action[start_idx + 12:start_idx + 18]
            # only Q control for PV
            pv_kvar_actions = action[0:self.actionlist[agent_idx]]
            bess_kw_actions = action[self.actionlist[agent_idx]:self.actionlist[agent_idx] + self.actionlist[agent_idx]]
            # Index offset based on agent_idx
            idx_offset = agent_idx * self.actionlist[agent_idx]

            # Actions for PVs
            # Q control, P will be curtailed to provide Q for Voltage control
            self.dssCircuit.SetActiveClass('PVSystem')
            for idx,kvar_action_value in enumerate(pv_kvar_actions):
                # handle kW actions
                irradiance = self.irradiance_profile[(self.current_step) % 8760]
                # handle kVAR actions
                pv_kvar = kvar_action_value
                S_max = self.KWrated  # Maximum apparent power (example value)
                p_max1 = np.sqrt(S_max ** 2 - np.power(pv_kvar, 2))
                pv_kw = np.clip(irradiance * self.KWrated, 0, p_max1)
                self.dssCircuit.SetActiveElement(f'PV{idx + idx_offset + 1}')
                self.dssCircuit.ActiveElement.Properties('kvar').Val = pv_kvar
                self.dssCircuit.ActiveElement.Properties('Pmpp').Val = pv_kw
#                self.controller.Text.Command =f"edit PVSystem.PV{idx + idx_offset + 1} phases=3 kV=4.16 Pmpp={scaled_pv_kw} kVAR={pv_kvar}"
            self.dssCircuit.SetActiveClass('Storage')
            for idx, action_value in enumerate(bess_kw_actions):
                self.dssCircuit.SetActiveElement(f'Battery{idx + idx_offset + 1}')
                self.dssCircuit.ActiveElement.Properties('kVAR').Val = 0
                self.dssCircuit.ActiveElement.Properties('kW').Val = action_value
                #self.controller.Text.Command =f"edit Storage.Battery{idx + idx_offset + 1} phases=3 kV=4.16 kW={action_value} kVAR=0"

            # Solve the power flow
        self.controller.Text.Command = "set controlmode=off"
        self.controller.Text.Command = "solve"

    def step(self, action_dict):
        # Execute the action
        self._take_action(action_dict)

        # Calculate the rewards
        losses = sum(self.dssCircuit.AllElementLosses)
        all_bus_voltages = self.dssCircuit.AllBusVmagPu
        all_bus_voltages_modified = all_bus_voltages[all_bus_voltages>0.5]      ## filter out open-node voltages
        voltage_violations = sum(1 for v in all_bus_voltages_modified if v <= 0.95 or v >= 1.05)/len(all_bus_voltages_modified)

        # Calculate the sum of squared voltage deviations from 1. This penalizes larger deviations more heavily.
        #voltage_deviations = sum((v - 1) ** 2 for v in all_bus_voltages)

        # Define penalty weights for different components
        #w_deviation = 1.0
        #w_violation = 5.0
        #w_loss = 0.001

        # Combine losses, violations, and deviations in the reward
        # reward = - w_deviation * voltage_deviations - w_violation * voltage_violations
        reward = - voltage_violations
        # Set the reward for all agents
        rewards = {agent_id: reward for agent_id in self._agent_ids}

        # Gather observations for all agents
        observations = {agent_id: self.get_observation(agent_id) for agent_id in self._agent_ids}

        # After taking action, increment control steps
        self.control_steps += 1

        # Set termination status for all agents
        terminations = {agent_id: (self.control_steps >= self.max_control_steps) or (voltage_violations == 0) for
                        agent_id in self._agent_ids}
        terminations['__all__'] = any(terminations.values())

        # Assuming no truncation; modify as needed
        truncateds = {agent_id: False for agent_id in self._agent_ids}
        truncateds['__all__'] = any(truncateds.values())

        # Gather additional information if required
        infos = {agent_id: {} for agent_id in self._agent_ids}



        return observations, rewards, terminations, truncateds, infos

    def reset(self, *, seed=None, options=None):
        # If a seed is provided, set the random seed for numpy
        if seed is not None:
            np.random.seed(seed)

        # You can use options to customize the reset function if needed
        # For now, we'll just print the options
        if options is not None:
            print(f"Reset options: {options}")
        # Reset power system to initial state
        self.controller.Text.Command = f"compile [{self.dss_file}]"

        if self.home_dir != os.getcwd():
            os.chdir(self.home_dir)

        # self.current_step = int(np.clip(np.random.normal(loc=0, scale=8760), 0, 8760))
        self.current_step = np.random.randint(0, 8761)

        # assuming the profile has 8760 hours  #use gaussian distribution
        self.control_steps = 0

        # Load names


        # Generate load scales using a Gaussian distribution
        #load_scales = np.random.normal(loc=self.load_profile[self.current_step % 8760], scale=0.3, size=len(self.load_names))
        load_scales = np.random.uniform(low=0.4,high=1.0, size=len(self.load_names))
        self.dssCircuit.SetActiveClass('load')
        for load_name, load_scale in zip(self.load_names, load_scales):
            # Get the load power corresponding to load_name from the dictionary
            load_power = self.load_dict[load_name]
            # Multiply load power by load_scale
            result = load_power * load_scale
            # Use the result in your controller
            self.dssCircuit.SetActiveElement(load_name)
            self.dssCircuit.ActiveElement.Properties('kW').Val = result
            #self.controller.Text.Command = f"edit Load.{load_name} kW={result}"

#        irradiance = self.irradiance_profile[self.current_step % 8760]
        # Initialize random states for controlled devices
        for i, bus in enumerate(self.ranked_buses):
            # Randomly initialize PV systems and batteries
            # kvar should be set to very small
            fixed_power_factor = 0.9  # Set a fixed power factor value (e.g., 0.9)
            pv_kw = 100 * self.irradiance_profile[self.current_step % 8760]  # Scale PV kW by irradiance
            power_factor_angle = np.arccos(fixed_power_factor)  # Calculate the angle corresponding to the power factor
            pv_kvar = pv_kw * np.tan(power_factor_angle)  # Calculate reactive power (kVAR) based on kW and power factor
            battery_kw = self.load_profile[self.current_step % 8760] * 10  # Scale Battery kW by load
            self.dssCircuit.SetActiveClass('PVSystem')
            self.dssCircuit.SetActiveElement(f'PV{i + 1}')
            self.dssCircuit.ActiveElement.Properties('kvar').Val = pv_kvar
            self.dssCircuit.ActiveElement.Properties('Pmpp').Val = pv_kw
            self.dssCircuit.SetActiveClass('Storage')
            self.dssCircuit.SetActiveElement(f'Battery{i + 1}')
            self.dssCircuit.ActiveElement.Properties('kVAR').Val = 0
            self.dssCircuit.ActiveElement.Properties('kW').Val = battery_kw

            ##self.controller.Text.Command = f"edit PVSystem.PV{i + 1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}"
            #self.controller.Text.Command =  f"edit Storage.Battery{i + 1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0"  # select charging or discharging
            # make a comparison study beween different modes
            # investigate batteries charging/dicharging pattern

        # Solve the power flow
        self.controller.Text.Command = "set controlmode=off"
        self.controller.Text.Command = "solve"

        # Get the observations for all agents
        observations = {agent_id: self.get_observation(agent_id) for agent_id in self._agent_ids}

        # You can include any additional information here. If there's nothing, just return an empty dictionary.
        infos = {}

        return observations, infos

    def render(self, mode='human'):
        pass

    def close(self):
        pass

    def get_observation(self, agent_id):
        # Get the voltage at all buses
        all_bus_voltages = self.dssCircuit.AllBusVmagPu
        # Flatten the list of bus voltages
        all_bus_voltages = np.array(all_bus_voltages).flatten()

        # Extract agent index from the agent_id
        agent_idx = int(agent_id.split('_')[1])

        # Determine the start and end index for the observation chunk for this agent
        if agent_idx < 4:
            start_idx = agent_idx * 50
            end_idx = start_idx + 50
        else:  # This is agent_4
            start_idx = 200
            end_idx = 278  # or you could use end_idx = len(all_bus_voltages) if the size is not guaranteed to be 278

        # Extract the observation chunk for this agent
        agent_observation = all_bus_voltages#[start_idx:end_idx]

        return agent_observation

    def get_agent_observation(self):
        agent_obs = {}
        for agent_id in self.agents:
            agent_obs[agent_id] = self.get_observation()
        return agent_obs

    def _end_of_episode(self):
        return self.current_step >= len(self.irradiance_profile)



if __name__ == '__main__':
    # perform a simple test for the environment here
    dss_file = '.\IEEE123Master.dss'
    irradiance_csv_file = ".\Irradiance_Profile_Santa_Clara.csv"
    load_profile_file = ".\LoadShape1.csv"

    env = PowerSystemEnv(
        dss_path= r" ",
        dss_file= dss_file,
        irradiance_csv_file=irradiance_csv_file,
        load_profile_file=load_profile_file)
    obs,_ = env.reset()
    print(obs)
    action = env.action_space.sample()

    #print('action: ', action)
    obs, rewards, terminations, truncateds, infos = env.step(action)
    print(obs)

In [ ]:
pip install dss

In [ ]:
pip show dss


In [ ]:


#Testing with partial observability of voltage violatiosn for each agent

import gymnasium as gym
from gymnasium import spaces
from gymnasium.spaces import Tuple, Dict

from ray.rllib.env.multi_agent_env import MultiAgentEnv
import numpy as np
from ray.rllib.algorithms.sac import SAC
from ray.rllib.algorithms.ppo import PPO
from ray.tune.registry import register_env
from ray import tune
import numpy as np

from py_dss_interface import DSSDLL
#import stable_baselines3
#from stable_baselines3 import SAC
import matplotlib.pyplot as plt
import csv
import pandas as pd
#from stable_baselines3 import A2C, DQN, PPO, TD3, SAC
import torch
import warnings
warnings.filterwarnings('ignore')
import logging

logging.basicConfig(level=logging.ERROR)





device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import time

start_time = time.time()



# Initialize OpenDSS
dss = DSSDLL(r"C:\Program Files\OpenDSS")
dss_file = r"D:\Alaa_Selim\123Bus\IEEE123Master.dss"
dss.text(f"compile [{dss_file}]")

# defining lengths for each segment
PV_KVAR_ACTION_LEN = 30
PV_KW_ACTION_LEN = 30
BESS_KW_ACTION_LEN = 30
TRANSFORMER_TAPS_ACTION_LEN = 1
CAPACITOR_ACTION_LEN = 4

# defining values for each segment
PV_KVAR_ACTION_LOW = -80
PV_KVAR_ACTION_HIGH = 80

PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 100

BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 100

TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1

CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1


class PowerSystemEnv(gym.Env):
    def __init__(self, dss_path, dss_file, irradiance_csv_file,load_profile_file):
        super(PowerSystemEnv, self).__init__()
        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        self.ranked_buses = [
                                '1', '7', '8', '13', '21', '23', '29', '250', '35', '40', '42', '55',
                                '56', '65', '76', '78', '66', '79', '81', '83', '91', '95', '100',
                                '197', '300', '110', '135', '160', '152', '610'
                            ]
        self.capacitor_names = ["C83", "C88a", "C90b", "C92c"]
        self.KWrated=100
        
        # Apply actions to PV systems and batteries
        for i in range(30):
            bus = self.ranked_buses[i]
            self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=100 Pmpp=80")
            self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=100 kVAR=0")
        
        with open(irradiance_csv_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            self.irradiance_profile = [float(row[0]) for row in reader]
    
        with open(load_profile_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader, None)  # Skip the header
            self.load_profile = [float(row[0]) for row in reader]
            
        

        # Define action and observation space
        # Actions are continuous values for pv_kvar, pv_kW, battery_kw, transformer_tap and capacitor_states
        
        self.action_space = spaces.Box(
            low=np.array(
                [PV_KVAR_ACTION_LOW]*PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_LOW]*PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_LOW]*BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_LOW]*TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_LOW]*CAPACITOR_ACTION_LEN
            ),
            high=np.array(
                [PV_KVAR_ACTION_HIGH]*PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_HIGH]*PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_HIGH]*BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_HIGH]*TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_HIGH]*CAPACITOR_ACTION_LEN
            ),
            dtype=np.float32
        )
        # Assuming observation space is the voltage at each bus
        self.observation_space = spaces.Box(low=0, high=100000, shape=(278,), dtype=np.float32)  # Modified shape
        
        self.current_step = 0

        
         # Initialize control step counter
        self.control_steps = 0

        # Maximum control steps allowed in one episode
        self.max_control_steps =10  # for example
        self.hourly_violations_count = []



    def _take_action(self, action):
        
        pv_kvar = action[:30].copy()
        pv_kw = action[30:60].copy()
        battery_kw = action[60:90].copy()
        transformer_tap = action[90].copy()
        capacitor_states = action[91:].copy()
        capacitor_states = np.round(capacitor_states)

        
        
        pv_kw_limits = [0, 100]  # Limits for PV kW (example values)
        q_max1_limits = [-80, 80]  # Limits for maximum reactive power (example values)
    
        irradiance = self.irradiance_profile[self.current_step % 8760]
        for z in range (0,30):
            if  pv_kw[z] <= irradiance* self.KWrated:
            # no clipping
                pv_kw[z] = pv_kw[z]
                #pv_kvar[z]  = np.sqrt(100**2 - np.power(pv_kw[z], 2))
            else:
                pv_kw[z] = irradiance* self.KWrated
     
        #pv_kw = np.clip(action[30:60], pv_kw_limits[0], pv_kw_limits[1])
    
        S_max = 100  # Maximum apparent power (example value)

        q_max1 = np.sqrt(S_max**2 - np.power(pv_kw, 2))
        
        #pv_kvar= 0.8 ** pv_kw
        #add checking condition on action space, or you can use PU. system or use clip action (make a constrined actio space)
           
    # Make pv_kvar controlled based on q_max1
        pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)
        
         
        
       
        # Apply actions to PV systems and batteries
        for i in range(30):
            bus = self.ranked_buses[i]
            scaled_pv_kw = pv_kw[i] 
            #* irradiance
          
            
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar[i]} Pmpp={scaled_pv_kw}")
            # need to add KVA in OpenDSS model
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw[i]} kVAR=0")

        # Apply transformer control
        self.controller.text(f"edit Transformer.reg1a taps={transformer_tap}")

        # Apply capacitor control
        for i, cap_name in enumerate(self.capacitor_names):
            self.controller.text(f"edit Capacitor.{cap_name} states={int(capacitor_states[i])}")
        #add control mode = off

        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve") 
        #print(f'dss.circuit_losses(): {dss.circuit_losses()}')


   


    def step(self, action):
    # Execute the action
        self._take_action(action)

    # Calculate the rewards
        losses = sum(self.controller.circuit_losses())
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        voltage_violations = sum(1 for v in all_bus_voltages if v <= 0.95 or v >= 1.05 and v != 0)

    # Calculate the sum of squared voltage deviations from 1. This penalizes larger deviations more heavily.
        voltage_deviations = sum((v - 1)**2 for v in all_bus_voltages)

    # Define penalty weights for different components
        w_deviation = 1.0
        w_violation = 5.0
        w_loss = 0.001

    # Combine losses, violations, and deviations in the reward
        reward = -voltage_violations 
        self.hourly_violations_count.append(voltage_violations)
        print(f"violations count: {self.hourly_violations_count}")


    # Gather observations
        observation = self.get_observation()

        termination = (self.control_steps >= self.max_control_steps) or (voltage_violations == 0)

    # After taking action, increment control steps
        self.control_steps += 1

        return observation, reward, termination, False, {}


   

    def reset(self, *, seed=None, options=None):
    # If a seed is provided, set the random seed for numpy
        if seed is not None:
            seed = 1 
            np.random.seed(seed)

    # You can use options to customize the reset function if needed
    # For now, we'll just print the options
        if options is not None:
            print(f"Reset options: {options}")
        # Reset power system to initial state
        self.controller.text(f"compile [{dss_file}]")
        
        #self.current_step = int(np.clip(np.random.normal(loc=0, scale=8760), 0, 8760))
        self.current_step = np.random.randint(0, 8761)
       

 # assuming the profile has 8760 hours  #use gaussian distribution
        self.control_steps = 0


     
        
        
         # Load names
        load_names = [
            "S1a", "S2b", "S4c", "S5c", "S6c", "S7a", "S9a", "S10a", "S11a", "S12b",
            "S16c", "S17c", "S19a", "S20a", "S22b", "S24c", "S28a", "S29a", "S30c", "S31c",
            "S32c", "S33a", "S34c", "S35a", "S37a", "S38b", "S39b", "S41c", "S42a", "S43b",
            "S45a", "S46a", "S47", "S48", "S49a", "S49b", "S49c", "S50c", "S51a", "S52a",
            "S53a", "S55a", "S56b", "S58b", "S59b", "S60a", "S62c", "S63a", "S64b", "S65a",
            "S65b", "S65c", "S66c", "S68a", "S69a", "S70a", "S71a", "S73c", "S74c", "S75c",
            "S76a", "S76b", "S76c", "S77b", "S79a", "S80b", "S82a", "S83c", "S84c", "S85c",
            "S86b"
            ]
  
            



        # Read CSV file into list
        #load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).tolist()
        # Convert the first column to a list
        load_powers = pd.read_csv('D:\Alaa_Selim\Loadpowers.csv', header=None).iloc[:, 0].tolist()


        # Create a dictionary with load names as keys and load powers as values
        load_dict = dict(zip(load_names, load_powers))

# Generate load scales using a Gaussian distribution
        load_scales = np.random.normal(loc=self.load_profile[self.current_step % 8760], scale=0.001, size=len(load_names))

        for load_name, load_scale in zip(load_names, load_scales):
    # Get the load power corresponding to load_name from the dictionary
            load_power = load_dict[load_name]
    
    # Multiply load power by load_scale
            result = load_power * load_scale
    
    # Use the result in your controller
            self.controller.text(f"edit Load.{load_name} kW={result}")

            
            
        # Load the irradiance for the current hour
        
        
        irradiance = self.irradiance_profile[self.current_step % 8760]
        

    
        

        # Initialize random states for controlled devices
        for i, bus in enumerate(self.ranked_buses):
            # Randomly initialize PV systems and batteries
            #kvar should be set to very small
            fixed_power_factor = 0.9  # Set a fixed power factor value (e.g., 0.9)
            pv_kw = 100 * self.irradiance_profile[self.current_step % 8760]  # Scale PV kW by irradiance
            power_factor_angle = np.arccos(fixed_power_factor)  # Calculate the angle corresponding to the power factor
            pv_kvar = pv_kw * np.tan(power_factor_angle)  # Calculate reactive power (kVAR) based on kW and power factor
            battery_kw = self.load_profile[self.current_step % 8760]*100  # Scale Battery kW by load
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0") #select charging or discharging
            #make a comparison study beween different modes
            #investigate batteries charging/dicharging pattern




        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        
        # Get the observations for all agents
        observations = self.get_observation()

    # You can include any additional information here. If there's nothing, just return an empty dictionary.
        infos = {}

        return observations, infos


    def render(self, mode='human'):
        pass

    def close(self):
        pass

    def get_observation(self):
    # Get the voltage at all buses
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
    
    # Flatten the list of bus voltages
        all_bus_voltages = np.array(all_bus_voltages).flatten()
    
    # As there's only one agent, you should decide the observation length. 
    # If you'd like to keep the previous structure, the agent might observe the first 50, or all of them.
    # Here, I'm assuming the agent observes all of them:
        agent_observation = all_bus_voltages

        return agent_observation

    def _end_of_episode(self):
        return self.current_step >= len(self.irradiance_profile)

    

irradiance_csv_file = r"D:\Alaa_Selim\Irradiance_Profile_Santa_Clara.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"

    
    
original_env = PowerSystemEnv(
    dss_path=r"C:\Program Files\OpenDSS",
    dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file
)


#from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility

#env = MultiAgentEnvCompatibility(original_env)

def env_creator(env_config):
    return PowerSystemEnv(
                            dss_path=r"C:\Program Files\OpenDSS",
                            dss_file=r"D:\Alaa_Selim\123Bus\IEEE123Master.dss",
                            irradiance_csv_file=irradiance_csv_file,
                            load_profile_file=load_profile_file
                            )

register_env("PowerSystemEnv", env_creator)



# Suppose you have N agents
N = 5  # adjust this as per your actual number of agents

# Only one policy for the single agent
#policies = {
  #  "single_policy": (None, 
        #              original_env.observation_space, 
           #           original_env.action_space, 
                #      {})
#}#

#def policy_mapping_fn(agent_id, episode, **kwargs):
  #  return "single_policy"





